# Vérification de relations

L'objectif de ce cahier est d'identifier et de valider les relations entre variables qui permettent d'interpoler des valeurs manquantes ou de corriger des valeurs aberrantes.

Par exemple, dans la table **`credit_card_balance`**, les nombres (`CNT`) et montants (`AMT`) des retraits mensuels (`DRAWINGS`) sont donnés en total et en ventilation par rapport aux 3 classes `ATM`, `POS` et `OTHER`. Pour ces deux ensembles de 4 variables, le total est a priori la somme des 3 variables ventilées. La violation de cette relation permet de détecter des valeurs aberrantes, et la relation permet également de compléter certaines valeurs manquantes.

L'approche reste ici empirique et expérimentale, mais une amélioration consisterait à tester certaines librairies spécialisées sur cette tâche.

# **`credit_card_balance`**

Rappelons que cette table contient 3 840 312 enregistrements.

## Données, conventions, utilitaires

### Chargement de la table principale

In [153]:
from home_credit.check import load_credit_card_balance 
from home_credit.utils import display_frame_basic_infos

data = load_credit_card_balance()
data.info()
display_frame_basic_infos(data)

### Groupes de variables, et abréviations des noms longs

In [2]:
from home_credit.tables import CreditCardBalance

print(CreditCardBalance.cols_group("drawings_amt_cnt_couples"))
print(CreditCardBalance.cols_group("drawings_amt_cnt_couples", shorten=True))

['AMT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT']
[('DRW', 'TOT', 'AMT'), ('DRW', 'TOT', 'CNT'), ('DRW', 'ATM', 'AMT'), ('DRW', 'ATM', 'CNT'), ('DRW', 'POS', 'AMT'), ('DRW', 'POS', 'CNT'), ('DRW', 'OTH', 'AMT'), ('DRW', 'OTH', 'CNT')]


In [188]:
from home_credit.check import get_drawings_amt_cnt_couples_cols

print(get_drawings_amt_cnt_couples_cols())
print(get_drawings_amt_cnt_couples_cols(shorten=True))

['AMT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT']
['AMT_TOT', 'CNT_TOT', 'AMT_ATM', 'CNT_ATM', 'AMT_POS', 'CNT_POS', 'AMT_OTH', 'CNT_OTH']


### Chargement de la sous-table `DRAWINGS`

In [13]:
from home_credit.check import get_credit_card_balance_drawings

display(get_credit_card_balance_drawings())

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\credit_card_balance.pqt


,TGT,PID,CID,M°,AMT_TOT,CNT_TOT,AMT_ATM,CNT_ATM,AMT_POS,CNT_POS,AMT_OTH,CNT_OTH
0,-1,2562384,378907,6,877.5,1,0.0,0.0,877.5,1.0,0.0,0.0
1,1,2582071,363914,1,2250.0,1,2250.0,1.0,0.0,0.0,0.0,0.0
2,-1,1740877,371185,7,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1389973,337855,4,2250.0,1,2250.0,1.0,0.0,0.0,0.0,0.0
4,0,1891521,126868,1,11547.0,1,0.0,0.0,11547.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,0,1036507,328243,9,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
3840308,0,1714892,347207,9,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3840309,0,1302323,215757,9,270000.0,2,270000.0,2.0,0.0,0.0,0.0,0.0
3840310,0,1624872,430337,10,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN


## Relations entre les NA

3 quantités synchronisées :
- 305 236 (8 %) sur `AMT_INST_MIN_REGULARITY` et `CNT_INSTALMENT_MATURE_CUM`
- 749 816 (19,5 %) sur `AMT_DRAWINGS_ATM_CURRENT`, `AMT_DRAWINGS_OTHER_CURRENT`, `AMT_DRAWINGS_POS_CURRENT`, `CNT_DRAWINGS_ATM_CURRENT`, `CNT_DRAWINGS_OTHER_CURRENT`, `CNT_DRAWINGS_POS_CURRENT`
- 767 988 (20 %) sur `AMT_PAYMENT_CURRENT`

## Groupe des 8 variables **`DRAWINGS`**

La relation, pour `AMT` et `CNT`, est que la variable qui représente le total doit égaler la somme des parties détaillées dans les variables `ATM`, `POS` et `OTHER`.

On commence par en extraire le sous-ensemble, puis par calculer les sommes des deux ventilations `AMT` et `CNT`, puis les comparer avec la variable de synthèse.

La majorité des enregistrements vérifient la relation :
- 80 % la vérifient parfaitement, tant pour `AMT` que pour `CNT`.
- 19.5 % ne la vérifient pas pour cause de présence de NA (données manquantes).
- Seuls 7 150 cas demandent un retraitement des données (moins de 2 pour 1000).

In [16]:
drawings = get_credit_card_balance_drawings()

drawings["AMT_SUM"] = drawings.AMT_ATM + drawings.AMT_POS + drawings.AMT_OTH
drawings["CNT_SUM"] = drawings.CNT_ATM + drawings.CNT_POS + drawings.CNT_OTH

drawings["AMT_DIFF"] = (drawings.AMT_TOT - drawings.AMT_SUM).round(2)
drawings["CNT_DIFF"] = drawings.CNT_TOT - drawings.CNT_SUM

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\credit_card_balance.pqt


### Nombre d'enregistrements qui vérifient que le total égale la somme des parties

80 % des enregistrements vérifient la relation (une fraction de 0.2 % la vérifie pour `CNT` et non pour `AMT`).

In [29]:
n = drawings.shape[0]
is_amt_ok = drawings.AMT_DIFF == 0
is_cnt_ok = drawings.CNT_DIFF == 0
n_full_ok = sum(is_amt_ok & is_cnt_ok)
n_amt_ok = sum(is_amt_ok)
n_cnt_ok = sum(is_cnt_ok)
display(drawings[is_amt_ok].head(2))
print(f"#(AMT_DIFF = 0 & CNT_DIFF = 0): {n_full_ok} ({100*(n_full_ok/n):.2f} %)")
print(f"#(AMT_DIFF = 0): {n_amt_ok} ({100*(n_amt_ok/n):.2f} %)")
print(f"#(CNT_DIFF = 0): {n_cnt_ok} ({100*(n_cnt_ok/n):.2f} %)")

,TGT,PID,CID,M°,AMT_TOT,CNT_TOT,AMT_ATM,CNT_ATM,AMT_POS,CNT_POS,AMT_OTH,CNT_OTH,AMT_SUM,CNT_SUM,AMT_DIFF,CNT_DIFF
0,-1,2562384,378907,6,877.5,1,0.0,0.0,877.5,1.0,0.0,0.0,877.5,1.0,0.0,0.0
1,1,2582071,363914,1,2250.0,1,2250.0,1.0,0.0,0.0,0.0,0.0,2250.0,1.0,0.0,0.0


#(AMT_DIFF = 0 & CNT_DIFF = 0): 3083346 (80.29 %)
#(AMT_DIFF = 0): 3083346 (80.29 %)
#(CNT_DIFF = 0): 3090496 (80.48 %)


749 816 cas sont NA, et ils le sont alors simultanément pour `AMT` et `CNT` :

In [28]:
is_amt_diff_na = drawings.AMT_DIFF.isna()
is_cnt_diff_na = drawings.CNT_DIFF.isna()
n_full_diff_na = sum(is_amt_diff_na & is_cnt_diff_na)
n_amt_diff_na = sum(is_amt_diff_na)
n_cnt_diff_na = sum(is_cnt_diff_na)
display(drawings[is_amt_diff_na].head(2))
print(f"#(AMT_DIFF is NA): {n_amt_diff_na} ({100*(n_amt_diff_na/n):.2f} %)")
print(f"#(AMT_DIFF is NA & CNT_DIFF is NA): {n_full_diff_na} ({100*(n_full_diff_na/n):.2f} %)")
print(f"#(AMT_DIFF is NA): {n_amt_diff_na} ({100*(n_amt_diff_na/n):.2f} %)")
print(f"#(CNT_DIFF is NA): {n_cnt_diff_na} ({100*(n_cnt_diff_na/n):.2f} %)")

,TGT,PID,CID,M°,AMT_TOT,CNT_TOT,AMT_ATM,CNT_ATM,AMT_POS,CNT_POS,AMT_OTH,CNT_OTH,AMT_SUM,CNT_SUM,AMT_DIFF,CNT_DIFF
45,0,2657726,399970,5,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,-1,1517613,121258,6,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#(AMT_DIFF is NA): 749816 (19.52 %)
#(AMT_DIFF is NA & CNT_DIFF is NA): 749816 (19.52 %)
#(AMT_DIFF is NA): 749816 (19.52 %)
#(CNT_DIFF is NA): 749816 (19.52 %)


Dans tous les cas (7 150) où `AMT_DIFF != 0`, alors `CNT_DIFF == 0` : en d'autres termes, quand `CNT_DIFF` n'est pas NA, il est nul.

Dans la plupart des cas, on a également `CNT_TOT == 0` : si la ventilation des montants n'est pas fournie, les nombres de retraits sont dénombrés à 0. La seule information exacte est alors `AMT_TOT`.

Mais dans 167 cas, `CNT_TOT > 0` : cela correspond à des cas où la ventilation est en partie effectuée, mais où elle est incomplète : on vérifie alors que (à l'exception d'un seul cas aberrant), que le total de contrôle des montants majore toujours la somme des montants ventilés.

In [42]:
is_amt_diff = (drawings.AMT_DIFF < 0) | (drawings.AMT_DIFF > 0)
is_cnt_diff_notna_and_not_0 = drawings.CNT_DIFF.notna() & (drawings.CNT_DIFF < 0) | (drawings.CNT_DIFF > 0)
print("Number of violations of 'CNT_DIFF is NA or == 0':", sum(is_cnt_diff_notna_and_not_0))
print(f"Standard AMT_DIFF ({sum(is_amt_diff)}):")
display(drawings[is_amt_diff].head(2))
is_cnt_tot_pos = drawings.CNT_TOT > 0
print(f"Special AMT_DIFF with CNT_TOT > 0 ({sum(is_amt_diff & is_cnt_tot_pos)}):")
display(drawings[is_amt_diff & is_cnt_tot_pos].head(2))

Number of violations of 'CNT_DIFF is NA or == 0': 0
Standard AMT_DIFF (7150):


,TGT,PID,CID,M°,AMT_TOT,CNT_TOT,AMT_ATM,CNT_ATM,AMT_POS,CNT_POS,AMT_OTH,CNT_OTH,AMT_SUM,CNT_SUM,AMT_DIFF,CNT_DIFF
53,0,1894367,113120,1,2145.780,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2145.78,0.0
650,0,2155865,344228,5,27003.645,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27003.64,0.0


Special AMT_DIFF with CNT_TOT > 0 (167):


,TGT,PID,CID,M°,AMT_TOT,CNT_TOT,AMT_ATM,CNT_ATM,AMT_POS,CNT_POS,AMT_OTH,CNT_OTH,AMT_SUM,CNT_SUM,AMT_DIFF,CNT_DIFF
54301,0,1845922,152839,1,1780.29,1,0.0,0.0,769.50,1.0,0.0,0.0,769.50,1.0,1010.79,0.0
69599,0,1096377,132325,6,53574.12,10,0.0,0.0,41127.12,10.0,0.0,0.0,41127.12,10.0,12447.00,0.0


Dans la majorité des cas, si `AMT_DIFF != 0` alors `AMT_DIFF > 0` :

In [52]:
display(sum(drawings.AMT_DIFF > 0))

7147

Dans 3 cas, il est négatif : dans de tels cas, on impute le montant sur `ATM` qui est le seul à permettre des retraits négatifs.

In [41]:
display(drawings[(drawings.AMT_DIFF < 0)])

,TGT,PID,CID,M°,AMT_TOT,CNT_TOT,AMT_ATM,CNT_ATM,AMT_POS,CNT_POS,AMT_OTH,CNT_OTH,AMT_SUM,CNT_SUM,AMT_DIFF,CNT_DIFF
438776,-1,2321147,198229,64,-1687.50,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1687.50,0.0
747302,0,2340075,114867,73,-519.57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-519.57,0.0
3284667,0,2592574,217360,96,-6211.62,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6211.62,0.0


Dans la majorité des cas, si `AMT_DIFF != 0` alors `AMT_SUM = 0` :

In [47]:
print(sum(is_amt_diff & (drawings.AMT_SUM == 0)))

6983


Les cas où `AMT_SUM != 0` identifient comme `CNT_TOT > 0` les 167 cas de ventilation partielle.

In [49]:
display(drawings[is_amt_diff & ((drawings.AMT_SUM < 0) | (drawings.AMT_SUM > 0))])

,TGT,PID,CID,M°,AMT_TOT,CNT_TOT,AMT_ATM,CNT_ATM,AMT_POS,CNT_POS,AMT_OTH,CNT_OTH,AMT_SUM,CNT_SUM,AMT_DIFF,CNT_DIFF
54301,0,1845922,152839,1,1780.290,1,0.0,0.0,769.500,1.0,0.0,0.0,769.500,1.0,1010.79,0.0
69599,0,1096377,132325,6,53574.120,10,0.0,0.0,41127.120,10.0,0.0,0.0,41127.120,10.0,12447.00,0.0
91165,-1,1961855,441577,19,18528.165,1,18000.0,1.0,0.000,0.0,0.0,0.0,18000.000,1.0,528.17,0.0
117513,0,2448064,422458,11,1269.990,2,0.0,0.0,904.050,2.0,0.0,0.0,904.050,2.0,365.94,0.0
123380,0,1007117,356088,6,4076.415,1,0.0,0.0,3046.500,1.0,0.0,0.0,3046.500,1.0,1029.92,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3661192,0,1305048,253519,7,8085.015,1,6300.0,1.0,0.000,0.0,0.0,0.0,6300.000,1.0,1785.02,0.0
3667457,0,1251827,369312,1,94554.990,24,10350.0,1.0,74698.650,23.0,0.0,0.0,85048.650,24.0,9506.34,0.0
3722530,0,1912727,422822,35,292863.330,4,292500.0,4.0,0.000,0.0,0.0,0.0,292500.000,4.0,363.33,0.0
3796081,0,1591648,298981,6,13299.075,2,0.0,0.0,12187.575,2.0,0.0,0.0,12187.575,2.0,1111.50,0.0


Il n'existe qu'un seul cas `PID = 1793522` et `M° = 20` pour lequel `AMT_SUM < 0`.

In [51]:
display(drawings[is_amt_diff & (drawings.AMT_SUM < 0)])
# display(drawings[(drawings.PID == 1793522) & (drawings["M°"] == 20)])

,TGT,PID,CID,M°,AMT_TOT,CNT_TOT,AMT_ATM,CNT_ATM,AMT_POS,CNT_POS,AMT_OTH,CNT_OTH,AMT_SUM,CNT_SUM,AMT_DIFF,CNT_DIFF
2047409,0,1793522,317928,20,0.0,1,-6827.31,1.0,0.0,0.0,0.0,0.0,-6827.31,1.0,6827.31,0.0


### Elaboration de l'heuristique d'imputation du groupe `DRAWINGS`

#### Stratégies

Plusieurs stratégies d'interpolation se dégagent qui peuvent être utilisées conjointement :
1. les montants de retraits négatifs sont nécessairement des `ATM`.
2. le client a probablement un moyen de payement préféré et un autre qu'il n'utilise jamais.
3. l'analyse modulaire du montant prélevé est un indicateur : les montants ATM sont entiers à 99.7 % et les POS seulement à 7%.

Nous laissons au lecteur qui souhaiterait améliorer la solution le soin d'explorer la stratégie 2. Pour cette version, nous allons utiliser la stratégie 3.

#### Moyens de paiement les plus utilisés

Fréquence d'emploi des trois moyens de payement (`ATM`, `POS`, `OTHER`).

`ATM` est le plus employé par les clients, mais ceux qui emploient le `POS` l'emploient plus fréquemment chaque mois que l'`ATM` (6 retraits mensuels contre 2).

In [111]:
import pandas as pd
n = drawings.shape[0]
atm = drawings.CNT_ATM
pos = drawings.CNT_POS
oth = drawings.CNT_OTH
atm = atm[atm.notna() & (atm != 0)]
pos = pos[pos.notna() & (pos != 0)]
oth = oth[oth.notna() & (oth != 0)]
n_months_atm = atm.shape[0]
n_months_pos = pos.shape[0]
n_months_oth = oth.shape[0]
n_atm = int(sum(atm))
n_pos = int(sum(pos))
n_oth = int(sum(oth))
m = n_atm + n_pos + n_oth
display(pd.DataFrame(
    [
        (n_months_atm, n_atm, round(n_atm/n_months_atm, 2), round(100*n_months_atm/n), round(100*n_atm/m)),
        (n_months_pos, n_pos, round(n_pos/n_months_pos, 2), round(100*n_months_pos/n), round(100*n_pos/m)),
        (n_months_oth, n_oth, round(n_atm/n_months_oth, 2), round(100*n_months_oth/n), round(100*n_oth/m))        
    ],
    columns=["n_months", "n", "n / n_months", "n_months %", "n %"],
    index=["ATM", "POS", "OTH"]
))

,n_months,n,n / n_months,n_months %,n %
ATM,424778,956351,2.25,11,35
POS,264902,1729068,6.53,7,64
OTH,12808,14873,74.67,0,1


#### Distribution des montants sur les trois moyens

In [87]:
import pandas as pd
atm = drawings.AMT_ATM
pos = drawings.AMT_POS
oth = drawings.AMT_OTH
atm = atm[atm.notna() & (atm != 0)]
pos = pos[pos.notna() & (pos != 0)]
oth = oth[oth.notna() & (oth != 0)]
display(pd.DataFrame(atm.value_counts()).T)
display(pd.DataFrame(pos.value_counts()).T)
display(pd.DataFrame(oth.value_counts()).T)

,4500.000,9000.000,45000.000,2250.000,22500.000,13500.000,6750.000,18000.000,90000.000,67500.000,...,101438.010,123399.360,4547.205,600750.000,935.505,72720.000,85725.000,43984.215,44134.830,33538.725
AMT_ATM,35851,27726,22946,22854,22676,21198,14712,13318,12034,10765,...,1,1,1,1,1,1,1,1,1,1


,450.000,900.000,2250.000,4500.000,1350.000,225.000,45000.000,1800.000,2700.000,675.000,...,4628.700,55823.355,7103.250,112869.585,9499.995,7727.085,155533.005,9367.425,47345.850,42374.250
AMT_POS,1287,976,910,801,784,609,462,407,379,377,...,1,1,1,1,1,1,1,1,1,1


,3343.500,6682.500,4455.000,8910.000,46800.000,4680.000,9360.000,23400.000,5571.000,14040.000,...,66881.250,36270.000,67635.000,40975.020,27430.425,24165.000,285480.000,22698.000,37872.000,23755.500
AMT_OTH,782,460,441,319,242,235,216,209,195,172,...,1,1,1,1,1,1,1,1,1,1


In [88]:
display(pd.DataFrame((atm % 25).value_counts(normalize=True)).T)
display(pd.DataFrame((pos % 25).value_counts(normalize=True)).T)
display(pd.DataFrame((oth % 25).value_counts(normalize=True)).T)

,0.000,10.000,15.000,5.000,20.000,22.500,17.500,13.500,11.250,6.750,...,23.380,12.115,17.605,6.195,10.765,5.910,9.810,2.545,2.245,13.725
AMT_ATM,0.997276,0.000271,0.000271,0.000271,0.000261,0.000014,0.000012,0.000009,0.000007,0.000007,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002


,0.000,5.000,10.000,15.000,20.500,20.000,16.000,0.500,2.500,7.500,...,15.485,3.590,21.470,15.815,21.195,22.575,1.525,15.645,10.470,12.205
AMT_POS,0.070158,0.024164,0.014957,0.012608,0.012514,0.011533,0.007818,0.006535,0.006138,0.005387,...,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004


,0.000,5.000,10.000,15.000,20.000,18.500,7.500,21.000,12.500,3.000,...,21.100,4.525,4.125,9.555,20.300,3.465,0.020,13.985,8.275,12.355
AMT_OTH,0.289711,0.122274,0.112706,0.079867,0.070137,0.06438,0.041596,0.021082,0.011271,0.008352,...,0.000081,0.000081,0.000081,0.000081,0.000081,0.000081,0.000081,0.000081,0.000081,0.000081


In [98]:
display(pd.DataFrame((atm % 1).value_counts(normalize=True)).T)
display(pd.DataFrame((pos % 1).value_counts(normalize=True)).T)
display(pd.DataFrame((oth % 1).value_counts(normalize=True)).T)

,0.000,0.500,0.250,0.625,0.750,0.125,0.090,0.465,0.380,0.780,...,0.265,0.380,0.405,0.685,0.935,0.850,0.075,0.300,0.730,0.725
AMT_ATM,0.998387,0.000061,0.000021,0.000019,0.000019,0.000014,0.000012,0.000012,0.000012,0.000012,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002


,0.000,0.500,0.250,0.750,0.375,0.100,0.875,0.550,0.600,0.700,...,0.455,0.290,0.225,0.495,0.370,0.910,0.030,0.545,0.215,0.205
AMT_POS,0.220596,0.118584,0.019819,0.019437,0.003749,0.003711,0.003666,0.003658,0.003654,0.003643,...,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004


,0.000,0.500,0.250,0.750,0.800,0.800,0.600,0.625,0.900,0.600,...,0.555,0.900,0.645,0.460,0.595,0.275,0.020,0.555,0.100,0.710
AMT_OTH,0.790238,0.167032,0.008514,0.006324,0.00073,0.000649,0.000649,0.000649,0.000568,0.000568,...,0.000081,0.000081,0.000081,0.000081,0.000081,0.000081,0.000081,0.000081,0.000081,0.000081


Les min et max

In [90]:
print(f"ATM min {min(atm)}, max {max(atm)}")
print(f"POS min {min(pos)}, max {max(pos)}")
print(f"OTH min {min(oth)}, max {max(oth)}")

ATM min -6827.31, max 2115000.0
POS min 0.045, max 2239274.16
OTH min 234.0, max 1529847.0


### Réalisation de la fonction d'imputation

In [151]:
import numpy as np

drawings = get_credit_card_balance_drawings()
drawings["AMT_SUM"] = drawings.AMT_ATM + drawings.AMT_POS + drawings.AMT_OTH
drawings["CNT_SUM"] = drawings.CNT_ATM + drawings.CNT_POS + drawings.CNT_OTH

drawings["AMT_DIFF"] = (drawings.AMT_TOT - drawings.AMT_SUM).round(2)
drawings["CNT_DIFF"] = drawings.CNT_TOT - drawings.CNT_SUM

# On isole les cas problématiques 749 816 NA et 7 150 diffs non nuls
is_outlier = drawings.AMT_DIFF != 0
print("# Outliers:", sum(is_outlier))

# Pour les 749 816 cas NA (`AMT_TOT = 0`) : fillna avec 0
is_na = drawings.AMT_DIFF.isna()
print("# NA:", sum(is_na))
drawings.fillna(0, inplace=True)

# Pour les 7 150 cas non NA
is_notna = is_outlier & ~is_na
print("# not NA:", sum(is_notna))

# Si le montant est négatif, c'est un ATM
is_tot_negative = is_notna & (drawings.AMT_TOT < 0)
print("# (AMT_TOT < 0):", sum(is_tot_negative))
updated_cols = ["CNT_ATM", "CNT_TOT", "AMT_DIFF", "AMT_ATM"]
drawings.loc[is_tot_negative, updated_cols] = \
    np.array((1, 1, 0, drawings[is_tot_negative].AMT_TOT), dtype=object)

# Sinon, on affecte le montant sur AMT_ATM si AMT_TOT est entier, et sur AMT_POS sinon
is_tot_positive = is_notna & drawings.AMT_TOT > 0
is_tot_integer = (drawings.AMT_TOT % 1) == 0
is_pos_int = is_tot_positive & is_tot_integer
print("# (AMT_TOT > 0) & int:", sum(is_pos_int))
is_pos_float = is_tot_positive & ~is_tot_integer
print("# (AMT_TOT > 0) & float:", sum(is_pos_float))
drawings.loc[is_pos_int, ["CNT_ATM", "AMT_ATM"]] = \
    np.array((1, drawings[is_pos_int].AMT_TOT), dtype=object)
drawings.loc[is_pos_float, ["CNT_POS", "AMT_POS"]] = \
    np.array((1, drawings[is_pos_float].AMT_TOT), dtype=object)
drawings.loc[is_tot_positive, ["CNT_TOT", "AMT_DIFF"]] = 1, 0

# display(drawings[is_na])
# display(drawings[is_tot_negative])
# display(drawings[is_pos_int])
# display(drawings[is_pos_float])

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\credit_card_balance.pqt
# Outliers: 756966
# NA: 749816
# not NA: 7150
# (AMT_TOT < 0): 3
# (AMT_TOT > 0) & int: 669
# (AMT_TOT > 0) & float: 6480


### Version intégrée

In [152]:
from home_credit.impute import impute_credit_card_balance_drawings
data = load_credit_card_balance()
impute_credit_card_balance_drawings(data)
display(data)

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\credit_card_balance.pqt


RAW_CREDIT_CARD_BALANCE,TARGET,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,-1,2562384,378907,6,56.970,135000,0.0,877.5,0.0,877.5,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,1,2582071,363914,1,63975.555,45000,2250.0,2250.0,0.0,0.0,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,-1,1740877,371185,7,31815.225,450000,0.0,0.0,0.0,0.0,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,0,1389973,337855,4,236572.110,225000,2250.0,2250.0,0.0,0.0,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,0,1891521,126868,1,453919.455,450000,0.0,11547.0,0.0,11547.0,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,0,1036507,328243,9,0.000,45000,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0,0.0,0.0,0.0,Active,0,0
3840308,0,1714892,347207,9,0.000,45000,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0,0.0,0.0,23.0,Active,0,0
3840309,0,1302323,215757,9,275784.975,585000,270000.0,270000.0,0.0,0.0,...,273093.975,273093.975,2.0,2,0.0,0.0,18.0,Active,0,0
3840310,0,1624872,430337,10,0.000,450000,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0,0.0,0.0,0.0,Active,0,0


## Groupe des 2 variables **`PAYMENT`**

Les deux variables **`AMT_PAYMENT_CURRENT`** et **`AMT_PAYMENT_TOTAL_CURRENT`** sont liées, mais la première comporte 305 236 valeurs manquantes (20 %).

D'après les définitions :
* **`PYT = AMT_PAYMENT_CURRENT`** : montant payé par le client au cours du dernier mois.
* **`PYT_TOT = AMT_PAYMENT_TOTAL_CURRENT`** : montant total payé par le client au cours du dernier mois.

Ces définitions ne permettent pas vraiment de comprendre la distinction entre les deux.

La relation entre ces deux variables a déjà été étudiée dans le cahier **`ea_credit_card_balance`**, section _Relation entre `PYT` et `PYT_TOT`_, dont nous reprenons certains éléments sans les détails d'exploration.

Pour cette étude, nous avons trouvé pertinent d'étudier la différence et le rapport entre les deux variables (le ratio de leur différence ramenée à celle des deux qui ne s'annule pas, `PYT`).

Relations observées :
- `PYT` $\ge$ `PYT_TOT` (100 % des non NA).
- `PYT` $=$ `PYT_TOT` (61 % des non NA).
- Parmi les 39 % de cas d'inégalité :
    - `PYT` $\ne 0$  et `PYT_TOT` est souvent nul (85 % des cas inégaux).
- Si `PYT` est NA alors `PYT_TOT` $= 0$ (la réciproque n'est pas vraie).

### Chargement de la sous-table `PAYMENT`

In [160]:
from home_credit.check import get_credit_card_balance_payment

payment = get_credit_card_balance_payment()
display(payment)

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\credit_card_balance.pqt


,TGT,PID,CID,M°,PYT,PYT_TOT
0,-1,2562384,378907,6,1800.00,1800.000
1,1,2582071,363914,1,2250.00,2250.000
2,-1,1740877,371185,7,2250.00,2250.000
3,0,1389973,337855,4,11925.00,11925.000
4,0,1891521,126868,1,27000.00,27000.000
...,...,...,...,...,...,...
3840307,0,1036507,328243,9,NaN,0.000
3840308,0,1714892,347207,9,1879.11,0.000
3840309,0,1302323,215757,9,375750.00,356994.675
3840310,0,1624872,430337,10,NaN,0.000


In [161]:
payment["DIFF"] = payment.PYT - payment.PYT_TOT
payment["%DIFF"] = payment.DIFF / payment.PYT

### Part des NA

In [162]:
n = payment.shape[0]
n_pyt_na = sum(payment.PYT.isna())
n_pyt_tot_na = sum(payment.PYT_TOT.isna())
print(f"n_pyt_na: {n_pyt_na} ({100*n_pyt_na/n:.2f} %)")
print(f"n_pyt_na: {n_pyt_tot_na} ({100*n_pyt_tot_na/n:.2f} %)")

n_pyt_na: 767988 (20.00 %)
n_pyt_na: 0 (0.00 %)


### `PYT` $=$ `PYT_TOT` (61 % des non NA)

In [166]:
m = n - n_pyt_na
n_eq = sum(payment.PYT == payment.PYT_TOT)
print(f"n_eq: {n_eq} ({100*n_eq/m:.2f} %)")

n_eq: 1885092 (61.36 %)


### `PYT` $\ge$ `PYT_TOT` (61 % des non NA)

In [168]:
n_ge = sum(payment.PYT >= payment.PYT_TOT)
print(f"n_ge: {n_ge} ({100*n_ge/m:.2f} %)")

n_ge: 3072324 (100.00 %)


### Parmi les 39 % de cas d'inégalité<br/>`PYT` $\ne 0$  et `PYT_TOT` est souvent nul (85 % des cas inégaux)

In [173]:
gt_case = payment.PYT > payment.PYT_TOT
is_pyt_nul = payment.PYT == 0
is_pyt_tot_nul = payment.PYT_TOT == 0
n_pyt_nul = sum(gt_case & is_pyt_nul)
n_pyt_tot_nul = sum(gt_case & is_pyt_tot_nul)
p = sum(gt_case)
print(f"n_pyt_nul: {n_pyt_nul} ({100*n_pyt_nul/p:.2f} %)")
print(f"n_pyt_tot_nul: {n_pyt_tot_nul} ({100*n_pyt_tot_nul/p:.2f} %)")

n_pyt_nul: 0 (0.00 %)
n_pyt_tot_nul: 1013728 (85.39 %)


### Si `PYT` est NA alors `PYT_TOT` $= 0$

In [174]:
is_pyt_na = payment.PYT.isna()
n_pyt_na = sum(is_pyt_na)
is_pyt_na_and_pyt_tot_null = is_pyt_na & is_pyt_tot_nul
is_pyt_na_and_pyt_tot_not_null = is_pyt_na & ~is_pyt_tot_nul
n_null = sum(is_pyt_na_and_pyt_tot_null)
n_not_null = sum(is_pyt_na_and_pyt_tot_not_null)
print(f"n_pyt_tot_nul: {n_null} ({100*n_null/n_pyt_na:.2f} %)")
print(f"n_pyt_tot_not_nul: {n_not_null} ({100*n_not_null/n_pyt_na:.2f} %)")

n_pyt_tot_nul: 767988 (100.00 %)
n_pyt_tot_not_nul: 0 (0.00 %)


##  **`BALANCE`**, **`DRAWINGS`** et **`PAYMENT`**

Ce peut-être la clé pour débloquer les NA.

### Chargement de la sous-table `BALANCE`

In [189]:
from home_credit.check import get_credit_card_balance_payment_balance

balance = get_credit_card_balance_payment_balance()
display(balance)

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\credit_card_balance.pqt


,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT
0,-1,2562384,378907,6,56.970,877.5,0.000,0.000,1800.00,1800.000
1,1,2582071,363914,1,63975.555,2250.0,60175.080,64875.555,2250.00,2250.000
2,-1,1740877,371185,7,31815.225,0.0,26926.425,31460.085,2250.00,2250.000
3,0,1389973,337855,4,236572.110,2250.0,224949.285,233048.970,11925.00,11925.000
4,0,1891521,126868,1,453919.455,11547.0,443044.395,453919.455,27000.00,27000.000
...,...,...,...,...,...,...,...,...,...,...
3840307,0,1036507,328243,9,0.000,0.0,0.000,0.000,NaN,0.000
3840308,0,1714892,347207,9,0.000,0.0,0.000,0.000,1879.11,0.000
3840309,0,1302323,215757,9,275784.975,270000.0,269356.140,273093.975,375750.00,356994.675
3840310,0,1624872,430337,10,0.000,0.0,0.000,0.000,NaN,0.000


### Commençons par quelques cas clients

Il ne faut pas oublier que les mois vont du présent au passé.

Certain prêts sont complètement vides, sans même des frais.

C'est un peu bête de les conserver, cela fait de nombreuses lignes pour rien.

Exemples : 328243

On pourrait se contenter d'un indicateur dans un coin.

Il y a des cas où les données sont clairement fausses : 347207, voir la fin.

En fait, il m'est encore difficile de trouver une logique stable qui se tient et partant de laquelle je peux identifier et compenser les erreurs.

In [190]:
display(balance[balance.CID == 378907].sort_values(by="M°", ascending=False))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT
1811769,-1,2562384,378907,48,69186.690,67500.000,67500.000,67841.190,0.000,0.000
2584127,-1,2562384,378907,47,62727.075,0.000,60979.410,63246.825,4500.000,4500.000
3284876,-1,2562384,378907,46,62577.900,0.000,60796.035,63082.080,3600.000,3600.000
2653597,-1,2562384,378907,45,61484.400,0.000,59734.170,61980.255,3600.000,3600.000
1039133,-1,2562384,378907,44,60736.680,0.000,59078.160,61224.345,3375.000,3375.000
2021610,-1,2562384,378907,43,59786.865,0.000,58095.090,60267.735,3375.000,3375.000
1764744,-1,2562384,378907,42,58743.675,0.000,57133.755,59216.985,3375.000,3375.000
2239489,-1,2562384,378907,41,57725.595,0.000,56079.270,58191.525,3375.000,3375.000
2648695,-1,2562384,378907,40,51462.360,0.000,49873.140,51920.145,8775.000,8775.000
561468,-1,2562384,378907,39,56570.940,0.000,53023.140,56570.940,0.000,0.000


In [214]:
from home_credit.check import analyze_loan

In [209]:
display(analyze_loan(378907))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
1811769,-1,2562384,378907,48,69186.690,67500.000,67500.000,67841.190,0.000,0.000,NaN,1345.500,341.190,NaN,NaN,67500.000
2584127,-1,2562384,378907,47,62727.075,0.000,60979.410,63246.825,4500.000,4500.000,-6459.615,-519.750,2267.415,67841.190,0.033422,-4500.000
3284876,-1,2562384,378907,46,62577.900,0.000,60796.035,63082.080,3600.000,3600.000,-149.175,-504.180,2286.045,63246.825,0.036145,-3600.000
2653597,-1,2562384,378907,45,61484.400,0.000,59734.170,61980.255,3600.000,3600.000,-1093.500,-495.855,2246.085,63082.080,0.035606,-3600.000
1039133,-1,2562384,378907,44,60736.680,0.000,59078.160,61224.345,3375.000,3375.000,-747.720,-487.665,2146.185,61980.255,0.034627,-3375.000
2021610,-1,2562384,378907,43,59786.865,0.000,58095.090,60267.735,3375.000,3375.000,-949.815,-480.870,2172.645,61224.345,0.035487,-3375.000
1764744,-1,2562384,378907,42,58743.675,0.000,57133.755,59216.985,3375.000,3375.000,-1043.190,-473.310,2083.230,60267.735,0.034566,-3375.000
2239489,-1,2562384,378907,41,57725.595,0.000,56079.270,58191.525,3375.000,3375.000,-1018.080,-465.930,2112.255,59216.985,0.035670,-3375.000
2648695,-1,2562384,378907,40,51462.360,0.000,49873.140,51920.145,8775.000,8775.000,-6263.235,-457.785,2047.005,58191.525,0.035177,-8775.000
561468,-1,2562384,378907,39,56570.940,0.000,53023.140,56570.940,0.000,0.000,5108.580,0.000,3547.800,51920.145,0.068332,0.000


In [210]:
display(analyze_loan(337855))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
2205888,0,1389973,337855,15,12286.125,12286.125,12286.125,12286.125,NaN,0.000,NaN,0.000,0.000,NaN,NaN,12286.125
632892,0,1389973,337855,14,37020.060,26309.250,37020.060,37020.060,2250.000,0.000,24733.935,0.000,0.000,12286.125,0.000000,26309.250
2450591,0,1389973,337855,13,46664.775,8287.515,44712.270,45880.065,595.305,595.305,9644.715,784.710,1167.795,37020.060,0.031545,7692.210
1520477,0,1389973,337855,12,161371.800,113164.605,156709.845,158542.470,3029.130,3029.130,114707.025,2829.330,1832.625,45880.065,0.039944,110135.475
3795788,0,1389973,337855,11,235819.980,72448.290,224704.125,229646.340,8792.685,8792.685,74448.180,6173.640,4942.215,158542.470,0.031173,63655.605
3000084,0,1389973,337855,10,235948.185,2852.550,224766.900,232436.295,13192.650,13192.650,128.205,3511.890,7669.395,229646.340,0.033397,-10340.100
2642364,0,1389973,337855,9,233909.820,2002.500,224894.475,231732.630,11804.670,11804.670,-2038.365,2177.190,6838.155,232436.295,0.029419,-9802.170
3017411,0,1389973,337855,8,229996.350,0.000,221054.985,227848.725,11765.610,11765.610,-3913.470,2147.625,6793.740,231732.630,0.029317,-11765.610
2208754,0,1389973,337855,7,226194.120,0.000,217253.745,224075.790,11475.000,11475.000,-3802.230,2118.330,6822.045,227848.725,0.029941,-11475.000
2563691,0,1389973,337855,6,237884.130,11250.000,224948.295,231670.035,11250.000,11250.000,11690.010,6214.095,6721.740,224075.790,0.029998,0.000


In [211]:
display(analyze_loan(126868))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
964490,0,1891521,126868,96,88487.235,0.000,85734.495,88487.235,4500.0,4500.0,NaN,0.0,2752.740,NaN,NaN,-4500.000
628178,0,1891521,126868,95,86285.655,4048.200,83535.435,86285.655,9000.0,9000.0,-2201.580,0.0,2750.220,88487.235,0.031080,-4951.800
3063006,0,1891521,126868,94,82088.640,0.000,79535.655,82088.640,6750.0,6750.0,-4197.015,0.0,2552.985,86285.655,0.029588,-6750.000
736819,0,1891521,126868,93,93788.325,14430.150,89768.790,93788.325,6750.0,6750.0,11699.685,0.0,4019.535,82088.640,0.048966,7680.150
1901497,0,1891521,126868,92,89775.990,0.000,87038.325,89775.990,6750.0,6750.0,-4012.335,0.0,2737.665,93788.325,0.029190,-6750.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56769,0,1891521,126868,5,459959.130,17469.000,448461.720,459959.130,27000.0,27000.0,1966.410,0.0,11497.410,457992.720,0.025104,-9531.000
2611945,0,1891521,126868,4,457896.285,13675.500,446634.630,457896.285,27000.0,27000.0,-2062.845,0.0,11261.655,459959.130,0.024484,-13324.500
728020,0,1891521,126868,3,461330.505,18000.000,448896.285,461330.505,27000.0,27000.0,3434.220,0.0,12434.220,457896.285,0.027155,-9000.000
1701346,0,1891521,126868,2,458497.395,13113.405,447443.910,458497.395,27000.0,27000.0,-2833.110,0.0,11053.485,461330.505,0.023960,-13886.595


Cas tordu qui montre à quel point les données suivies peuvent être erronées.

In [217]:
loan = analyze_loan(347207)
display(loan[:21])
display(loan[21:42])
display(loan[42:63])
display(loan[63:])

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
3187071,0,1714892,347207,84,0.000,0.0,0.000,0.000,0.000,0.0,NaN,0.000,0.000,NaN,NaN,0.0
1541206,0,1714892,347207,83,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,NaN,0.0
1389604,0,1714892,347207,82,24152.085,22500.0,22500.000,24152.085,0.000,0.0,24152.085,0.000,1652.085,0.000,inf,22500.0
1469595,0,1714892,347207,81,21482.055,0.0,19652.085,21482.055,4500.000,4500.0,-2670.030,0.000,1829.970,24152.085,0.075769,-4500.0
1667337,0,1714892,347207,80,55869.120,36000.0,52982.055,55869.120,4500.000,4500.0,34387.065,0.000,2887.065,21482.055,0.134394,31500.0
3744754,0,1714892,347207,79,53963.100,0.0,51369.120,53963.100,4500.000,4500.0,-1906.020,0.000,2593.980,55869.120,0.046430,-4500.0
1455709,0,1714892,347207,78,74989.350,22500.0,71963.100,74989.350,4500.000,4500.0,21026.250,0.000,3026.250,53963.100,0.056080,18000.0
2310816,0,1714892,347207,77,87951.060,13500.0,83989.350,87951.060,4500.000,4500.0,12961.710,0.000,3961.710,74989.350,0.052830,9000.0
2904617,0,1714892,347207,76,84479.985,0.0,81201.060,84479.985,6750.000,6750.0,-3471.075,0.000,3278.925,87951.060,0.037281,-6750.0
1944337,0,1714892,347207,75,80972.685,0.0,77729.985,80972.685,6750.000,6750.0,-3507.300,0.000,3242.700,84479.985,0.038384,-6750.0


,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
512990,0,1714892,347207,63,88996.860,0.0,85523.220,88996.860,4500.00,4500.0,-1026.360,0.00,3473.640,90023.220,0.038586,-4500.0
3081605,0,1714892,347207,62,87943.500,0.0,84496.860,87943.500,4500.00,4500.0,-1053.360,0.00,3446.640,88996.860,0.038728,-4500.0
1352698,0,1714892,347207,61,86819.220,0.0,83443.500,86819.220,4500.00,4500.0,-1124.280,0.00,3375.720,87943.500,0.038385,-4500.0
978912,0,1714892,347207,60,85710.645,0.0,82319.220,85710.645,4500.00,4500.0,-1108.575,0.00,3391.425,86819.220,0.039063,-4500.0
589664,0,1714892,347207,59,84538.980,0.0,81210.645,84538.980,4500.00,4500.0,-1171.665,0.00,3328.335,85710.645,0.038832,-4500.0
2903024,0,1714892,347207,58,83382.705,0.0,80038.980,83382.705,4500.00,4500.0,-1156.275,0.00,3343.725,84538.980,0.039552,-4500.0
3141574,0,1714892,347207,57,41148.765,0.0,38382.705,41148.765,45000.00,45000.0,-42233.940,0.00,2766.060,83382.705,0.033173,-45000.0
1765814,0,1714892,347207,56,38907.000,0.0,36648.765,38907.000,4500.00,4500.0,-2241.765,0.00,2258.235,41148.765,0.054880,-4500.0
3373151,0,1714892,347207,55,658.890,0.0,0.000,0.000,41445.00,41445.0,-38248.110,658.89,0.000,38907.000,0.000000,-41445.0
804328,0,1714892,347207,54,0.000,0.0,0.000,0.000,2538.00,0.0,-658.890,0.00,0.000,0.000,NaN,0.0


,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
987638,0,1714892,347207,42,0.0,0.0,0.0,-1879.11,1879.11,0.0,0.0,1879.11,-1879.11,-1879.11,1.0,0.0
2439841,0,1714892,347207,41,0.0,0.0,0.0,-1879.11,1879.11,0.0,0.0,1879.11,-1879.11,-1879.11,1.0,0.0
2815666,0,1714892,347207,40,0.0,0.0,0.0,-1879.11,1879.11,0.0,0.0,1879.11,-1879.11,-1879.11,1.0,0.0
3465009,0,1714892,347207,39,0.0,0.0,0.0,-1879.11,1879.11,0.0,0.0,1879.11,-1879.11,-1879.11,1.0,0.0
2498907,0,1714892,347207,38,0.0,0.0,0.0,0.00,1879.11,0.0,0.0,0.00,0.00,-1879.11,-0.0,0.0
49733,0,1714892,347207,37,0.0,0.0,0.0,0.00,1879.11,0.0,0.0,0.00,0.00,0.00,NaN,0.0
455787,0,1714892,347207,36,0.0,0.0,0.0,0.00,1879.11,0.0,0.0,0.00,0.00,0.00,NaN,0.0
1931377,0,1714892,347207,35,0.0,0.0,0.0,0.00,1879.11,0.0,0.0,0.00,0.00,0.00,NaN,0.0
3331869,0,1714892,347207,34,0.0,0.0,0.0,0.00,1879.11,0.0,0.0,0.00,0.00,0.00,NaN,0.0
860794,0,1714892,347207,33,0.0,0.0,0.0,0.00,1879.11,0.0,0.0,0.00,0.00,0.00,NaN,0.0


,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
2825921,0,1714892,347207,21,0.0,0.0,0.0,0.0,1879.11,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2004841,0,1714892,347207,20,0.0,0.0,0.0,0.0,1879.11,0.0,0.0,0.0,0.0,0.0,NaN,0.0
308268,0,1714892,347207,19,0.0,0.0,0.0,0.0,1879.11,0.0,0.0,0.0,0.0,0.0,NaN,0.0
407194,0,1714892,347207,18,0.0,0.0,0.0,0.0,1879.11,0.0,0.0,0.0,0.0,0.0,NaN,0.0
3414846,0,1714892,347207,17,0.0,0.0,0.0,0.0,1879.11,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2844950,0,1714892,347207,16,0.0,0.0,0.0,0.0,1879.11,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2458555,0,1714892,347207,15,0.0,0.0,0.0,0.0,1879.11,0.0,0.0,0.0,0.0,0.0,NaN,0.0
3023980,0,1714892,347207,14,0.0,0.0,0.0,0.0,1879.11,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1969920,0,1714892,347207,13,0.0,0.0,0.0,0.0,1879.11,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2829304,0,1714892,347207,12,0.0,0.0,0.0,0.0,1879.11,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [219]:
loan = analyze_loan(215757)
display(loan)

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
2026341,0,1302323,215757,28,90015.930,87750.000,87750.000,88670.430,NaN,0.000,NaN,1345.500,920.430,NaN,NaN,87750.000
672049,0,1302323,215757,27,250901.010,247500.000,247495.545,248885.685,91867.500,87274.350,160885.080,2015.325,1390.140,88670.430,0.015678,160225.650
1313526,0,1302323,215757,26,1416.015,0.000,0.000,3157.425,256275.000,243469.485,-249484.995,-1741.410,3157.425,248885.685,0.012686,-243469.485
2702459,0,1302323,215757,25,274522.815,270000.000,269901.720,271831.815,1514.295,98.280,273106.800,2691.000,1930.095,3157.425,0.611288,269901.720
2789119,0,1302323,215757,24,1801.575,0.000,0.000,1933.965,282150.000,268139.790,-272721.240,-132.390,1933.965,271831.815,0.007115,-268139.790
3153743,0,1302323,215757,23,276465.465,270000.000,269855.730,272428.965,1945.845,144.270,274663.890,4036.500,2573.235,1933.965,1.330549,269855.730
433707,0,1302323,215757,22,418252.545,413955.000,412600.905,417639.465,283500.000,269392.725,141787.080,613.080,5038.560,272428.965,0.018495,144562.275
554052,0,1302323,215757,21,273764.700,270000.000,267543.900,274250.745,427500.000,406247.805,-144487.845,-486.045,6706.845,417639.465,0.016059,-136247.805
2946835,0,1302323,215757,20,306069.075,307390.500,301236.615,303378.075,299456.100,297206.100,32304.375,2691.000,2141.460,274250.745,0.007808,10184.400
904952,0,1302323,215757,19,271767.645,270000.000,266920.515,271396.170,315000.000,299403.990,-34301.430,371.475,4475.655,303378.075,0.014753,-29403.990


### Identification des prêts vides

328243

In [221]:
data = load_credit_card_balance()

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\credit_card_balance.pqt


In [222]:
display(data)

RAW_CREDIT_CARD_BALANCE,TARGET,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,-1,2562384,378907,6,56.970,135000,0.0,877.5,0.0,877.5,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,1,2582071,363914,1,63975.555,45000,2250.0,2250.0,0.0,0.0,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,-1,1740877,371185,7,31815.225,450000,0.0,0.0,0.0,0.0,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,0,1389973,337855,4,236572.110,225000,2250.0,2250.0,0.0,0.0,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,0,1891521,126868,1,453919.455,450000,0.0,11547.0,0.0,11547.0,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,0,1036507,328243,9,0.000,45000,NaN,0.0,NaN,NaN,...,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0
3840308,0,1714892,347207,9,0.000,45000,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0,0.0,0.0,23.0,Active,0,0
3840309,0,1302323,215757,9,275784.975,585000,270000.0,270000.0,0.0,0.0,...,273093.975,273093.975,2.0,2,0.0,0.0,18.0,Active,0,0
3840310,0,1624872,430337,10,0.000,450000,NaN,0.0,NaN,NaN,...,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0


In [220]:
display(analyze_loan(328243))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
218340,0,1036507,328243,11,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0
1018393,0,1036507,328243,10,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0
3840307,0,1036507,328243,9,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1951046,0,1036507,328243,8,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0
3255913,0,1036507,328243,7,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1040278,0,1036507,328243,6,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0
3082941,0,1036507,328243,5,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2351112,0,1036507,328243,4,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0
978046,0,1036507,328243,3,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1507926,0,1036507,328243,2,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [225]:
display(data[data.SK_ID_CURR == 328243].sort_values(by="MONTHS_BALANCE", ascending=False))

RAW_CREDIT_CARD_BALANCE,TARGET,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
218340,0,1036507,328243,11,0.0,45000,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
1018393,0,1036507,328243,10,0.0,45000,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
3840307,0,1036507,328243,9,0.0,45000,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
1951046,0,1036507,328243,8,0.0,90000,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
3255913,0,1036507,328243,7,0.0,90000,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
1040278,0,1036507,328243,6,0.0,90000,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
3082941,0,1036507,328243,5,0.0,90000,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
2351112,0,1036507,328243,4,0.0,90000,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
978046,0,1036507,328243,3,0.0,90000,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
1507926,0,1036507,328243,2,0.0,90000,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0


Pour les identifier tous :
1. trier par PID, CID, M° (desc)
2. ...

In [228]:
balance = get_credit_card_balance_payment_balance()
balance = balance.sort_values(by=["PID", "CID", "M°"], ascending=False)
display(balance)

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\credit_card_balance.pqt


,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT
818129,-1,2843496,425374,15,0.000,0.000,0.000,0.000,NaN,0.000
709719,-1,2843496,425374,14,0.000,1704.780,0.000,0.000,1800.000,945.675
2850284,-1,2843496,425374,13,117387.045,137109.555,115653.780,117387.045,24750.000,17835.300
3662596,-1,2843496,425374,12,179685.045,74143.350,175775.670,178114.545,9000.000,2686.230
1337878,-1,2843496,425374,11,189972.450,35983.845,186200.955,188401.950,31050.000,22459.140
...,...,...,...,...,...,...,...,...,...,...
1375415,0,1000018,394447,6,38879.145,51042.645,37542.645,37542.645,9000.000,9000.000
277653,0,1000018,394447,5,40934.070,2335.500,39576.780,40934.070,3900.870,3900.870
2559582,0,1000018,394447,4,44360.505,2032.560,43376.760,44360.505,3646.710,3646.710
2192275,0,1000018,394447,3,113862.285,69156.945,108091.800,109150.785,3190.635,3190.635


Fixer les entêtes :

```python
# Utilisez le style pour fixer les entêtes
styled_df = df.style.set_sticky(axis='index', headers=True)

# Affichez le DataFrame avec les entêtes figées
styled_df
```

Il il y a 104 307 prêts dont 33 787 nuls, soit près d'un tiers.

In [240]:
aggregated = (
    balance[balance.columns[1:]]
    .groupby(by=list(balance.columns[1:3]))
    .agg({"M°": "count", "BAL": "sum", "DRAW": "sum", "PYT": "sum", "PYT_TOT": "sum"})
)
display(aggregated)
display(aggregated[aggregated.BAL == 0])
print(list(aggregated[aggregated.BAL == 0].index.get_level_values(1)))

,,M°,BAL,DRAW,PYT,PYT_TOT
PID,CID,,,,,
1000018,394447,5,374731.425,147394.980,27708.750,27708.750
1000030,361282,8,447928.515,138059.505,43320.420,21263.580
1000031,131335,16,838311.030,463353.840,354519.090,354519.090
1000035,436351,5,0.000,0.000,0.000,0.000
1000077,181153,11,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...
2843476,197090,95,3604092.165,90000.000,329404.905,328513.500
2843477,168439,85,141361.515,58500.000,81197.550,67410.000
2843478,424526,90,460026.450,90000.000,122021.550,109980.000


,,M°,BAL,DRAW,PYT,PYT_TOT
PID,CID,,,,,
1000035,436351,5,0.0,0.00,0.00,0.00
1000077,181153,11,0.0,0.00,0.00,0.00
1000083,309691,13,0.0,0.00,0.00,0.00
1000089,161517,5,0.0,0.00,0.00,0.00
1000123,387909,8,0.0,222740.28,234736.83,224571.33
...,...,...,...,...,...,...
2843329,114701,35,0.0,0.00,0.00,0.00
2843332,298901,46,0.0,0.00,0.00,0.00
2843336,174805,52,0.0,22482.00,45166.50,45166.50


[436351, 181153, 309691, 161517, 387909, 173111, 248741, 311379, 360218, 112719, 113311, 252066, 116580, 133825, 382368, 252395, 104049, 128435, 335741, 409953, 269309, 407962, 264055, 258806, 140511, 436141, 316247, 338194, 399710, 214709, 319083, 400832, 115867, 309757, 331596, 203873, 336918, 273326, 151624, 120826, 106765, 369504, 138924, 211287, 267216, 181271, 141689, 151448, 270003, 304388, 337361, 357855, 336758, 295925, 423837, 423374, 278723, 152654, 155790, 233547, 261931, 136172, 206414, 219015, 320150, 147626, 143041, 410078, 209429, 195816, 193217, 188093, 261238, 163268, 322817, 339940, 426743, 120287, 296109, 406835, 231106, 311725, 363887, 346155, 387506, 407596, 229621, 140653, 437929, 114139, 361016, 189499, 196765, 366367, 352539, 337223, 217410, 178344, 115938, 380561, 339359, 359475, 417993, 381113, 417350, 425422, 409234, 315622, 382341, 406547, 262014, 230786, 413154, 169462, 143769, 430182, 328171, 140741, 264688, 399805, 338959, 234924, 440812, 217244, 299426,

31 620 cas vraiment nuls :

In [241]:
null_row = (
    (aggregated.BAL == 0) & (aggregated.DRAW == 0)
    & (aggregated.PYT == 0) & (aggregated.PYT_TOT == 0)
)
display(aggregated[null_row])
print(list(aggregated[null_row].index.get_level_values(1)))

,,M°,BAL,DRAW,PYT,PYT_TOT
PID,CID,,,,,
1000035,436351,5,0.0,0.0,0.0,0.0
1000077,181153,11,0.0,0.0,0.0,0.0
1000083,309691,13,0.0,0.0,0.0,0.0
1000089,161517,5,0.0,0.0,0.0,0.0
1000132,173111,18,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
2843249,283041,27,0.0,0.0,0.0,0.0
2843320,401272,27,0.0,0.0,0.0,0.0
2843329,114701,35,0.0,0.0,0.0,0.0


[436351, 181153, 309691, 161517, 173111, 248741, 311379, 360218, 112719, 113311, 252066, 116580, 133825, 382368, 252395, 104049, 128435, 335741, 409953, 269309, 407962, 264055, 258806, 140511, 436141, 316247, 338194, 399710, 214709, 319083, 400832, 115867, 309757, 331596, 203873, 336918, 273326, 120826, 106765, 369504, 138924, 211287, 267216, 181271, 141689, 151448, 270003, 304388, 337361, 357855, 336758, 295925, 423374, 152654, 155790, 233547, 136172, 206414, 320150, 147626, 143041, 410078, 209429, 195816, 193217, 188093, 163268, 322817, 339940, 426743, 120287, 296109, 406835, 311725, 363887, 346155, 387506, 407596, 229621, 140653, 114139, 361016, 189499, 196765, 366367, 352539, 337223, 217410, 178344, 115938, 380561, 339359, 359475, 417993, 381113, 425422, 409234, 315622, 382341, 406547, 262014, 230786, 413154, 169462, 143769, 430182, 328171, 140741, 264688, 399805, 338959, 234924, 217244, 299426, 107978, 386706, 219244, 200574, 238967, 301071, 338836, 285934, 230450, 290570, 151653,

2 167 cas partiellement nuls : ils sont intéressants, car il mettent en évidence la mécanique interne.

Ce sont les cas où une somme, un fois ou plusieurs, est systématiquement soldée dans le mois.

Sur le cas 423837, on observe la persistence d'une somme mensuelle, de la moitié du montant d'intérêt initial.

Cette somme persiste (frais, par exemple en fonction du niveau de crédit autorisé ?) bien que le prêt soit soldé.

27 8723 fait partie des cas où il n'y a ni frais ni intérêts, une opération blanche : et aussi 437929, ..

26 1931 montre que certaines BAL n'ont pas été calculées, et sont à 0 alors qu'il y a des mouvements sur le compte. Autres exemples : 219 015

231 106 et d'autres semblent montrer que la limite de crédit fixée entraîne des frais mensuels fixes, d'autant plus élevés que le plafond de crédit est élevé : c'est une hypothèse importante à vérifier.

In [242]:
partial_null_row = (
    (aggregated.BAL == 0) & ~(
        (aggregated.DRAW == 0) & (aggregated.PYT == 0) & (aggregated.PYT_TOT == 0)
    )
)
display(aggregated[partial_null_row])
print(list(aggregated[partial_null_row].index.get_level_values(1)))

,,M°,BAL,DRAW,PYT,PYT_TOT
PID,CID,,,,,
1000123,387909,8,0.0,222740.28,234736.830,224571.330
1001409,151624,8,0.0,225000.00,225000.000,225000.000
1001865,423837,21,0.0,7065.00,7339.725,7339.725
1001932,278723,7,0.0,675000.00,675000.000,675000.000
1002299,261931,9,0.0,103950.00,33295.500,31968.000
...,...,...,...,...,...,...
2841446,230687,5,0.0,247500.00,247500.000,247500.000
2841512,399846,3,0.0,166500.00,166995.000,166995.000
2843012,424873,13,0.0,130.50,391.500,391.500


[387909, 151624, 423837, 278723, 261931, 219015, 261238, 231106, 437929, 417350, 440812, 229559, 308169, 215926, 238938, 267829, 438513, 447972, 149636, 434912, 399294, 368179, 182941, 401447, 225759, 318075, 154274, 407145, 362715, 293939, 178730, 319764, 225937, 126737, 328981, 283803, 226346, 120535, 219869, 352504, 145584, 201482, 300367, 130445, 140004, 255598, 256009, 419710, 237310, 166895, 218927, 130310, 433237, 129132, 444509, 355174, 256509, 387775, 204864, 207711, 204589, 410766, 313667, 344238, 158745, 230089, 232810, 314721, 330418, 384210, 413064, 337541, 335236, 108444, 343944, 254100, 314608, 455720, 288185, 264299, 123959, 368601, 250678, 307873, 123299, 222451, 117837, 107868, 420099, 106289, 323943, 453469, 372321, 141622, 240526, 351030, 270861, 206717, 360378, 432998, 107645, 317593, 380854, 449663, 283120, 436322, 402538, 375501, 145323, 205793, 138481, 444150, 178549, 372881, 208581, 191293, 390331, 172818, 277288, 431237, 371977, 416905, 130693, 261528, 109021,

In [243]:
display(analyze_loan(387909))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
64530,0,1000123,387909,9,0.0,0.000,0.0,0.0,NaN,0.00,NaN,0.0,0.0,NaN,NaN,0.000
3444999,0,1000123,387909,8,0.0,43630.470,0.0,0.0,44415.00,42165.00,0.0,0.0,0.0,0.0,NaN,1465.470
2560300,0,1000123,387909,7,0.0,875.655,0.0,0.0,1745.19,1745.19,0.0,0.0,0.0,0.0,NaN,-869.535
544297,0,1000123,387909,6,0.0,178234.155,0.0,0.0,178290.00,170374.50,0.0,0.0,0.0,0.0,NaN,7859.655
2054438,0,1000123,387909,5,0.0,0.000,0.0,0.0,6314.58,6314.58,0.0,0.0,0.0,0.0,NaN,-6314.580
2370104,0,1000123,387909,4,0.0,0.000,0.0,0.0,3972.06,3972.06,0.0,0.0,0.0,0.0,NaN,-3972.060
1344205,0,1000123,387909,3,0.0,0.000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,NaN,0.000
1461911,0,1000123,387909,2,0.0,0.000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,NaN,0.000


In [244]:
display(analyze_loan(151624))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
708398,0,1001409,151624,9,0.0,0.0,0.0,0.0,225000.0,225000.0,NaN,0.0,0.0,NaN,NaN,-225000.0
853551,0,1001409,151624,8,0.0,225000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,225000.0
1095994,0,1001409,151624,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
579004,0,1001409,151624,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
351210,0,1001409,151624,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
683954,0,1001409,151624,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1527255,0,1001409,151624,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1630959,0,1001409,151624,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [245]:
display(analyze_loan(423837))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
3234468,0,1001865,423837,22,0.0,0.000,0.0,0.0,NaN,0.000,NaN,0.0,0.0,NaN,NaN,0.000
2938918,0,1001865,423837,21,0.0,0.000,0.0,0.0,NaN,0.000,0.0,0.0,0.0,0.0,NaN,0.000
1249936,0,1001865,423837,20,0.0,0.000,0.0,0.0,NaN,0.000,0.0,0.0,0.0,0.0,NaN,0.000
2295324,0,1001865,423837,19,0.0,0.000,0.0,0.0,NaN,0.000,0.0,0.0,0.0,0.0,NaN,0.000
3521223,0,1001865,423837,18,0.0,7046.685,0.0,0.0,7083.315,7083.315,0.0,0.0,0.0,0.0,NaN,-36.630
1768263,0,1001865,423837,17,0.0,0.000,0.0,0.0,18.315,18.315,0.0,0.0,0.0,0.0,NaN,-18.315
1326144,0,1001865,423837,16,0.0,0.000,0.0,0.0,18.315,18.315,0.0,0.0,0.0,0.0,NaN,-18.315
365679,0,1001865,423837,15,0.0,0.000,0.0,0.0,18.315,18.315,0.0,0.0,0.0,0.0,NaN,-18.315
176817,0,1001865,423837,14,0.0,0.000,0.0,0.0,18.315,18.315,0.0,0.0,0.0,0.0,NaN,-18.315
2196762,0,1001865,423837,13,0.0,0.000,0.0,0.0,18.315,18.315,0.0,0.0,0.0,0.0,NaN,-18.315


In [257]:
display(data[data.SK_ID_CURR == 423837].sort_values(by="MONTHS_BALANCE", ascending=False).T)

,3234468,2938918,1249936,2295324,3521223,1768263,1326144,365679,176817,2196762,...,3565976,2205722,791752,3057670,771370,1175755,419464,3177440,1214364,1964492
RAW_CREDIT_CARD_BALANCE,,,,,,,,,,,,,,,,,,,,,
TARGET,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SK_ID_PREV,1001865,1001865,1001865,1001865,1001865,1001865,1001865,1001865,1001865,1001865,...,1001865,1001865,1001865,1001865,1001865,1001865,1001865,1001865,1001865,1001865
SK_ID_CURR,423837,423837,423837,423837,423837,423837,423837,423837,423837,423837,...,423837,423837,423837,423837,423837,423837,423837,423837,423837,423837
MONTHS_BALANCE,22,21,20,19,18,17,16,15,14,13,...,11,10,9,8,7,6,5,4,3,2
AMT_BALANCE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMT_CREDIT_LIMIT_ACTUAL,450000,450000,450000,450000,450000,450000,450000,450000,0,0,...,0,0,0,0,0,0,0,0,0,0
AMT_DRAWINGS_ATM_CURRENT,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMT_DRAWINGS_CURRENT,0.0,0.0,0.0,0.0,7046.685,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.315,0.0
AMT_DRAWINGS_OTHER_CURRENT,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [246]:
display(analyze_loan(278723))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
2192790,-1,1001932,278723,8,0.0,675000.0,0.0,0.0,675000.0,675000.0,NaN,0.0,0.0,NaN,NaN,0.0
1387700,-1,1001932,278723,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1485032,-1,1001932,278723,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1273344,-1,1001932,278723,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
868019,-1,1001932,278723,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
33470,-1,1001932,278723,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1049427,-1,1001932,278723,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [250]:
display(data[data.SK_ID_CURR == 278723].sort_values(by="MONTHS_BALANCE", ascending=False).T)

,2192790,1387700,1485032,1273344,868019,33470,1049427
RAW_CREDIT_CARD_BALANCE,,,,,,,
TARGET,-1,-1,-1,-1,-1,-1,-1
SK_ID_PREV,1001932,1001932,1001932,1001932,1001932,1001932,1001932
SK_ID_CURR,278723,278723,278723,278723,278723,278723,278723
MONTHS_BALANCE,8,7,6,5,4,3,2
AMT_BALANCE,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMT_CREDIT_LIMIT_ACTUAL,0,0,0,0,0,0,0
AMT_DRAWINGS_ATM_CURRENT,675000.0,0.0,0.0,0.0,0.0,0.0,0.0
AMT_DRAWINGS_CURRENT,675000.0,0.0,0.0,0.0,0.0,0.0,0.0
AMT_DRAWINGS_OTHER_CURRENT,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [251]:
display(analyze_loan(261931))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
3327582,0,1002299,261931,10,0.0,92250.0,0.0,0.0,27450.0,26388.0,NaN,0.0,0.0,NaN,NaN,65862.0
161162,0,1002299,261931,9,0.0,11700.0,0.0,0.0,963.0,697.5,0.0,0.0,0.0,0.0,NaN,11002.5
1700029,0,1002299,261931,8,0.0,0.0,0.0,0.0,697.5,697.5,0.0,0.0,0.0,0.0,NaN,-697.5
2367776,0,1002299,261931,7,0.0,0.0,0.0,0.0,697.5,697.5,0.0,0.0,0.0,0.0,NaN,-697.5
1280928,0,1002299,261931,6,0.0,0.0,0.0,0.0,697.5,697.5,0.0,0.0,0.0,0.0,NaN,-697.5
2179479,0,1002299,261931,5,0.0,0.0,0.0,0.0,697.5,697.5,0.0,0.0,0.0,0.0,NaN,-697.5
2444217,0,1002299,261931,4,0.0,0.0,0.0,0.0,697.5,697.5,0.0,0.0,0.0,0.0,NaN,-697.5
3271467,0,1002299,261931,3,0.0,0.0,0.0,0.0,697.5,697.5,0.0,0.0,0.0,0.0,NaN,-697.5
2643087,0,1002299,261931,2,0.0,0.0,0.0,0.0,697.5,697.5,0.0,0.0,0.0,0.0,NaN,-697.5


In [252]:
display(analyze_loan(219015))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
2450884,0,1002655,219015,16,0.0,0.0,0.0,0.0,NaN,0.00,NaN,0.0,0.0,NaN,NaN,0.00
2110079,0,1002655,219015,15,0.0,33216.3,0.0,0.0,39233.70,36983.70,0.0,0.0,0.0,0.0,NaN,-3767.40
93144,0,1002655,219015,14,0.0,0.0,0.0,0.0,129.51,129.51,0.0,0.0,0.0,0.0,NaN,-129.51
1982497,0,1002655,219015,13,0.0,0.0,0.0,0.0,129.51,129.51,0.0,0.0,0.0,0.0,NaN,-129.51
2391566,0,1002655,219015,12,0.0,0.0,0.0,0.0,129.51,129.51,0.0,0.0,0.0,0.0,NaN,-129.51
2840452,0,1002655,219015,11,0.0,0.0,0.0,0.0,129.51,129.51,0.0,0.0,0.0,0.0,NaN,-129.51
1356738,0,1002655,219015,10,0.0,0.0,0.0,0.0,129.51,129.51,0.0,0.0,0.0,0.0,NaN,-129.51
3827931,0,1002655,219015,9,0.0,0.0,0.0,0.0,129.51,129.51,0.0,0.0,0.0,0.0,NaN,-129.51
2423450,0,1002655,219015,8,0.0,0.0,0.0,0.0,129.51,129.51,0.0,0.0,0.0,0.0,NaN,-129.51
597638,0,1002655,219015,7,0.0,0.0,0.0,0.0,129.51,129.51,0.0,0.0,0.0,0.0,NaN,-129.51


In [253]:
display(analyze_loan(261238))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
3200928,0,1002951,261238,5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0
2503155,0,1002951,261238,4,0.0,0.0,0.0,0.0,270.0,270.0,0.0,0.0,0.0,0.0,NaN,-270.0
912152,0,1002951,261238,3,0.0,0.0,0.0,0.0,135.0,135.0,0.0,0.0,0.0,0.0,NaN,-135.0
906823,0,1002951,261238,2,0.0,0.0,0.0,0.0,135.0,135.0,0.0,0.0,0.0,0.0,NaN,-135.0


In [254]:
display(data[data.SK_ID_CURR == 261238].sort_values(by="MONTHS_BALANCE", ascending=False).T)

,3200928,2503155,912152,906823
RAW_CREDIT_CARD_BALANCE,,,,
TARGET,0,0,0,0
SK_ID_PREV,1002951,1002951,1002951,1002951
SK_ID_CURR,261238,261238,261238,261238
MONTHS_BALANCE,5,4,3,2
AMT_BALANCE,0.0,0.0,0.0,0.0
AMT_CREDIT_LIMIT_ACTUAL,225000,225000,0,0
AMT_DRAWINGS_ATM_CURRENT,NaN,NaN,NaN,NaN
AMT_DRAWINGS_CURRENT,0.0,0.0,0.0,0.0
AMT_DRAWINGS_OTHER_CURRENT,NaN,NaN,NaN,NaN


In [255]:
display(analyze_loan(231106))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
3186514,0,1003088,231106,96,0.0,0.0,0.0,0.0,27.99,0.0,NaN,0.0,0.0,NaN,NaN,0.0
340922,0,1003088,231106,95,0.0,0.0,0.0,0.0,27.99,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1126906,0,1003088,231106,94,0.0,0.0,0.0,0.0,27.99,0.0,0.0,0.0,0.0,0.0,NaN,0.0
3138189,0,1003088,231106,93,0.0,0.0,0.0,0.0,27.99,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1137572,0,1003088,231106,92,0.0,0.0,0.0,0.0,27.99,0.0,0.0,0.0,0.0,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206796,0,1003088,231106,6,0.0,0.0,0.0,0.0,27.99,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1223997,0,1003088,231106,5,0.0,0.0,0.0,0.0,27.99,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2490422,0,1003088,231106,4,0.0,0.0,0.0,0.0,27.99,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1606305,0,1003088,231106,3,0.0,0.0,0.0,0.0,27.99,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [256]:
display(data[data.SK_ID_CURR == 231106].sort_values(by="MONTHS_BALANCE", ascending=False).T)

,3186514,340922,1126906,3138189,1137572,521223,620345,2832886,1052100,1975351,...,2313391,3136594,3177452,3788758,2583136,1206796,1223997,2490422,1606305,1460833
RAW_CREDIT_CARD_BALANCE,,,,,,,,,,,,,,,,,,,,,
TARGET,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SK_ID_PREV,1003088,1003088,1003088,1003088,1003088,1003088,1003088,1003088,1003088,1003088,...,1003088,1003088,1003088,1003088,1003088,1003088,1003088,1003088,1003088,1003088
SK_ID_CURR,231106,231106,231106,231106,231106,231106,231106,231106,231106,231106,...,231106,231106,231106,231106,231106,231106,231106,231106,231106,231106
MONTHS_BALANCE,96,95,94,93,92,91,90,89,88,87,...,11,10,9,8,7,6,5,4,3,2
AMT_BALANCE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMT_CREDIT_LIMIT_ACTUAL,90000,90000,90000,90000,90000,90000,90000,90000,90000,90000,...,0,0,0,0,0,0,0,0,0,0
AMT_DRAWINGS_ATM_CURRENT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMT_DRAWINGS_CURRENT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMT_DRAWINGS_OTHER_CURRENT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [259]:
display(analyze_loan(417350))

,TGT,PID,CID,M°,BAL,DRAW,RCV,RCV_TOT,PYT,PYT_TOT,BAL_diff,BAL-RCV,IF,REF_pre,rate,D-P
3120979,0,1003828,417350,5,0.0,194400.0,0.0,0.0,737100.0,734850.0,NaN,0.0,0.0,NaN,NaN,-540450.0
746684,0,1003828,417350,4,0.0,263497.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,263497.5
2670665,0,1003828,417350,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1220316,0,1003828,417350,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2937985,0,1003828,417350,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


### Relation entre plafond de crédit et frais

AMT_CREDIT_LIMIT_ACTUAL = 9000 => AMT_PAYMENT_CURRENT = 27.99
...

### Cas consistants

A l'autre extrémité du spectre, recherchons des cas consistant, car l'exploration précédente laisse l'impression que ces données ne le sont pas. Ce sont probablement de réelles aberrations qu'il faudra en partie supprimer dans le cadre du nettoyage des données.

On repart de l'agrégation et on tente un raisonnement comptable de vérification de cohérence des comptes.

**TODO** C'est à améliorer de la manière suivante : la fenêtre de suivie démarre avec un solde initial qui n'est pas nécessairement nul puisque le prêt peut avoir été contracté antérieurement au suivi. Il faut donc spécifiquement identifier (FIRST) ce solde initial au moment de l'agrégation.

In [262]:
aggregated = (
    balance[balance.columns[1:]]
    .groupby(by=list(balance.columns[1:3]))
    .agg({"M°": "count", "BAL": "sum", "DRAW": "sum", "PYT": "sum", "PYT_TOT": "sum"})
)
display(aggregated)
display(aggregated[aggregated.BAL != 0])
print(list(aggregated[aggregated.BAL != 0].index.get_level_values(1)))

,,M°,BAL,DRAW,PYT,PYT_TOT
PID,CID,,,,,
1000018,394447,5,374731.425,147394.980,27708.750,27708.750
1000030,361282,8,447928.515,138059.505,43320.420,21263.580
1000031,131335,16,838311.030,463353.840,354519.090,354519.090
1000035,436351,5,0.000,0.000,0.000,0.000
1000077,181153,11,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...
2843476,197090,95,3604092.165,90000.000,329404.905,328513.500
2843477,168439,85,141361.515,58500.000,81197.550,67410.000
2843478,424526,90,460026.450,90000.000,122021.550,109980.000


,,M°,BAL,DRAW,PYT,PYT_TOT
PID,CID,,,,,
1000018,394447,5,374731.425,147394.980,27708.750,27708.750
1000030,361282,8,447928.515,138059.505,43320.420,21263.580
1000031,131335,16,838311.030,463353.840,354519.090,354519.090
1000087,399664,32,1250499.330,136911.195,197287.425,197287.425
1000094,359175,88,2583953.145,138601.035,260075.205,256591.035
...,...,...,...,...,...,...
2843476,197090,95,3604092.165,90000.000,329404.905,328513.500
2843477,168439,85,141361.515,58500.000,81197.550,67410.000
2843478,424526,90,460026.450,90000.000,122021.550,109980.000


[394447, 361282, 131335, 399664, 359175, 306118, 398841, 142073, 108082, 445824, 121879, 382594, 372666, 217839, 357952, 282139, 269865, 182312, 140708, 377673, 308466, 274118, 233424, 216298, 128665, 377582, 282625, 344868, 380211, 199167, 346585, 126099, 336128, 438352, 133176, 216123, 246242, 274792, 101961, 332626, 175229, 187389, 120740, 382971, 413871, 406568, 440622, 120761, 198725, 181724, 381938, 439240, 389124, 430179, 363834, 305987, 193067, 393187, 328160, 165880, 202644, 120987, 373592, 268971, 314738, 365481, 429235, 221001, 203522, 443292, 243542, 187286, 110345, 272767, 218425, 351052, 357955, 358557, 236843, 135332, 263951, 275001, 183467, 139229, 440029, 206661, 106753, 155646, 193857, 369154, 331896, 415748, 433028, 276356, 191404, 199709, 271220, 431281, 379919, 187740, 446568, 308902, 361840, 337250, 175060, 329914, 108045, 128171, 296139, 166074, 127489, 384882, 242425, 398865, 280550, 204683, 434100, 424031, 425600, 453005, 195905, 122569, 381754, 217138, 125046,

# **`application`**

Cette table contient 356 255 enregistrements qui sont les demandes de crédit actuelles.

## Données, conventions, utilitaires

### Chargement de la table principale

In [4]:
from home_credit.load import get_table 
from home_credit.utils import display_frame_basic_infos

data = get_table("application")
display_frame_basic_infos(data)
data.info()

n_samples: 356 255
n_columns: 122, [('SK', 1), ('FLAG', 28), ('NAME', 6), ('DAYS', 5), ('CNT', 2), ('AMT', 10), ('APARTMENTS', 3), ('BASEMENTAREA', 3), ('CODE', 1), ('COMMONAREA', 3), ('DEF', 2), ('ELEVATORS', 3), ('EMERGENCYSTATE', 1), ('ENTRANCES', 3), ('EXT', 3), ('FLOORSMAX', 3), ('FLOORSMIN', 3), ('FONDKAPREMONT', 1), ('HOUR', 1), ('HOUSETYPE', 1), ('LANDAREA', 3), ('LIVE', 2), ('LIVINGAPARTMENTS', 3), ('LIVINGAREA', 3), ('NONLIVINGAPARTMENTS', 3), ('NONLIVINGAREA', 3), ('OBS', 2), ('OCCUPATION', 1), ('ORGANIZATION', 1), ('OWN', 1), ('REG', 4), ('REGION', 3), ('TARGET', 1), ('TOTALAREA', 1), ('WALLSMATERIAL', 1), ('WEEKDAY', 1), ('YEARS', 6)]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 331.6+ MB


### Groupes de variables, et abréviations des noms longs

In [2]:
from home_credit.tables import Application

print(Application.cols_group("financial_statement"))
print(Application.cols_group("financial_statement", shorten=True))

['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE']
[('INCOME', '', 'AMT'), ('LOAN', '', 'AMT'), ('ANNUITY', '', 'AMT'), ('GOODS', '', 'AMT')]


## Propositions sur les NA

### Les 4 XNA de `CODE_GENDER`

In [79]:
vc_dict = data["CODE_GENDER"].value_counts(dropna=False).to_dict()
print(f"CODE_GENDER ({len(vc_dict)}): {vc_dict}")

CODE_GENDER (3): {'F': 235126, 'M': 121125, 'XNA': 4}


In [81]:
display(data[data.CODE_GENDER == "XNA"].T.head(60))

,35657,38566,83382,189640
application,,,,
SK_ID_CURR,141289,144669,196708,319880
TARGET,0,0,0,0
NAME_CONTRACT_TYPE,Revolving loans,Revolving loans,Revolving loans,Revolving loans
CODE_GENDER,XNA,XNA,XNA,XNA
FLAG_OWN_CAR,Y,N,N,Y
FLAG_OWN_REALTY,Y,Y,Y,Y
CNT_CHILDREN,0,2,1,0
AMT_INCOME_TOTAL,207000.0,157500.0,135000.0,247500.0
AMT_CREDIT,382500.0,270000.0,405000.0,540000.0


### Pourquoi `DAYS_REGISTRATION` est en `float64` ?

Sachant qu'il ne déclare aucun NA est que son intervalle de définition est [-24612, 0].

En raison de l'enregistrement `408583` qui indique 10116 et 1h (10116.041666666662).

In [17]:
reg = (-data["DAYS_REGISTRATION"])
vc_dict = reg.value_counts(dropna=False).to_dict()
print(f"DAYS_REGISTRATION ({len(vc_dict)}): {vc_dict}")
print({k: v for k, v in vc_dict.items() if (k+1) % 1 != 0})
print(round(.041666666662 * 24))
display(data[(reg + 1) % 1 != 0])



DAYS_REGISTRATION (15898): {1.0: 123, 7.0: 111, 6.0: 107, 4.0: 99, 2.0: 99, 5.0: 95, 3.0: 94, -0.0: 93, 21.0: 92, 9.0: 91, 14.0: 89, 10.0: 87, 511.0: 80, 56.0: 75, 11.0: 75, 15.0: 74, 8.0: 73, 13.0: 72, 69.0: 72, 363.0: 71, 602.0: 71, 34.0: 71, 569.0: 71, 427.0: 71, 923.0: 71, 70.0: 71, 41.0: 70, 679.0: 70, 819.0: 70, 251.0: 70, 23.0: 70, 17.0: 70, 889.0: 69, 19.0: 69, 20.0: 69, 308.0: 69, 42.0: 69, 614.0: 68, 742.0: 68, 29.0: 68, 630.0: 68, 868.0: 67, 161.0: 67, 406.0: 67, 812.0: 67, 35.0: 67, 333.0: 67, 756.0: 67, 139.0: 67, 827.0: 66, 342.0: 66, 146.0: 66, 973.0: 66, 27.0: 66, 4256.0: 66, 434.0: 66, 125.0: 66, 245.0: 66, 4606.0: 66, 273.0: 66, 621.0: 66, 490.0: 66, 736.0: 65, 854.0: 65, 287.0: 65, 83.0: 65, 735.0: 65, 356.0: 65, 680.0: 65, 77.0: 65, 182.0: 65, 519.0: 65, 47.0: 65, 1161.0: 64, 371.0: 64, 825.0: 64, 389.0: 64, 393.0: 64, 994.0: 64, 693.0: 64, 55.0: 64, 260.0: 64, 910.0: 64, 580.0: 64, 391.0: 64, 300.0: 64, 203.0: 64, 399.0: 63, 106.0: 63, 294.0: 63, 428.0: 63, 186.0: 

application,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
266366,408583,0,Cash loans,F,Y,N,2,157500.0,455040.0,12132.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,7.0


### L'unique NA `DAYS_LAST_PHONE_CHANGE`

C'est le **`SK_ID_CURR`** `118330`.

On lui impute la valeur la plus fréquente : 0 (qui semble plutôt signifier NA qu'un changement de téléphone le jour de la demande de prêt, quoique cela se pourrait si le prêt finance l'achat dudit téléphone).

In [3]:
print(data.DAYS_LAST_PHONE_CHANGE.value_counts(dropna=False).to_dict())
display(data[data.DAYS_LAST_PHONE_CHANGE.isna()].T.head(15))

{0.0: 43473, -1.0: 2986, -2.0: 2389, -3.0: 1808, -4.0: 1328, -5.0: 859, -6.0: 551, -7.0: 469, -8.0: 298, -476.0: 247, -364.0: 244, -352.0: 242, -399.0: 235, -392.0: 234, -362.0: 232, -329.0: 231, -293.0: 230, -404.0: 230, -355.0: 227, -441.0: 226, -496.0: 226, -301.0: 226, -358.0: 226, -304.0: 225, -479.0: 225, -483.0: 225, -500.0: 225, -347.0: 225, -427.0: 223, -308.0: 223, -334.0: 223, -350.0: 223, -532.0: 220, -391.0: 220, -546.0: 220, -448.0: 219, -463.0: 219, -415.0: 219, -363.0: 219, -609.0: 218, -413.0: 218, -353.0: 218, -509.0: 216, -469.0: 216, -499.0: 216, -504.0: 215, -9.0: 215, -11.0: 215, -428.0: 215, -534.0: 214, -630.0: 213, -447.0: 213, -566.0: 213, -475.0: 212, -574.0: 212, -296.0: 211, -524.0: 211, -484.0: 211, -541.0: 211, -288.0: 211, -354.0: 211, -420.0: 211, -531.0: 211, -349.0: 210, -625.0: 210, -565.0: 209, -511.0: 208, -297.0: 208, -518.0: 208, -461.0: 208, -343.0: 207, -417.0: 207, -371.0: 206, -271.0: 206, -372.0: 206, -510.0: 206, -628.0: 205, -340.0: 205, -

,15709
application,
SK_ID_CURR,118330
TARGET,0
NAME_CONTRACT_TYPE,Cash loans
CODE_GENDER,M
FLAG_OWN_CAR,Y
FLAG_OWN_REALTY,Y
CNT_CHILDREN,0
AMT_INCOME_TOTAL,126000.0
AMT_CREDIT,278613.0


### Les deux NA `CNT_FAM_MEMBERS`

Ce sont les **`SK_ID_CURR`** `148605` et `317181`.

Ces deux personnes, un homme et une femme, à peu près du même âge, associés commerciaux tous deux, on également en commun de faire partie des 278 NA concernant `AMT_GOODS_PRICE`.

Ils n'ont pas d'enfants, n'étaient pas accompagnés au moment de la demande de prêt. Leur statut familial n'est pas connu. Nous imputons leur valeur à 1 (famille d'une seule personne).

In [26]:
print(data.CNT_FAM_MEMBERS.value_counts(dropna=False).to_dict())
display(data[data.CNT_FAM_MEMBERS.isna()].T.head(60))
display(data[data.CNT_FAM_MEMBERS.isna()].T.tail(60))

{2.0: 184411, 1.0: 78098, 3.0: 60774, 4.0: 28387, 5.0: 3990, 6.0: 451, 7.0: 93, 8.0: 22, 9.0: 7, 10.0: 6, 13.0: 3, nan: 2, 14.0: 2, 12.0: 2, 20.0: 2, 16.0: 2, 15.0: 1, 11.0: 1, 21.0: 1}


,41982,187348
application,,
SK_ID_CURR,148605,317181
TARGET,0,0
NAME_CONTRACT_TYPE,Revolving loans,Revolving loans
CODE_GENDER,M,F
FLAG_OWN_CAR,N,N
FLAG_OWN_REALTY,Y,Y
CNT_CHILDREN,0,0
AMT_INCOME_TOTAL,450000.0,202500.0
AMT_CREDIT,675000.0,585000.0


,41982,187348
application,,
COMMONAREA_MODE,NaN,0.0082
ELEVATORS_MODE,NaN,0.0806
ENTRANCES_MODE,NaN,0.069
FLOORSMAX_MODE,NaN,0.375
FLOORSMIN_MODE,NaN,0.4167
LANDAREA_MODE,NaN,NaN
LIVINGAPARTMENTS_MODE,NaN,0.0918
LIVINGAREA_MODE,NaN,0.0917
NONLIVINGAPARTMENTS_MODE,NaN,0.0


### Les 36 NA `AMT_ANNUITY`

Ce sont tous des contrats de type `Cash loans`.

On tentera de croiser avec les données de bureau et de previous_application, mais si l'annuité n'est pas définie, il est difficile d'évaluer un risque qui dépend essentiellement de la charge à supporter en rapport avec les autres montants dimensionnants.

In [68]:
display(data[data.AMT_ANNUITY.isna()])

application,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
47531,155054,0,Cash loans,M,N,N,0,180000.0,450000.0,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
50035,157917,0,Cash loans,F,N,N,0,94500.0,450000.0,NaN,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
51594,159744,0,Cash loans,F,N,N,0,202500.0,539100.0,NaN,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
55025,163757,0,Cash loans,F,N,N,0,162000.0,296280.0,NaN,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
59934,169487,0,Cash loans,M,Y,N,0,202500.0,360000.0,NaN,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,6.0
75873,187985,0,Cash loans,M,Y,N,0,144000.0,219249.0,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,11.0
89343,203726,0,Cash loans,F,Y,N,0,90000.0,157500.0,NaN,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0
123872,243648,0,Cash loans,F,N,Y,0,202500.0,929088.0,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,5.0
207186,340147,0,Cash loans,M,N,N,0,171000.0,486000.0,NaN,...,0,0,0,0,0.0,0.0,1.0,1.0,0.0,2.0
227939,364022,0,Cash loans,F,N,Y,0,315000.0,628069.5,NaN,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,2.0


### Les 278 NA `AMT_GOODS_PRICE`

Ce sont tous des contrats de type `Cash loans`.

On tentera de croiser avec les données de bureau et de previous_application, mais si l'annuité n'est pas définie, il est difficile d'évaluer un risque qui dépend essentiellement de la charge à supporter en rapport avec les autres montants dimensionnants.

In [66]:
display(data[data.AMT_GOODS_PRICE.isna()].T.head(30))

,724,5937,6425,6703,7647,7880,7995,10819,11287,13008,...,300107,302923,303603,303621,304621,304678,304784,305833,306126,306273
application,,,,,,,,,,,,,,,,,,,,,
SK_ID_CURR,100837,106955,107494,107822,108913,109190,109322,112595,113148,115162,...,447675,450959,451760,451781,452926,452992,453120,454341,454683,454852
TARGET,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NAME_CONTRACT_TYPE,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,...,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans,Revolving loans
CODE_GENDER,F,F,F,F,M,F,M,F,F,F,...,F,F,F,F,F,F,M,F,M,M
FLAG_OWN_CAR,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
FLAG_OWN_REALTY,Y,N,N,N,Y,N,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
CNT_CHILDREN,2,0,0,1,0,0,0,0,4,1,...,1,0,0,0,0,2,1,0,0,2
AMT_INCOME_TOTAL,45000.0,157500.0,67500.0,121500.0,180000.0,121500.0,112500.0,90000.0,225000.0,157500.0,...,405000.0,126000.0,135000.0,112500.0,90000.0,67500.0,112500.0,126000.0,135000.0,67500.0
AMT_CREDIT,135000.0,450000.0,202500.0,180000.0,450000.0,270000.0,180000.0,270000.0,135000.0,450000.0,...,810000.0,337500.0,135000.0,180000.0,247500.0,202500.0,270000.0,270000.0,270000.0,202500.0


L'intersection avec les 36 NA `AMT_ANNUITY` est vide :

In [70]:
print(sum(data.AMT_ANNUITY.isna() & data.AMT_GOODS_PRICE.isna()))

0


### Les 64623 valeurs `365243` de `DAYS_EMPLOYED`

C'est un code pour NA, qui correspond à 100 ans.

On peut croiser avec les informations suivantes qui concernent également l'emploi :
- `AMT_INCOME_TOTAL`
- `FLAG_EMP_PHONE`
- `FLAG_WORK_PHONE`
- `NAME_INCOME_TYPE`
- `NAME_EDUCATION_TYPE`
- `OCCUPATION_TYPE`
- `ORGANIZATION_TYPE`

Cela nous permet de voir que l'essentiel 64 600, sont des retraités pensionnés, et seulement 23 des personnes privées d'emploi.

Cela est corroboré par :
- l'absence de téléphone employeur,
- l'absence de téléphone professionnel (sauf pour 1 cas),
- Un type d'occupation valant NA,
- Un type d'organisation valant XNA.

Le pensionné n'est plus en emploi, mais cela ne l'empêche pas d'avoir des revenus et d'être solvable.

Si l'on considère ce que représente cet indicateur, à savoir la durée continue de la dernière activité en cours, pour une personne privée d'emploi ou un retraité, nous pouvons simplement la fixer à 0 avec :

```python
data.DAYS_EMPLOYED.replace(365_243, 0, inplace=True)
```

In [4]:
print(sum(data.DAYS_EMPLOYED == 365243))
print(sum(data.DAYS_EMPLOYED > 0))

64648
64648


In [78]:
work_cols = [
    "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE",
    "FLAG_EMP_PHONE", "FLAG_WORK_PHONE",
    "NAME_EDUCATION_TYPE", "OCCUPATION_TYPE", "ORGANIZATION_TYPE"
]
work_data = data.loc[data.DAYS_EMPLOYED == 365243, work_cols]
display(work_data)
for col in work_cols:
    vc_dict = work_data[col].value_counts(dropna=False).to_dict()
    print(f"{col} ({len(vc_dict)}): {vc_dict}")

application,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,NAME_EDUCATION_TYPE,OCCUPATION_TYPE,ORGANIZATION_TYPE
8,112500.000,Pensioner,0,0,Secondary / secondary special,None,XNA
11,38419.155,Pensioner,0,0,Secondary / secondary special,None,XNA
23,83250.000,Pensioner,0,0,Secondary / secondary special,None,XNA
38,99000.000,Pensioner,0,0,Secondary / secondary special,None,XNA
43,108000.000,Pensioner,0,0,Higher education,None,XNA
...,...,...,...,...,...,...,...
356228,81000.000,Pensioner,0,0,Secondary / secondary special,None,XNA
356239,247500.000,Pensioner,0,0,Higher education,None,XNA
356242,112500.000,Pensioner,0,0,Higher education,None,XNA
356246,85500.000,Pensioner,0,0,Secondary / secondary special,None,XNA


AMT_INCOME_TOTAL (788): {112500.0: 6859, 135000.0: 6544, 90000.0: 5929, 157500.0: 4533, 67500.0: 4041, 180000.0: 3475, 225000.0: 2696, 202500.0: 2371, 81000.0: 1806, 54000.0: 1160, 270000.0: 1151, 126000.0: 1110, 99000.0: 1052, 76500.0: 1047, 45000.0: 1024, 103500.0: 917, 121500.0: 895, 72000.0: 867, 144000.0: 729, 247500.0: 726, 117000.0: 678, 94500.0: 645, 58500.0: 626, 85500.0: 594, 108000.0: 584, 63000.0: 561, 315000.0: 531, 292500.0: 525, 171000.0: 525, 49500.0: 505, 166500.0: 441, 162000.0: 438, 130500.0: 370, 360000.0: 363, 148500.0: 340, 189000.0: 324, 153000.0: 276, 216000.0: 265, 139500.0: 251, 40500.0: 230, 175500.0: 202, 450000.0: 202, 193500.0: 195, 36000.0: 194, 337500.0: 184, 211500.0: 178, 405000.0: 170, 31500.0: 138, 207000.0: 134, 184500.0: 127, 382500.0: 124, 234000.0: 121, 252000.0: 104, 56250.0: 101, 198000.0: 91, 47250.0: 88, 60750.0: 84, 261000.0: 81, 51750.0: 78, 220500.0: 76, 427500.0: 73, 83250.0: 73, 74250.0: 70, 243000.0: 68, 256500.0: 67, 65250.0: 66, 78750

### Pourquoi `WEEKDAY_APPR_PROCESS_START` est inférée en tant que `object` ?

Car ce sont les noms des jours et non leur code de position ordinale dans la semaine.

Pour encoder, il suffit donc de mapper avec `index` sur la liste (locale `en_US`) des noms de jours de la semaine.

In [5]:
display(data.WEEKDAY_APPR_PROCESS_START.value_counts(dropna=False))

TUESDAY      63652
WEDNESDAY    60391
MONDAY       59120
THURSDAY     59009
FRIDAY       57588
SATURDAY     38455
SUNDAY       18040
Name: WEEKDAY_APPR_PROCESS_START, dtype: int64

In [13]:
from pepper.utils import get_weekdays
weekdays = get_weekdays()
print(weekdays)
print(weekdays.index("TUESDAY"))

['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']
1


In [17]:
import pandas as pd
wd_str = data.WEEKDAY_APPR_PROCESS_START
wd_code = wd_str.replace({d: i for i, d in enumerate(weekdays)})
print(pd.concat([wd_str, wd_code], axis=1))

        WEEKDAY_APPR_PROCESS_START  WEEKDAY_APPR_PROCESS_START
0                        WEDNESDAY                           2
1                           MONDAY                           0
2                           MONDAY                           0
3                        WEDNESDAY                           2
4                         THURSDAY                           3
...                            ...                         ...
356250                   WEDNESDAY                           2
356251                      MONDAY                           0
356252                   WEDNESDAY                           2
356253                      MONDAY                           0
356254                     TUESDAY                           1

[356255 rows x 2 columns]


### Pourquoi `REGION_RATING_CLIENT_W_CITY` est dans [-1, 3] mais ne prend que 4 valeurs ?

Il n'y a qu'un seul cas à -1 !

Et effectivement, la note de la région devrait être 1, 2, ou 3.

C'est peut-être l'occasion d'un hack très simple.

Les valeurs flottantes relatives des populations des régions (`REGION_POPULATION_RELATIVE`) n'ont que 82 valeurs uniques, qui laissent entendre que les valeurs actuelles sont synchrones dans la table pour une même région à l'instant t, probablement en relation avec une table des régions qui n'a pas pas été fournie avec le jeu de données.

In [18]:
display(data.REGION_RATING_CLIENT_W_CITY.value_counts(dropna=False))

 2    265260
 3     50652
 1     40342
-1         1
Name: REGION_RATING_CLIENT_W_CITY, dtype: int64

In [22]:
display(data[data.REGION_RATING_CLIENT_W_CITY == -1])

application,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
324688,224393,-1,Cash loans,F,N,N,0,270000.0,385164.0,16321.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


L'agrégation des scores de région et de ville par région identifiée par sa population donne un résultat intéressant :
- Il y a 82 régions.
- Le nombre de clients est en général plus important dans les régions de population plus importante, mais ce n'est pas une règle absolue, puisque cela dépend du taux de pénétration de ses marchés par Home Credit.
- Il n'y a qu'une seule note de région qui est la même pour l'ensemble des clients de cette région.
- Dans la majorité des régions (3 exceptions) les notes de villes sont identiques.
- Dans les 3 régions avec plusieurs notes de villes, ces notes sont toujours inférieures ou égale à celle de la région.
- Le cas unique avec une note de ville à -1 correspond à l'unique client (`224393`) de la plus petite région (en population), notée à 2.

Par conséquent, il n'est pas déraisonnable d'attribuer la note 2 à la ville de ce client (`224393`).

In [32]:
reg_cols = ["REGION_POPULATION_RELATIVE", "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY"]
reg_data = data[reg_cols]

def vc_dict(x):
    return x.value_counts(dropna=False).to_dict()

grouped = reg_data.groupby(by="REGION_POPULATION_RELATIVE").agg(vc_dict)
display(grouped)

application,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY
REGION_POPULATION_RELATIVE,,
0.000253,{2: 1},{-1: 1}
0.000290,{2: 2},{2: 2}
0.000533,{3: 40},{3: 40}
0.000938,{3: 33},{3: 33}
0.001276,{2: 627},{2: 627}
...,...,...
0.031329,{2: 12916},{2: 12916}
0.032561,{1: 7724},{1: 7724}
0.035792,{2: 18990},{2: 18990}


Nombre de scores distincts :

In [38]:
grouped_dict_lens = grouped.apply(lambda x: x.apply(len))
display(grouped_dict_lens)
display(grouped_dict_lens[grouped_dict_lens.sum(axis=1) > 2])
display(grouped[grouped_dict_lens.sum(axis=1) > 2])

application,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY
REGION_POPULATION_RELATIVE,,
0.000253,1,1
0.000290,1,1
0.000533,1,1
0.000938,1,1
0.001276,1,1
...,...,...
0.031329,1,1
0.032561,1,1
0.035792,1,1


application,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY
REGION_POPULATION_RELATIVE,,
0.010006,1,2
0.018029,1,2
0.020713,1,3


application,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY
REGION_POPULATION_RELATIVE,,
0.010006,{2: 4237},"{2: 2725, 1: 1512}"
0.018029,{3: 7064},"{3: 5863, 2: 1201}"
0.020713,{3: 9393},"{3: 5292, 2: 3232, 1: 869}"


Différence entre le score de région et le score de ville :

In [42]:
grouped_dict_1st_key = grouped.apply(lambda x: x.apply(lambda y: list(y.keys())[0]))
display(grouped_dict_1st_key)
display(grouped_dict_1st_key[
    grouped_dict_1st_key.REGION_RATING_CLIENT
    != grouped_dict_1st_key.REGION_RATING_CLIENT_W_CITY
])
#display(grouped[grouped_dict_lens.sum(axis=1) > 2])

application,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY
REGION_POPULATION_RELATIVE,,
0.000253,2,-1
0.000290,2,2
0.000533,3,3
0.000938,3,3
0.001276,2,2
...,...,...
0.031329,2,2
0.032561,1,1
0.035792,2,2


application,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY
REGION_POPULATION_RELATIVE,,
0.000253,2,-1


### Les 668 NA de `EXT_SOURCE_2`

`EXT_SOURCE_2` évalue presque 100 % des clients, contrairement aux deux autres sources externes.

Qu'est ce qui distingue ces 668 clients ?

Rien de particulièrement discriminant ne ressort en première analyse.

In [49]:
display(data[data.EXT_SOURCE_2.isna()].T.head(60))

,329,349,617,1028,1520,2098,2426,2448,2756,3212,...,307029,307387,316023,317969,328160,338813,342465,347261,349108,355455
application,,,,,,,,,,,,,,,,,,,,,
SK_ID_CURR,100377,100402,100706,101189,101787,102466,102831,102855,103215,103750,...,455713,456113,162248,175958,249770,327587,355560,392730,405125,450310
TARGET,0,0,0,0,0,0,0,0,0,0,...,0,0,-1,-1,-1,-1,-1,-1,-1,-1
NAME_CONTRACT_TYPE,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,...,Cash loans,Cash loans,Cash loans,Cash loans,Revolving loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans
CODE_GENDER,M,F,F,F,M,F,F,F,F,F,...,F,M,F,M,F,F,F,F,F,F
FLAG_OWN_CAR,N,N,N,Y,Y,N,N,Y,N,N,...,Y,N,N,N,N,Y,Y,Y,N,N
FLAG_OWN_REALTY,N,Y,Y,Y,Y,N,Y,Y,N,Y,...,N,Y,Y,N,N,Y,Y,Y,N,Y
CNT_CHILDREN,2,0,0,0,0,0,0,0,0,0,...,2,3,0,0,0,0,0,0,0,0
AMT_INCOME_TOTAL,157500.0,81000.0,180000.0,121500.0,225000.0,135000.0,49500.0,202500.0,112500.0,54000.0,...,153000.0,180000.0,315000.0,157500.0,90000.0,135000.0,225000.0,112500.0,126000.0,157500.0
AMT_CREDIT,239850.0,276277.5,533668.5,909000.0,269550.0,432661.5,292500.0,912240.0,253737.0,135000.0,...,808650.0,1096020.0,900000.0,180000.0,157500.0,450000.0,1354500.0,479700.0,260640.0,450000.0


In [50]:
some_cols = [
    "NAME_CONTRACT_TYPE", "CODE_GENDER", "NAME_TYPE_SUITE"
]
some_data = data.loc[data.EXT_SOURCE_2.isna(), some_cols]
#display(some_data)
for col in some_cols:
    vc_dict = some_data[col].value_counts(dropna=False).to_dict()
    print(f"{col} ({len(vc_dict)}): {vc_dict}")

NAME_CONTRACT_TYPE (2): {'Cash loans': 606, 'Revolving loans': 62}
CODE_GENDER (2): {'F': 509, 'M': 159}
NAME_TYPE_SUITE (8): {'Unaccompanied': 525, 'Family': 89, 'Spouse, partner': 30, 'Children': 12, 'Other_B': 8, 'Group of people': 2, 'Other_A': 1, None: 1}


### Que sont ces 4 faux amis, les 4 derniers `_MODE` ?

`FONDKAPREMONT_MODE`, `HOUSETYPE_MODE`, `WALLSMATERIAL_MODE`, `EMERGENCYSTATE_MODE` complètent la section de statistiques de tendance centrale d'indicateurs sur le lieu de résidence du demandeur. Leur définition est identique à celle des autres variables de cette famille, sans précision sur le rôle de chacune d'entre elles, qu'il faut deviner d'après le nom de la variable.

Ces 4 variables ne font pourtant pas partie de cette famille, car ce sont des variables catégorielles.

Jetons un oeil à leurs modalités.

In [51]:
some_cols = [
    "FONDKAPREMONT_MODE", "HOUSETYPE_MODE", "WALLSMATERIAL_MODE", "EMERGENCYSTATE_MODE"
]
some_data = data[some_cols]
#display(some_data)
for col in some_cols:
    vc_dict = some_data[col].value_counts(dropna=False).to_dict()
    print(f"{col} ({len(vc_dict)}): {vc_dict}")

FONDKAPREMONT_MODE (5): {None: 243092, 'reg oper account': 85954, 'reg oper spec account': 14070, 'not specified': 6600, 'org spec account': 6539}
HOUSETYPE_MODE (4): {None: 177916, 'block of flats': 175162, 'specific housing': 1761, 'terraced house': 1416}
WALLSMATERIAL_MODE (8): {None: 180234, 'Panel': 77309, 'Stone, brick': 75249, 'Block': 10681, 'Wooden': 6156, 'Mixed': 2649, 'Monolithic': 2068, 'Others': 1909}
EMERGENCYSTATE_MODE (3): {'No': 185607, None: 167964, 'Yes': 2684}


### Les 1 050 NA de `_CIRCLE`

Ces 4 variables entières positives donnent une statistique sur l'ensemble du _cercle social_ auquel appartient le demandeur. Seulement, la définition de ce cercle n'est pas communiquée.

Il y a 36 x 10 = 360 classes pour `30_CNT` et 35 x 9 = 315 classes pour `60_CNT`.

Ce qui suit fait suspecter que ces 4 variables sont plus susceptible de produire du bruit que de d'aider à la prédiction. Les échantillons sont notamment trop peu représentatifs. **Il faudra donc essayer d'entraîner les modèles avec et sans**. Enfin, dans le cadre de l'objectif de transparence, évoquer un cercle social, d'autant plus qu'il est mal défini, risque de poser problème.

Pour imputer les 1 050 NA, il suffit d'affecter 0 aux 4 variables.

Les ratios seront ainsi tout aussi NA que ceux de la configuration majoritaire qui est justement 0, 0, 0, 0 (absence de données).

In [58]:
display(data[data.OBS_30_CNT_SOCIAL_CIRCLE.isna()])

application,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
68,100080,0,Revolving loans,F,N,N,1,157500.0,450000.0,22500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0
394,100457,0,Revolving loans,F,Y,Y,0,211500.0,270000.0,13500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
397,100460,0,Revolving loans,F,N,N,0,315000.0,540000.0,27000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
457,100527,0,Revolving loans,F,N,N,0,112500.0,270000.0,13500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1042,101209,0,Revolving loans,M,Y,N,0,180000.0,540000.0,27000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344447,370323,-1,Revolving loans,F,Y,Y,0,360000.0,1237500.0,61875.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
344629,371514,-1,Revolving loans,F,Y,N,0,112500.0,157500.0,7875.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
352518,429623,-1,Revolving loans,F,N,Y,0,54000.0,450000.0,22500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
352610,430300,-1,Revolving loans,M,Y,Y,0,450000.0,1350000.0,67500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
circle_cols = [
    "OBS_30_CNT_SOCIAL_CIRCLE", "DEF_30_CNT_SOCIAL_CIRCLE",
    "OBS_60_CNT_SOCIAL_CIRCLE", "DEF_60_CNT_SOCIAL_CIRCLE"
]
circle_data = data[circle_cols].dropna().copy()
circle_data.columns = ["OBS_30", "DEF_30", "OBS_60", "DEF_60"]
circle_data_unique = circle_data.drop_duplicates()
display(circle_data_unique)
#display(circle_data_unique.to_records(index=False))
display(pd.DataFrame(circle_data.apply(lambda row: tuple(row), axis=1).value_counts()))


,OBS_30,DEF_30,OBS_60,DEF_60
0,2.0,2.0,2.0,2.0
1,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0
3,2.0,0.0,2.0,0.0
17,4.0,0.0,4.0,0.0
...,...,...,...,...
346380,354.0,34.0,351.0,24.0
346448,12.0,3.0,11.0,1.0
347075,9.0,3.0,8.0,1.0
347182,352.0,34.0,349.0,24.0


,0
"(0.0, 0.0, 0.0, 0.0)",189935
"(1.0, 0.0, 1.0, 0.0)",43466
"(2.0, 0.0, 2.0, 0.0)",24810
"(3.0, 0.0, 3.0, 0.0)",16685
"(4.0, 0.0, 4.0, 0.0)",11443
...,...
"(9.0, 6.0, 9.0, 5.0)",1
"(15.0, 4.0, 15.0, 4.0)",1
"(9.0, 2.0, 7.0, 2.0)",1
"(8.0, 1.0, 6.0, 0.0)",1


### Les 47 568 NA de `AMT_REQ_CREDIT_BUREAU_`

Ces 6 variables comptent le nombre de demandes d'information effectuées auprès du Bureau concernant le demandeur, dans des délais plus ou moins long avant la demande.

Les profils ne font rien ressortir de particulier, sinon qu'ils sont a priori de bons clients.

Ces variables pourraient quasiment être traitées comme des catégories.

Seul `QRT` a un nombre important de valeurs possibles, jusqu'à 261, mais en nombre de valeurs uniques, cela reste 25 pour que ce cas maximum.

Dans tous les cas, il est difficile d'imputer une valeur qui dépend complètement du comportement des chargés de clientèle.

Codons cette valeur à -1 de le temps des pré-traitements, pour éviter du `float64`, mais il ne faudra pas oublier de retraiter avant de passer à l'entraînement des modèles.

In [78]:
display(data[data.AMT_REQ_CREDIT_BUREAU_HOUR.isna()])

application,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
9,100012,0,Revolving loans,M,N,Y,0,135000.0,405000.0,20250.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
14,100018,0,Cash loans,F,N,Y,0,189000.0,773680.5,32778.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
17,100021,0,Revolving loans,F,N,Y,1,81000.0,270000.0,13500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
20,100024,0,Revolving loans,M,Y,Y,0,135000.0,427500.0,21375.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356211,455854,-1,Cash loans,M,N,N,0,216000.0,454500.0,26091.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
356227,456007,-1,Cash loans,M,N,Y,0,135000.0,318645.0,16398.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
356240,456120,-1,Cash loans,F,N,N,2,81000.0,135000.0,16020.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
356246,456169,-1,Cash loans,F,N,N,0,85500.0,109008.0,7411.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


# **`installments_payments`**

Cette table à 13 602 496 entrées est la clé de compréhension et de correction des données financières. Elle comporte le détail le plus fin des mouvements sur les comptes de carte de crédit et de prêts en cash.

Elle n'est pas triviale à interpréter et donc ensuite à traiter, notamment pour l'agrégation.

Les relations clés à bien maîtriser sont celles entre les numéros d'échéances, les versions de l'échéancier, les montants appelés et perçus.

## Chargement des données

In [68]:
from home_credit.tables import InstallmentsPayments
from home_credit.utils import display_frame_basic_infos

data = InstallmentsPayments.clean()
display_frame_basic_infos(data)
display(data)

n_samples: 13 602 496
n_columns: 7, [('NUM', 1), ('DAYS', 2), ('AMT', 2), ('MONTHS', 1), ('TARGET', 1)]


CLEAN_INSTALLMENTS_PAYMENTS                  MONTHS_BALANCE  TARGET  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                           
100001     1369693    1                                  57      -1   
                      2                                  56      -1   
                      3                                  55      -1   
                      4                                  54      -1   
           1851984    2                                  96      -1   
...                                                     ...     ...   
456255     2631384    23                                  4       0   
                      24                                  3       0   
           2729207    1                                  16       0   
                      2                                  15       0   
                      3                                  14       0   

CLEAN_INSTALLMENTS_PAYMENTS                  NUM_INSTALMENT_VERSION  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                           
100001     1369693    1                                           1   
                      2                                           1   
                      3                                           1   
                      4                                           2   
           1851984    2                                           1   
...                                                             ...   
456255     2631384    23                                          3   
                      24                                          4   
           2729207    1                                           1   
                      2                                           1   
                      3                                           2   

CLEAN_INSTALLMENTS_PAYMENTS                  DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                    
100001     1369693    1                                 1709   
                      2                                 1679   
                      3                                 1649   
                      4                                 1619   
           1851984    2                                 2916   
...                                                      ...   
456255     2631384    23                                  96   
                      24                                  66   
           2729207    1                                  469   
                      2                                  439   
                      3                                  409   

CLEAN_INSTALLMENTS_PAYMENTS                  DAYS_ENTRY_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                       
100001     1369693    1                                    1715   
                      2                                    1715   
                      3                                    1660   
                      4                                    1628   
           1851984    2                                    2916   
...                                                         ...   
456255     2631384    23                                     98   
                      24                                     76   
           2729207    1                                     482   
                      2                                     455   
                      3                                     435   

CLEAN_INSTALLMENTS_PAYMENTS                  AMT_INSTALMENT    AMT_PAYMENT  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100001     1369693    1                         3951.000000    3951.000000  
                      2                         3951.000000    3951.000000  
                      3                         3951.000000    3951.000000  
                      4                        17397.900391   

## Extension de la table avec le _Principal Outstanding_

`AMT_OUTSTANDING` (_Principal Outstanding_) est la différence entre le montant de l'échéance et le montant effectivement payé par le client.

Dans 90 % des cas, l'échéance a été intégralement réglée (9,52 % d'outstanding).

In [2]:
data["AMT_OUTSTANDING"] = data.AMT_INSTALMENT - data.AMT_PAYMENT
outstanding = data[data.AMT_OUTSTANDING > 0]
n_samples = data.shape[0]
n_outstanding = outstanding.shape[0]

print(f"% of oustanding installments: {100*n_outstanding/n_samples:.2f} %")

display(outstanding)

% of oustanding installments: 9.52 %


CLEAN_INSTALLMENTS_PAYMENTS                  MONTHS_BALANCE  TARGET  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                           
100007     1940724    1                                  36       0   
                      1                                  36       0   
           2462742    10                                 49       0   
                      10                                 49       0   
                      12                                 47       0   
...                                                     ...     ...   
456255     2631384    18                                  9       0   
                      20                                  7       0   
                      20                                  7       0   
                      22                                  5       0   
                      22                                  5       0   

CLEAN_INSTALLMENTS_PAYMENTS                  NUM_INSTALMENT_VERSION  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                           
100007     1940724    1                                           1   
                      1                                           1   
           2462742    10                                          1   
                      10                                          1   
                      12                                          1   
...                                                             ...   
456255     2631384    18                                          3   
                      20                                          3   
                      20                                          3   
                      22                                          3   
                      22                                          3   

CLEAN_INSTALLMENTS_PAYMENTS                  DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                    
100007     1940724    1                                 1076   
                      1                                 1076   
           2462742    10                                1466   
                      10                                1466   
                      12                                1406   
...                                                      ...   
456255     2631384    18                                 246   
                      20                                 186   
                      20                                 186   
                      22                                 126   
                      22                                 126   

CLEAN_INSTALLMENTS_PAYMENTS                  DAYS_ENTRY_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                       
100007     1940724    1                                    1074   
                      1                                    1106   
           2462742    10                                   1459   
                      10                                   1497   
                      12                                   1436   
...                                                         ...   
456255     2631384    18                                    266   
                      20                                    182   
                      20                                    221   
                      22                                    160   
                      22                                    123   

CLEAN_INSTALLMENTS_PAYMENTS                  AMT_INSTALMENT   AMT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100007     1940724    1                        22678.785156  22655.654297   
                      1                        22678.785156     23.129999   
           2462742    10                        3601.530029   3598.290039   
                      10                        3601.530029   

## Versions d'une échéance vs. versions de l'échéancier

Une échéance peut avoir plusieurs versions.

Cela signifie _a priori_ qu'elle peut être payée en plusieurs fractions.

Si on fait une somme bête des des outstandings on passe à côté, car en cas d'échéance fractionnée, l'outstanding comptera plusieurs fois le même montant de dette. Il faudrait plutôt calculer l'oustanding de l'échéance comme la différence entre le montant de l'échéance (`AMT_INSTALMENT`) répété à l'identique sur les différentes versions, et la somme des règlements partiels de cette dette (`AMT_PAYMENT`).

La première hypothèse que nous nous devons donc valider est que :

_une échéance qui apparaît en plusieurs versions a toujours le même montant_.

### Tri de la table par échéance puis version

Pour tenter d'y voir plus clair, commençons par ordonner la table par échéance puis numéro de version :

In [6]:
version = data[[
    "NUM_INSTALMENT_VERSION",
    "AMT_INSTALMENT", "AMT_PAYMENT", "AMT_OUTSTANDING"
]].reset_index()
version = version.sort_values(by=[
    "SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER", "NUM_INSTALMENT_VERSION"
])
display(version)

CLEAN_INSTALLMENTS_PAYMENTS,SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,AMT_INSTALMENT,AMT_PAYMENT,AMT_OUTSTANDING
0,100001,1369693,1,1,3951.000000,3951.000000,0.0
1,100001,1369693,2,1,3951.000000,3951.000000,0.0
2,100001,1369693,3,1,3951.000000,3951.000000,0.0
3,100001,1369693,4,2,17397.900391,17397.900391,0.0
4,100001,1851984,2,1,3982.050049,3982.050049,0.0
...,...,...,...,...,...,...,...
13602491,456255,2631384,23,3,27489.689453,27489.689453,0.0
13602492,456255,2631384,24,4,308277.312500,308277.312500,0.0
13602493,456255,2729207,1,1,11514.554688,11514.554688,0.0
13602494,456255,2729207,2,1,11514.554688,11514.554688,0.0


### Agrégation par échéance

Les cas qui demandent une vérification sont ceux où les échéances ont des versions multiples.

Pour les isoler, produisons une table agrégée des échéances où l'on détermine :
- le _nombre_ de versions de l'échéance (et accessoirement les ordinaux _min_ et _max_ de versions)
- les montants _min_ et _max_ des échéances et de leurs règlements (pour nous assurer de leur constance)
- la _somme_ des règlements (pour nous assurer qu'elle coïncide avec le montant de l'échéance)

Cette table a 12 859 104 entrées : la majorité des échéances n'ont donc qu'une version.

In [7]:
aggregated_version = version.groupby(by=[
    "SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER"
])

aggregated_version = aggregated_version.agg({
    "NUM_INSTALMENT_VERSION" : ["min", "max", "count"],
    "AMT_INSTALMENT" : ["min", "max"],
    "AMT_PAYMENT": ["min", "max", "sum"]
})

display(aggregated_version)

CLEAN_INSTALLMENTS_PAYMENTS                 NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100001     1369693    1                                          1   1     1   
                      2                                          1   1     1   
                      3                                          1   1     1   
                      4                                          2   2     1   
           1851984    2                                          1   1     1   
...                                                            ...  ..   ...   
456255     2631384    23                                         3   3     1   
                      24                                         4   4     1   
           2729207    1                                          1   1     1   
                      2                                          1   1     1   
                      3                                          2   2     1   

CLEAN_INSTALLMENTS_PAYMENTS                 AMT_INSTALMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100001     1369693    1                        3951.000000    3951.000000   
                      2                        3951.000000    3951.000000   
                      3                        3951.000000    3951.000000   
                      4                       17397.900391   17397.900391   
           1851984    2                        3982.050049    3982.050049   
...                                                    ...            ...   
456255     2631384    23                      27489.689453   27489.689453   
                      24                     308277.312500  308277.312500   
           2729207    1                       11514.554688   11514.554688   
                      2                       11514.554688   11514.554688   
                      3                       42754.230469   42754.230469   

CLEAN_INSTALLMENTS_PAYMENTS                    AMT_PAYMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100001     1369693    1                        3951.000000    3951.000000   
                      2                        3951.000000    3951.000000   
                      3                        3951.000000    3951.000000   
                      4                       17397.900391   17397.900391   
           1851984    2                        3982.050049    3982.050049   
...                                                    ...            ...   
456255     2631384    23                      27489.689453   27489.689453   
                      24                     308277.312500  308277.312500   
           2729207    1                       11514.554688   11514.554688   
                      2                       11514.554688   11514.554688   
                      3                       42754.230469   42754.230469   

CLEAN_INSTALLMENTS_PAYMENTS                                 
                                                       sum  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                 
100001     1369693    1                        3951.000000  
                      2                        3951.000000  
                      3                        3951.000000  
                      4                       17397.900391  
           1851984    2                        3982.050049  
...                                                    ...  
456255     2631384    23                      27489.689453  
                      24                     308277.312500  
           2729207    1                       1151

### Échéances à versions multiples

Sur la base de l'agrégation précédente nous pouvons préciser que :
- 94 % des échéances n'ont qu'une seule version,
- 730 619 cas (sur 12 859 104) sont multi-versions,
- 5,5 % en ont deux,
- moins de 1% en ont 3,
- il peut y avoir jusqu'à 12 versions,
- les cas à plus de 3 versions sont marginaux.

In [14]:
vc_dict = dict(aggregated_version[("NUM_INSTALMENT_VERSION", "count")].value_counts())
print(vc_dict)
vc_dict_norm = dict(aggregated_version[("NUM_INSTALMENT_VERSION", "count")].value_counts(normalize=True))
print({k: f"{100*v:.3f} %" for k, v in vc_dict_norm.items()})

multi_version = aggregated_version[aggregated_version[("NUM_INSTALMENT_VERSION", "count")] > 1]
display(multi_version)

{1: 12128485, 2: 718875, 3: 11001, 4: 586, 6: 69, 5: 68, 8: 10, 7: 7, 10: 1, 9: 1, 12: 1}
{1: '94.318 %', 2: '5.590 %', 3: '0.086 %', 4: '0.005 %', 6: '0.001 %', 5: '0.001 %', 8: '0.000 %', 7: '0.000 %', 10: '0.000 %', 9: '0.000 %', 12: '0.000 %'}


CLEAN_INSTALLMENTS_PAYMENTS                 NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100007     1940724    1                                          1   1     2   
           2462742    10                                         1   1     2   
                      12                                         1   1     2   
100008     1907290    10                                         1   1     2   
100011     1430602    9                                          1   1     2   
...                                                            ...  ..   ...   
456255     2631384    1                                          1   2     2   
                      17                                         3   3     2   
                      18                                         3   3     2   
                      20                                         3   3     2   
                      22                                         3   3     2   

CLEAN_INSTALLMENTS_PAYMENTS                 AMT_INSTALMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100007     1940724    1                       22678.785156   22678.785156   
           2462742    10                       3601.530029    3601.530029   
                      12                       3577.050049    3577.050049   
100008     1907290    10                      11986.155273   11986.155273   
100011     1430602    9                       14614.334961   14614.334961   
...                                                    ...            ...   
456255     2631384    1                       54022.140625  615229.500000   
                      17                      27489.689453   27489.689453   
                      18                      27489.689453   27489.689453   
                      20                      27489.689453   27489.689453   
                      22                      27489.689453   27489.689453   

CLEAN_INSTALLMENTS_PAYMENTS                    AMT_PAYMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100007     1940724    1                          23.129999   22655.654297   
           2462742    10                          3.240000    3598.290039   
                      12                          0.180000    3576.870117   
100008     1907290    10                        227.160004   11758.995117   
100011     1430602    9                        5614.334961    9000.000000   
...                                                    ...            ...   
456255     2631384    1                      669251.625000  669251.625000   
                      17                        737.909973   26751.779297   
                      18                        110.879997   27378.810547   
                      20                        447.255005   27042.435547   
                      22                        457.875000   27031.814453   

CLEAN_INSTALLMENTS_PAYMENTS                                
                                                      sum  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                
100007     1940724    1                      2.267879e+04  
           2462742    10                     3.601530e+03  
                      12                     3.577050e+03  
100008     1907290    10                     1.198616e+04  
100011     1430602    9                      1.461433e+04  
...                                                   ...  
456255     2631384    1                      1.338503e+06  
                      17                     2.748969e+04  
                      18                     2.748969e+04  
 

### Constance de l'échéance

Dans l'extrait ci-dessus, on observe que l'hypothèse de constance de l'échéance est majoritairement vérifiée, mais pas absolument.

Ajoutons une colonne de _diff_ entre les _min_ et _max_ du montant d'échéance, afin de pouvoir isoler les cas qui violent la relation d'identité.

In [15]:
if ("AMT_INSTALMENT", "diff") in multi_version:
    multi_version = multi_version.drop(columns=("AMT_INSTALMENT", "diff"))
amt_inst_max = multi_version[("AMT_INSTALMENT", "max")]
amt_inst_min = multi_version[("AMT_INSTALMENT", "min")]
amt_inst_diff = amt_inst_max - amt_inst_min
multi_version.insert(5, ("AMT_INSTALMENT", "diff"), amt_inst_diff)
display(multi_version)

CLEAN_INSTALLMENTS_PAYMENTS                 NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100007     1940724    1                                          1   1     2   
           2462742    10                                         1   1     2   
                      12                                         1   1     2   
100008     1907290    10                                         1   1     2   
100011     1430602    9                                          1   1     2   
...                                                            ...  ..   ...   
456255     2631384    1                                          1   2     2   
                      17                                         3   3     2   
                      18                                         3   3     2   
                      20                                         3   3     2   
                      22                                         3   3     2   

CLEAN_INSTALLMENTS_PAYMENTS                 AMT_INSTALMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100007     1940724    1                       22678.785156   22678.785156   
           2462742    10                       3601.530029    3601.530029   
                      12                       3577.050049    3577.050049   
100008     1907290    10                      11986.155273   11986.155273   
100011     1430602    9                       14614.334961   14614.334961   
...                                                    ...            ...   
456255     2631384    1                       54022.140625  615229.500000   
                      17                      27489.689453   27489.689453   
                      18                      27489.689453   27489.689453   
                      20                      27489.689453   27489.689453   
                      22                      27489.689453   27489.689453   

CLEAN_INSTALLMENTS_PAYMENTS                                AMT_PAYMENT  \
                                                   diff            min   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                              
100007     1940724    1                           0.000      23.129999   
           2462742    10                          0.000       3.240000   
                      12                          0.000       0.180000   
100008     1907290    10                          0.000     227.160004   
100011     1430602    9                           0.000    5614.334961   
...                                                 ...            ...   
456255     2631384    1                      561207.375  669251.625000   
                      17                          0.000     737.909973   
                      18                          0.000     110.879997   
                      20                          0.000     447.255005   
                      22                          0.000     457.875000   

CLEAN_INSTALLMENTS_PAYMENTS                                               
                                                       max           sum  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                               
100007     1940724    1                       22655.654297  2.267879e+04  
           2462742    10                       3598.290039  3.601530e+03  
                      12                       3576.870117  3.577050e+03  
100008     1907290    10                      11758.995117  1.198616e+04  
100011     1430602    9                        9000.000000  1.461433e+04  
...                                                    ...           ...  
456255     2631384    1                      669251.625000  1.338503e+06  
             

### Échéances à montant révisé

Sur les 730 619 échéance à versions multiples, 89 857 cas (12,3 %) ne respectent pas l'hypothèse de constance, et on a donc alors des montants d'échéance révisés d'une version à l'autre.

In [17]:
outliers = multi_version[multi_version[("AMT_INSTALMENT", "diff")] > 0]
n_multi_version = multi_version.shape[0]
n_outliers = outliers.shape[0]
print(f"% outliers: {100*n_outliers/n_multi_version:.2f}")
display(outliers)

% outliers: 12.30


CLEAN_INSTALLMENTS_PAYMENTS                 NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100012     2480304    1                                          1   2     2   
100019     1620327    6                                          1   2     2   
100034     1390369    1                                          1   2     2   
100039     1077565    1                                          1   2     2   
                      2                                          3   4     2   
...                                                            ...  ..   ...   
456202     1333058    1                                          1   2     2   
456207     2230233    1                                          1   2     2   
456227     1772808    7                                          1   2     2   
456240     1508947    7                                          1   2     2   
456255     2631384    1                                          1   2     2   

CLEAN_INSTALLMENTS_PAYMENTS                 AMT_INSTALMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100012     2480304    1                        9482.355469   49204.890625   
100019     1620327    6                       10518.615234   33850.664062   
100034     1390369    1                       22430.429688   36069.570312   
100039     1077565    1                       52513.515625  172486.484375   
                      2                       46326.734375  178673.265625   
...                                                    ...            ...   
456202     1333058    1                        7147.890137   10852.110352   
456207     2230233    1                        4403.250000   36096.750000   
456227     1772808    7                       11813.580078   38804.941406   
456240     1508947    7                       16721.910156   18542.429688   
456255     2631384    1                       54022.140625  615229.500000   

CLEAN_INSTALLMENTS_PAYMENTS                                   AMT_PAYMENT  \
                                                      diff            min   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100012     2480304    1                       39722.535156   58687.246094   
100019     1620327    6                       23332.048828   44369.281250   
100034     1390369    1                       13639.140625   58500.000000   
100039     1077565    1                      119972.968750  225000.000000   
                      2                      132346.531250  225000.000000   
...                                                    ...            ...   
456202     1333058    1                        3704.220215   18000.000000   
456207     2230233    1                       31693.500000   40500.000000   
456227     1772808    7                       26991.361328   50618.519531   
456240     1508947    7                        1820.519531   35264.339844   
456255     2631384    1                      561207.375000  669251.625000   

CLEAN_INSTALLMENTS_PAYMENTS                                               
                                                       max           sum  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                               
100012     2480304    1                       58687.246094  1.173745e+05  
100019     1620327    6                       44369.281250  8.873856e+04  
100034     1390369    1                       58500.000000  1.170000e+05  
100039     1077565    1                      225000.000000  4.500000e+05  
                      2                      225000.000000  4.500000e+05  
...                                                    ...           ...  
456202     1333058    1                       

### Égalité de l'échéance constante et de la somme des règlements

Avant d'approfondir notre étude du cas des échéances à montant révisé, validons que la somme des règlements équilibre le montant des échéances constantes, et conduit à un solde nul (dans les cas où le client ne fait pas défaut).

On extrait les cas constants.

Cela met en évidence un fait inattendu : **les cas de règlements fractionnés correspondent à plusieurs lignes de la table de base, mais avec le même numéro de version. La table comporte donc autant de lignes que de fraction de règlement pour un couple (échéance, version)**.

Cela remet donc en question l'hypothèse formulée plus haut que les versions sont associées à des règlements partiels d'une échéance.

En revanche, cela nous dit que la première étape de traitement de la table pour une agrégation correcte des données consiste à grouper par échéance et version, en effectuant la somme des règlements.

In [19]:
const_inst_multi = multi_version[multi_version[("AMT_INSTALMENT", "diff")] == 0]
display(const_inst_multi)

CLEAN_INSTALLMENTS_PAYMENTS                 NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100007     1940724    1                                          1   1     2   
           2462742    10                                         1   1     2   
                      12                                         1   1     2   
100008     1907290    10                                         1   1     2   
100011     1430602    9                                          1   1     2   
...                                                            ...  ..   ...   
456255     2073384    2                                          1   1     2   
           2631384    17                                         3   3     2   
                      18                                         3   3     2   
                      20                                         3   3     2   
                      22                                         3   3     2   

CLEAN_INSTALLMENTS_PAYMENTS                 AMT_INSTALMENT                     \
                                                       min           max diff   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                     
100007     1940724    1                       22678.785156  22678.785156  0.0   
           2462742    10                       3601.530029   3601.530029  0.0   
                      12                       3577.050049   3577.050049  0.0   
100008     1907290    10                      11986.155273  11986.155273  0.0   
100011     1430602    9                       14614.334961  14614.334961  0.0   
...                                                    ...           ...  ...   
456255     2073384    2                       16400.609375  16400.609375  0.0   
           2631384    17                      27489.689453  27489.689453  0.0   
                      18                      27489.689453  27489.689453  0.0   
                      20                      27489.689453  27489.689453  0.0   
                      22                      27489.689453  27489.689453  0.0   

CLEAN_INSTALLMENTS_PAYMENTS                  AMT_PAYMENT                \
                                                     min           max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                              
100007     1940724    1                        23.129999  22655.654297   
           2462742    10                        3.240000   3598.290039   
                      12                        0.180000   3576.870117   
100008     1907290    10                      227.160004  11758.995117   
100011     1430602    9                      5614.334961   9000.000000   
...                                                  ...           ...   
456255     2073384    2                        81.495003  16319.115234   
           2631384    17                      737.909973  26751.779297   
                      18                      110.879997  27378.810547   
                      20                      447.255005  27042.435547   
                      22                      457.875000  27031.814453   

CLEAN_INSTALLMENTS_PAYMENTS                                
                                                      sum  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                
100007     1940724    1                      22678.785156  
           2462742    10                      3601.530029  
                      12                      3577.050049  
100008     1907290    10                     11986.155273  
100011     1430602    9                      14614.334961  
...                                                   ...  
456255     2073384    2                      16400.609375  
           2631384    17                     27489.689453  
                      18                     27

### Constance des montant et date d'échéance pour une version donnée

Par souci de rigueur, compte tenu de la surprise précédente, nous agrégeons la table de base par échéance et version, et vérifions comme précédemment, que le montant d'une échéance dans une version donnée est constante. Nous effectuons également cette vérification pour la date d'échéance (`DAYS_INSTALMENT`).

Cette vérification qui confirme nous donne une base rassurante et solide pour poursuivre.

La première étape d'agrégation est celle qui suit, et elle est est absolument indispensable. En faire l'impasse conduit à des agrégations inconsistantes.

In [31]:
version = data[[
    "NUM_INSTALMENT_VERSION",
    "AMT_INSTALMENT", "AMT_PAYMENT",
    "DAYS_INSTALMENT", "DAYS_ENTRY_PAYMENT"
]].reset_index()
version = version.sort_values(by=[
    "SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER", "NUM_INSTALMENT_VERSION"
])
display(version)

CLEAN_INSTALLMENTS_PAYMENTS,SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,AMT_INSTALMENT,AMT_PAYMENT,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT
0,100001,1369693,1,1,3951.000000,3951.000000,1709,1715
1,100001,1369693,2,1,3951.000000,3951.000000,1679,1715
2,100001,1369693,3,1,3951.000000,3951.000000,1649,1660
3,100001,1369693,4,2,17397.900391,17397.900391,1619,1628
4,100001,1851984,2,1,3982.050049,3982.050049,2916,2916
...,...,...,...,...,...,...,...,...
13602491,456255,2631384,23,3,27489.689453,27489.689453,96,98
13602492,456255,2631384,24,4,308277.312500,308277.312500,66,76
13602493,456255,2729207,1,1,11514.554688,11514.554688,469,482
13602494,456255,2729207,2,1,11514.554688,11514.554688,439,455


In [32]:
aggregated_by_version = version.groupby(by=[
    "SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER", "NUM_INSTALMENT_VERSION"
])

aggregated_by_version = aggregated_by_version.agg({
    "AMT_INSTALMENT" : ["min", "max", "count"],
    "AMT_PAYMENT": ["min", "max", "sum"],
    "DAYS_INSTALMENT": ["min", "max"],
    "DAYS_ENTRY_PAYMENT": ["min", "max"]
})

display(aggregated_by_version)

CLEAN_INSTALLMENTS_PAYMENTS                                        AMT_INSTALMENT  \
                                                                              min   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                  
100001     1369693    1                     1                         3951.000000   
                      2                     1                         3951.000000   
                      3                     1                         3951.000000   
                      4                     2                        17397.900391   
           1851984    2                     1                         3982.050049   
...                                                                           ...   
456255     2631384    23                    3                        27489.689453   
                      24                    4                       308277.312500   
           2729207    1                     1                        11514.554688   
                      2                     1                        11514.554688   
                      3                     2                        42754.230469   

CLEAN_INSTALLMENTS_PAYMENTS                                                        \
                                                                              max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                  
100001     1369693    1                     1                         3951.000000   
                      2                     1                         3951.000000   
                      3                     1                         3951.000000   
                      4                     2                        17397.900391   
           1851984    2                     1                         3982.050049   
...                                                                           ...   
456255     2631384    23                    3                        27489.689453   
                      24                    4                       308277.312500   
           2729207    1                     1                        11514.554688   
                      2                     1                        11514.554688   
                      3                     2                        42754.230469   

CLEAN_INSTALLMENTS_PAYMENTS                                               \
                                                                   count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION         
100001     1369693    1                     1                          1   
                      2                     1                          1   
                      3                     1                          1   
                      4                     2                          1   
           1851984    2                     1                          1   
...                                                                  ...   
456255     2631384    23                    3                          1   
                      24                    4                          1   
           2729207    1                     1                          1   
                      2                     1                          1   
                      3                     2                          1   

CLEAN_INSTALLMENTS_PAYMENTS                                           AMT_PAYMENT  \
                                                                              min   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                  
100001     1369693    1                     1                         3951.000000   
                      2                     1                         3951.000000   
                      3                     1                         3951.000000   
                      4                    

In [33]:
multi_by_version = aggregated_by_version[aggregated_by_version[("AMT_INSTALMENT", "count")] > 1]
display(multi_by_version)

CLEAN_INSTALLMENTS_PAYMENTS                                        AMT_INSTALMENT  \
                                                                              min   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                  
100007     1940724    1                     1                        22678.785156   
           2462742    10                    1                         3601.530029   
                      12                    1                         3577.050049   
100008     1907290    10                    1                        11986.155273   
100011     1430602    9                     1                        14614.334961   
...                                                                           ...   
456255     2073384    2                     1                        16400.609375   
           2631384    17                    3                        27489.689453   
                      18                    3                        27489.689453   
                      20                    3                        27489.689453   
                      22                    3                        27489.689453   

CLEAN_INSTALLMENTS_PAYMENTS                                                       \
                                                                             max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                 
100007     1940724    1                     1                       22678.785156   
           2462742    10                    1                        3601.530029   
                      12                    1                        3577.050049   
100008     1907290    10                    1                       11986.155273   
100011     1430602    9                     1                       14614.334961   
...                                                                          ...   
456255     2073384    2                     1                       16400.609375   
           2631384    17                    3                       27489.689453   
                      18                    3                       27489.689453   
                      20                    3                       27489.689453   
                      22                    3                       27489.689453   

CLEAN_INSTALLMENTS_PAYMENTS                                               \
                                                                   count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION         
100007     1940724    1                     1                          2   
           2462742    10                    1                          2   
                      12                    1                          2   
100008     1907290    10                    1                          2   
100011     1430602    9                     1                          2   
...                                                                  ...   
456255     2073384    2                     1                          2   
           2631384    17                    3                          2   
                      18                    3                          2   
                      20                    3                          2   
                      22                    3                          2   

CLEAN_INSTALLMENTS_PAYMENTS                                         AMT_PAYMENT  \
                                                                            min   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100007     1940724    1                     1                         23.129999   
           2462742    10                    1                          3.240000   
                      12                    1                          0.180000   
100008     1907290    10                    1                        

In [55]:
def insert_diff(data, group, pos):
    if (group, "diff") in data:
        data = data.drop(columns=(group, "diff"))
    group_max = data[(group, "max")]
    group_min = data[(group, "min")]
    group_diff = group_max - group_min
    data.insert(pos, (group, "diff"), group_diff)
    return data

data = insert_diff(multi_by_version, "AMT_INSTALMENT", 3)
data = insert_diff(multi_by_version, "DAYS_INSTALMENT", 9)
data = insert_diff(multi_by_version, "DAYS_ENTRY_PAYMENT", 12)

display(multi_by_version)

CLEAN_INSTALLMENTS_PAYMENTS                                        AMT_INSTALMENT  \
                                                                              min   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                  
100007     1940724    1                     1                        22678.785156   
           2462742    10                    1                         3601.530029   
                      12                    1                         3577.050049   
100008     1907290    10                    1                        11986.155273   
100011     1430602    9                     1                        14614.334961   
...                                                                           ...   
456255     2073384    2                     1                        16400.609375   
           2631384    17                    3                        27489.689453   
                      18                    3                        27489.689453   
                      20                    3                        27489.689453   
                      22                    3                        27489.689453   

CLEAN_INSTALLMENTS_PAYMENTS                                                       \
                                                                             max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                 
100007     1940724    1                     1                       22678.785156   
           2462742    10                    1                        3601.530029   
                      12                    1                        3577.050049   
100008     1907290    10                    1                       11986.155273   
100011     1430602    9                     1                       14614.334961   
...                                                                          ...   
456255     2073384    2                     1                       16400.609375   
           2631384    17                    3                       27489.689453   
                      18                    3                       27489.689453   
                      20                    3                       27489.689453   
                      22                    3                       27489.689453   

CLEAN_INSTALLMENTS_PAYMENTS                                                    \
                                                                   count diff   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION              
100007     1940724    1                     1                          2  0.0   
           2462742    10                    1                          2  0.0   
                      12                    1                          2  0.0   
100008     1907290    10                    1                          2  0.0   
100011     1430602    9                     1                          2  0.0   
...                                                                  ...  ...   
456255     2073384    2                     1                          2  0.0   
           2631384    17                    3                          2  0.0   
                      18                    3                          2  0.0   
                      20                    3                          2  0.0   
                      22                    3                          2  0.0   

CLEAN_INSTALLMENTS_PAYMENTS                                         AMT_PAYMENT  \
                                                                            min   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100007     1940724    1                     1                         23.129999   
           2462742    10                    1                          3.240000   
                      12                    1                          0.180000   

In [56]:
outliers_by_version = multi_by_version[multi_by_version[("AMT_INSTALMENT", "diff")] > 0]
n_multi_version = multi_by_version.shape[0]
n_outliers = outliers_by_version.shape[0]
print(f"% outliers: {100*n_outliers/n_multi_version:.2f}")
display(outliers_by_version)

% outliers: 0.00


Empty DataFrame
Columns: [(AMT_INSTALMENT, min), (AMT_INSTALMENT, max), (AMT_INSTALMENT, count), (AMT_INSTALMENT, diff), (AMT_PAYMENT, min), (AMT_PAYMENT, max), (AMT_PAYMENT, sum), (DAYS_INSTALMENT, min), (DAYS_INSTALMENT, max), (DAYS_INSTALMENT, diff), (DAYS_ENTRY_PAYMENT, min), (DAYS_ENTRY_PAYMENT, max), (DAYS_ENTRY_PAYMENT, diff)]
Index: []

In [57]:
outliers_by_version = multi_by_version[multi_by_version[("DAYS_INSTALMENT", "diff")] > 0]
n_multi_version = multi_by_version.shape[0]
n_outliers = outliers_by_version.shape[0]
print(f"% outliers: {100*n_outliers/n_multi_version:.2f}")
display(outliers_by_version)

% outliers: 0.00


Empty DataFrame
Columns: [(AMT_INSTALMENT, min), (AMT_INSTALMENT, max), (AMT_INSTALMENT, count), (AMT_INSTALMENT, diff), (AMT_PAYMENT, min), (AMT_PAYMENT, max), (AMT_PAYMENT, sum), (DAYS_INSTALMENT, min), (DAYS_INSTALMENT, max), (DAYS_INSTALMENT, diff), (DAYS_ENTRY_PAYMENT, min), (DAYS_ENTRY_PAYMENT, max), (DAYS_ENTRY_PAYMENT, diff)]
Index: []

In [60]:
vc_dict = dict(multi_by_version[("DAYS_ENTRY_PAYMENT", "diff")].value_counts())
print(vc_dict)
vc_dict_norm = dict(multi_by_version[("DAYS_ENTRY_PAYMENT", "diff")].value_counts(normalize=True))
print({k: f"{100*v:.3f} %" for k, v in vc_dict_norm.items()})

{32: 46544, 31: 42394, 33: 40315, 30: 35892, 34: 35241, 28: 34950, 29: 34109, 35: 33966, 36: 18700, 27: 17325, 37: 12284, 38: 11823, 26: 11520, 39: 10575, 25: 10541, 21: 10317, 24: 9716, 42: 9694, 40: 8983, 41: 8771, 14: 8369, 22: 8262, 23: 8131, 20: 7701, 7: 7034, 15: 6645, 13: 6434, 18: 6424, 17: 6347, 19: 6243, 16: 5948, 43: 5833, 12: 5737, 11: 5712, 8: 5618, 10: 5290, 6: 5208, 9: 5030, 5: 4409, 4: 4390, 44: 4308, 45: 4151, 3: 3987, 46: 3730, 49: 3505, 2: 3380, 1: 3375, 48: 3265, 47: 3159, 50: 2292, 56: 1955, 52: 1720, 51: 1623, 53: 1620, 55: 1563, 63: 1559, 62: 1474, 54: 1398, 57: 1304, 59: 1296, 60: 1279, 61: 1230, 58: 1145, 64: 844, 0: 667, 65: 582, 66: 559, 70: 525, 68: 481, 67: 480, 69: 456, 71: 359, 73: 277, 77: 270, 72: 269, 74: 242, 75: 235, 76: 224, 91: 183, 78: 182, 84: 168, 92: 158, 83: 153, 79: 153, 90: 150, 80: 147, 81: 146, 88: 145, 85: 140, 82: 131, 89: 118, 86: 115, 87: 107, 98: 104, 93: 104, 97: 99, 94: 93, 95: 89, 112: 88, 105: 86, 119: 72, 104: 72, 106: 71, 126: 7

### Agrégation par échéance et version

Pour reprendre les raisonnements qui précèdent, il nous faut une table agrégée où il n'y a plus qu'une ligne par version d'une échéance, avec :
- le nombre de règlements (de fraction de règlement de l'échéance)
- les montant et date constants de l'échéance,
- les première (max) et dernière (min) dates de règlement,
- la somme des règlements,

On complète avec l'outstanding qui devient à présent une donnée comparable au montant de l'échéance.

Par souci de complétude, nous avons envisagé de conserver en 2 colonnes additionnelles, les deux listes complètes des dates et des montants des règlement, mais le coût de ces deux agrégations s'est révélé prohibitif.

In [69]:
version = data[[
    "NUM_INSTALMENT_VERSION",
    "AMT_INSTALMENT", "AMT_PAYMENT",
    "DAYS_INSTALMENT", "DAYS_ENTRY_PAYMENT"
]].reset_index()
version = version.sort_values(by=[
    "SK_ID_CURR", "SK_ID_PREV",
    "NUM_INSTALMENT_NUMBER", "NUM_INSTALMENT_VERSION",
    "DAYS_ENTRY_PAYMENT"
])
display(version)

CLEAN_INSTALLMENTS_PAYMENTS,SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,AMT_INSTALMENT,AMT_PAYMENT,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT
0,100001,1369693,1,1,3951.000000,3951.000000,1709,1715
1,100001,1369693,2,1,3951.000000,3951.000000,1679,1715
2,100001,1369693,3,1,3951.000000,3951.000000,1649,1660
3,100001,1369693,4,2,17397.900391,17397.900391,1619,1628
4,100001,1851984,2,1,3982.050049,3982.050049,2916,2916
...,...,...,...,...,...,...,...,...
13602491,456255,2631384,23,3,27489.689453,27489.689453,96,98
13602492,456255,2631384,24,4,308277.312500,308277.312500,66,76
13602493,456255,2729207,1,1,11514.554688,11514.554688,469,482
13602494,456255,2729207,2,1,11514.554688,11514.554688,439,455


In [76]:
aggregated_by_version = version.groupby(by=[
    "SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER", "NUM_INSTALMENT_VERSION"
])

aggregated_by_version = aggregated_by_version.agg({
    "DAYS_INSTALMENT": "first",
    "AMT_INSTALMENT": "first",
    "DAYS_ENTRY_PAYMENT": ["count", "last", "first"],
    "AMT_PAYMENT": "sum"
})

aggregated_by_version.columns = [
    "DAYS_INSTALMENT", "AMT_INSTALMENT",
    "CNT_PAYMENT", "DAYS_ENTRY_PAYMENT_START", "DAYS_ENTRY_PAYMENT_END",
    "AMT_PAYMENT"
]

display(aggregated_by_version)

DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
100001     1369693    1                     1                                  1709   
                      2                     1                                  1679   
                      3                     1                                  1649   
                      4                     2                                  1619   
           1851984    2                     1                                  2916   
...                                                                             ...   
456255     2631384    23                    3                                    96   
                      24                    4                                    66   
           2729207    1                     1                                   469   
                      2                     1                                   439   
                      3                     2                                   409   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
100001     1369693    1                     1                          3951.000000   
                      2                     1                          3951.000000   
                      3                     1                          3951.000000   
                      4                     2                         17397.900391   
           1851984    2                     1                          3982.050049   
...                                                                            ...   
456255     2631384    23                    3                         27489.689453   
                      24                    4                        308277.312500   
           2729207    1                     1                         11514.554688   
                      2                     1                         11514.554688   
                      3                     2                         42754.230469   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100001     1369693    1                     1                                 1   
                      2                     1                                 1   
                      3                     1                                 1   
                      4                     2                                 1   
           1851984    2                     1                                 1   
...                                                                         ...   
456255     2631384    23                    3                                 1   
                      24                    4                                 1   
           2729207    1                     1                                 1   
                      2                     1                                 1   
                      3                     2                                 1   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
100001     1369693    1                     1                                           1715   
                      2                     1                                           1715   
                      3                     1                                           1660   
                      4                     2                                           1628   
           1851984    2                     1                                           2916   
...                                                              

In [ ]:
display(aggregated_by_version[
    aggregated_by_version.DAYS_ENTRY_PAYMENT_START
    != aggregated_by_version.DAYS_ENTRY_PAYMENT_END
])

DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
100007     1940724    1                     1                                  1076   
           2462742    10                    1                                  1466   
                      12                    1                                  1406   
100008     1907290    10                    1                                  2029   
100011     1430602    9                     1                                  2237   
...                                                                             ...   
456255     2073384    2                     1                                   588   
           2631384    17                    3                                   276   
                      18                    3                                   246   
                      20                    3                                   186   
                      22                    3                                   126   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
100007     1940724    1                     1                         22678.785156   
           2462742    10                    1                          3601.530029   
                      12                    1                          3577.050049   
100008     1907290    10                    1                         11986.155273   
100011     1430602    9                     1                         14614.334961   
...                                                                            ...   
456255     2073384    2                     1                         16400.609375   
           2631384    17                    3                         27489.689453   
                      18                    3                         27489.689453   
                      20                    3                         27489.689453   
                      22                    3                         27489.689453   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100007     1940724    1                     1                                 2   
           2462742    10                    1                                 2   
                      12                    1                                 2   
100008     1907290    10                    1                                 2   
100011     1430602    9                     1                                 2   
...                                                                         ...   
456255     2073384    2                     1                                 2   
           2631384    17                    3                                 2   
                      18                    3                                 2   
                      20                    3                                 2   
                      22                    3                                 2   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
100007     1940724    1                     1                                           1106   
           2462742    10                    1                                           1497   
                      12                    1                                           1436   
100008     1907290    10                    1                                           2056   
100011     1430602    9                     1                                           2253   
...                                                              

### Agrégation par échéance (V2)

Les cas qui demandent une vérification sont ceux où les échéances ont des versions multiples.

Pour les isoler, produisons une table agrégée des échéances où l'on détermine :
- le _nombre_ de versions de l'échéance (et accessoirement les ordinaux _min_ et _max_ de versions)
- les montants _min_ et _max_ des échéances et de leurs règlements (pour nous assurer de leur constance)
- la _somme_ des règlements (pour nous assurer qu'elle coïncide avec le montant de l'échéance)

Cette table a 12 859 104 entrées : la majorité des échéances n'ont donc qu'une versions.

In [79]:
version = aggregated_by_version.reset_index()

aggregated_version = version.groupby(by=[
    "SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER"
])

aggregated_version = aggregated_version.agg({
    "NUM_INSTALMENT_VERSION" : ["min", "max", "count"],
    "AMT_INSTALMENT" : ["min", "max"],
    "AMT_PAYMENT": ["min", "max", "sum"]
})

display(aggregated_version)

NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100001     1369693    1                                          1   1     1   
                      2                                          1   1     1   
                      3                                          1   1     1   
                      4                                          2   2     1   
           1851984    2                                          1   1     1   
...                                                            ...  ..   ...   
456255     2631384    23                                         3   3     1   
                      24                                         4   4     1   
           2729207    1                                          1   1     1   
                      2                                          1   1     1   
                      3                                          2   2     1   

                                            AMT_INSTALMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100001     1369693    1                        3951.000000    3951.000000   
                      2                        3951.000000    3951.000000   
                      3                        3951.000000    3951.000000   
                      4                       17397.900391   17397.900391   
           1851984    2                        3982.050049    3982.050049   
...                                                    ...            ...   
456255     2631384    23                      27489.689453   27489.689453   
                      24                     308277.312500  308277.312500   
           2729207    1                       11514.554688   11514.554688   
                      2                       11514.554688   11514.554688   
                      3                       42754.230469   42754.230469   

                                               AMT_PAYMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100001     1369693    1                        3951.000000    3951.000000   
                      2                        3951.000000    3951.000000   
                      3                        3951.000000    3951.000000   
                      4                       17397.900391   17397.900391   
           1851984    2                        3982.050049    3982.050049   
...                                                    ...            ...   
456255     2631384    23                      27489.689453   27489.689453   
                      24                     308277.312500  308277.312500   
           2729207    1                       11514.554688   11514.554688   
                      2                       11514.554688   11514.554688   
                      3                       42754.230469   42754.230469   

                                                            
                                                       sum  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                 
100001     1369693    1                        3951.000000  
                      2                        3951.000000  
                      3                        3951.000000  
                      4                       17397.900391  
           1851984    2                        3982.050049  
...                                                    ...  
456255     2631384    23                      27489.689453  
                      24                     308277.312500  
           2729207    1                       11514.554688  
                      2          

### Échéances à versions multiples (V2)

Nous pouvons reprendre nos vérifications effectuées plus haut sur une base assainie.

Les chiffres changent sensiblement et se simplifient (par exemple, 3 versions maximum) :
- 99.3 % des échéances n'ont qu'une seule version,
- 89 882 cas (sur 12 859 104) sont multi-versions,
- 0.7 % en ont deux,
- les cas à 3 versions sont marginaux (27 cas).

In [80]:
vc_dict = dict(aggregated_version[("NUM_INSTALMENT_VERSION", "count")].value_counts())
print(vc_dict)
vc_dict_norm = dict(aggregated_version[("NUM_INSTALMENT_VERSION", "count")].value_counts(normalize=True))
print({k: f"{100*v:.3f} %" for k, v in vc_dict_norm.items()})

multi_version = aggregated_version[aggregated_version[("NUM_INSTALMENT_VERSION", "count")] > 1]
display(multi_version)

{1: 12769222, 2: 89855, 3: 27}
{1: '99.301 %', 2: '0.699 %', 3: '0.000 %'}


NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100012     2480304    1                                          1   2     2   
100019     1620327    6                                          1   2     2   
100034     1390369    1                                          1   2     2   
100039     1077565    1                                          1   2     2   
                      2                                          3   4     2   
...                                                            ...  ..   ...   
456202     1333058    1                                          1   2     2   
456207     2230233    1                                          1   2     2   
456227     1772808    7                                          1   2     2   
456240     1508947    7                                          1   2     2   
456255     2631384    1                                          1   2     2   

                                            AMT_INSTALMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100012     2480304    1                        9482.355469   49204.890625   
100019     1620327    6                       10518.615234   33850.664062   
100034     1390369    1                       22430.429688   36069.570312   
100039     1077565    1                       52513.515625  172486.484375   
                      2                       46326.734375  178673.265625   
...                                                    ...            ...   
456202     1333058    1                        7147.890137   10852.110352   
456207     2230233    1                        4403.250000   36096.750000   
456227     1772808    7                       11813.580078   38804.941406   
456240     1508947    7                       16721.910156   18542.429688   
456255     2631384    1                       54022.140625  615229.500000   

                                               AMT_PAYMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100012     2480304    1                       58687.246094   58687.246094   
100019     1620327    6                       44369.281250   44369.281250   
100034     1390369    1                       58500.000000   58500.000000   
100039     1077565    1                      225000.000000  225000.000000   
                      2                      225000.000000  225000.000000   
...                                                    ...            ...   
456202     1333058    1                       18000.000000   18000.000000   
456207     2230233    1                       40500.000000   40500.000000   
456227     1772808    7                       50618.519531   50618.519531   
456240     1508947    7                       35264.339844   35264.339844   
456255     2631384    1                      669251.625000  669251.625000   

                                                           
                                                      sum  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                
100012     2480304    1                      1.173745e+05  
100019     1620327    6                      8.873856e+04  
100034     1390369    1                      1.170000e+05  
100039     1077565    1                      4.500000e+05  
                      2                      4.500000e+05  
...                                                   ...  
456202     1333058    1                      3.600000e+04  
456207     2230233    1                      8.100000e+04  
456227     1772808    7                      1.012370e+05  
456240     1508947    7                      

### Constance du règlement

Contrairement à notre hypothèse initiale, ce qui semble ressortir ici, c'est qu'en cas de version multiples, c'est le montant total réglé qui est constant. Cela signifie notamment que de ce point de vue, les lignes qui comptabilisent les règlements sont des doublons qu'il faut impérativement éviter de compter plusieurs fois dans nos agrégations.

Vérifions cette hypothèse de constance.

Remplaçons la colonnes `sum` qui devient inutile, par un diff des _min_ et _max_ de règlement :

In [81]:
aggregated_version = aggregated_version.drop(columns=("AMT_PAYMENT", "sum"))

pyt_max = aggregated_version[("AMT_PAYMENT", "max")]
pyt_min = aggregated_version[("AMT_PAYMENT", "min")]
aggregated_version[("AMT_PAYMENT", "diff")] = pyt_max - pyt_min

Dans la majorité des cas, notre hypothèse est vérifiée.

Nous trouvons cependant une marge de 299 outliers (dont 24 à 3 versions).

Cela concerne toujours la première échéance (vérification à deux cellules d'ici).

L'observation de cas semble indiquer qu'il s'agit d'hypothèse initiales de montant de prêt qui peuvent être ajustées une ou plusieurs fois avant d'établir l'échéancier.

Il y a peu de cas et même si ce n'est qu'une hypothèse raisonnable, nous la posons comme base de corrections des outliers. Cette correction consiste, dans ces 299 cas, à ne conserver que la dernière version de cette échéance (donc d'éliminer les précédentes qui introduisent du bruit et du déséquilibre comptable) 

In [106]:
outliers = aggregated_version[aggregated_version[("AMT_PAYMENT", "diff")] > 0]
print("3 versions cases :", outliers[outliers[("NUM_INSTALMENT_VERSION", "count")] > 2].shape[0])
display(outliers)

3 versions cases : 24


NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
101080     2290883    1                                          2   3     2   
101378     1133646    1                                          2   3     2   
101826     2010823    1                                          2   3     2   
102206     1096061    1                                          2   3     2   
102731     1175528    1                                          2   3     2   
...                                                            ...  ..   ...   
443189     1277527    1                                          2   3     2   
445335     2207258    1                                          2   3     2   
449589     1265758    1                                          2   3     2   
450065     1932681    1                                          2   3     2   
452983     1087181    1                                          2   5     3   

                                            AMT_INSTALMENT           \
                                                       min      max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                           
101080     2290883    1                       11298.870117  22500.0   
101378     1133646    1                        9143.865234  31500.0   
101826     2010823    1                        6092.459961  45000.0   
102206     1096061    1                        5368.770020  12600.0   
102731     1175528    1                       16128.764648  45000.0   
...                                                    ...      ...   
443189     1277527    1                        6620.714844  22500.0   
445335     2207258    1                       14576.849609  45000.0   
449589     1265758    1                        3932.729980  18000.0   
450065     1932681    1                         525.825012  32647.5   
452983     1087181    1                        7105.185059  22500.0   

                                              AMT_PAYMENT           \
                                                      min      max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
101080     2290883    1                      11298.870117  22500.0   
101378     1133646    1                       9143.865234  31500.0   
101826     2010823    1                       6092.459961  45000.0   
102206     1096061    1                       5368.770020  12600.0   
102731     1175528    1                      16128.764648  45000.0   
...                                                   ...      ...   
443189     1277527    1                       6620.715332  22500.0   
445335     2207258    1                      14576.849609  45000.0   
449589     1265758    1                       3932.729980  18000.0   
450065     1932681    1                        525.825012  32647.5   
452983     1087181    1                       7105.185059  22500.0   

                                                           
                                                     diff  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                
101080     2290883    1                      11201.129883  
101378     1133646    1                      22356.134766  
101826     2010823    1                      38907.539062  
102206     1096061    1                       7231.229980  
102731     1175528    1                      28871.234375  
...                                                   ...  
443189     1277527    1                      15879.285156  
445335     2207258    1                      30423.150391  
449589     1265758    1                      14067.269531  
450065     1932681    1                      32121.675781  
452983     1087181    1                      15394.814453  

[299 rows x 8 columns]

In [114]:
outliers_index = outliers.index
num_inst = outliers_index.get_level_values(2)
display(all(num_inst == 1))

True

Observons quelque uns de ces cas à la loupe :

In [96]:
def show_case_raw(cid, pid):
    display(data.loc[(cid, pid)].sort_values(by="NUM_INSTALMENT_VERSION"))

def show_case_agg(cid, pid):
    display(aggregated_by_version.loc[(cid, pid)].sort_values(by="NUM_INSTALMENT_VERSION"))

In [97]:
show_case_raw(101080, 2290883)
show_case_agg(101080, 2290883)

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,1,0,2,14,14,22500.000000,22500.000000
1,1,0,3,9,8,11298.870117,11298.870117


DAYS_INSTALMENT  AMT_INSTALMENT  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                                    
1                     2                                    14    22500.000000   
                      3                                     9    11298.870117   

                                              CNT_PAYMENT  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
1                     2                                 1   
                      3                                 1   

                                              DAYS_ENTRY_PAYMENT_START  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
1                     2                                             14   
                      3                                              8   

                                              DAYS_ENTRY_PAYMENT_END  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                           
1                     2                                           14   
                      3                                            8   

                                               AMT_PAYMENT  
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
1                     2                       22500.000000  
                      3                       11298.870117

In [98]:
show_case_raw(101378, 1133646)
show_case_agg(101378, 1133646)

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,5,0,2,132,132,31500.000000,31500.000000
1,5,0,3,126,125,9143.865234,9143.865234
2,4,0,4,96,101,42552.808594,42552.808594


DAYS_INSTALMENT  AMT_INSTALMENT  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                                    
1                     2                                   132    31500.000000   
                      3                                   126     9143.865234   
2                     4                                    96    42552.808594   

                                              CNT_PAYMENT  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
1                     2                                 1   
                      3                                 1   
2                     4                                 1   

                                              DAYS_ENTRY_PAYMENT_START  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
1                     2                                            132   
                      3                                            125   
2                     4                                            101   

                                              DAYS_ENTRY_PAYMENT_END  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                           
1                     2                                          132   
                      3                                          125   
2                     4                                          101   

                                               AMT_PAYMENT  
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
1                     2                       31500.000000  
                      3                        9143.865234  
2                     4                       42552.808594

In [102]:
show_case_raw(118231, 1509640)
show_case_agg(118231, 1509640)

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,3,0,2,66,68,31500.000000,31500.000000
1,2,0,3,38,40,17339.894531,20252.115234
1,2,0,4,38,40,2912.219971,20252.115234
2,1,0,5,8,7,16291.259766,16291.259766


DAYS_INSTALMENT  AMT_INSTALMENT  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                                    
1                     2                                    66    31500.000000   
                      3                                    38    17339.894531   
                      4                                    38     2912.219971   
2                     5                                     8    16291.259766   

                                              CNT_PAYMENT  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
1                     2                                 1   
                      3                                 1   
                      4                                 1   
2                     5                                 1   

                                              DAYS_ENTRY_PAYMENT_START  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
1                     2                                             68   
                      3                                             40   
                      4                                             40   
2                     5                                              7   

                                              DAYS_ENTRY_PAYMENT_END  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                           
1                     2                                           68   
                      3                                           40   
                      4                                           40   
2                     5                                            7   

                                               AMT_PAYMENT  
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
1                     2                       31500.000000  
                      3                       20252.115234  
                      4                       20252.115234  
2                     5                       16291.259766

In [103]:
show_case_raw(327954, 1385739)
show_case_agg(327954, 1385739)

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,7,-1,2,209,209,450000.000000,450000.000000
1,7,-1,4,207,209,147303.000000,147303.000000
1,7,-1,5,205,201,83342.382812,83342.382812
2,6,-1,5,175,194,83342.382812,83342.382812
3,5,-1,5,145,149,83342.382812,83342.382812
4,4,-1,5,115,128,83342.382812,83342.382812
5,3,-1,5,85,97,83342.382812,83342.382812
6,2,-1,5,55,68,83342.382812,83342.382812
7,1,-1,5,25,29,83342.382812,83342.382812


DAYS_INSTALMENT  AMT_INSTALMENT  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                                    
1                     2                                   209   450000.000000   
                      4                                   207   147303.000000   
                      5                                   205    83342.382812   
2                     5                                   175    83342.382812   
3                     5                                   145    83342.382812   
4                     5                                   115    83342.382812   
5                     5                                    85    83342.382812   
6                     5                                    55    83342.382812   
7                     5                                    25    83342.382812   

                                              CNT_PAYMENT  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
1                     2                                 1   
                      4                                 1   
                      5                                 1   
2                     5                                 1   
3                     5                                 1   
4                     5                                 1   
5                     5                                 1   
6                     5                                 1   
7                     5                                 1   

                                              DAYS_ENTRY_PAYMENT_START  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
1                     2                                            209   
                      4                                            209   
                      5                                            201   
2                     5                                            194   
3                     5                                            149   
4                     5                                            128   
5                     5                                             97   
6                     5                                             68   
7                     5                                             29   

                                              DAYS_ENTRY_PAYMENT_END  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                           
1                     2                                          209   
                      4                                          209   
                      5                                          201   
2                     5                                          194   
3                     5                                          149   
4                     5                                          128   
5                     5                                           97   
6                     5                                           68   
7                     5                                           29   

                                                AMT_PAYMENT  
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                 
1                     2                       450000.000000  
                      4                       147303.000000  
                      5                        83342.382812  
2                     5                        83342.382812  
3                     5                        83342.382812  
4                     5                        83342.382812  
5                     5                        83342.382812  
6                     5                        83342.382812  
7                     5                        83342.382812

In [105]:
show_case_raw(275232, 2540309)
show_case_agg(275232, 2540309)

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,8,-1,2,215,215,90000.000000,90000.000000
1,7,-1,3,200,214,41233.679688,135000.000000
1,7,-1,4,200,214,93766.320312,135000.000000
2,6,-1,5,170,190,32238.990234,45000.000000
2,6,-1,6,170,190,12761.009766,45000.000000
3,5,-1,7,140,149,30915.404297,45000.000000
3,5,-1,8,140,149,14084.594727,45000.000000
4,4,-1,9,110,115,29304.585938,29304.585938
5,3,-1,9,80,92,29304.585938,29304.585938


DAYS_INSTALMENT  AMT_INSTALMENT  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                                    
1                     2                                   215    90000.000000   
                      3                                   200    41233.679688   
                      4                                   200    93766.320312   
2                     5                                   170    32238.990234   
                      6                                   170    12761.009766   
3                     7                                   140    30915.404297   
                      8                                   140    14084.594727   
4                     9                                   110    29304.585938   
5                     9                                    80    29304.585938   
6                     9                                    50    29304.585938   
7                     9                                    20    29304.585938   

                                              CNT_PAYMENT  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
1                     2                                 1   
                      3                                 1   
                      4                                 1   
2                     5                                 1   
                      6                                 1   
3                     7                                 1   
                      8                                 1   
4                     9                                 1   
5                     9                                 1   
6                     9                                 1   
7                     9                                 1   

                                              DAYS_ENTRY_PAYMENT_START  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
1                     2                                            215   
                      3                                            214   
                      4                                            214   
2                     5                                            190   
                      6                                            190   
3                     7                                            149   
                      8                                            149   
4                     9                                            115   
5                     9                                             92   
6                     9                                             60   
7                     9                                             25   

                                              DAYS_ENTRY_PAYMENT_END  \
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                           
1                     2                                          215   
                      3                                          214   
                      4                                          214   
2                     5                                          190   
                      6                                          190   
3                     7                                          149   
                      8                                          149   
4                     9                                          115   
5                     9                                           92   
6                     9                                           60   
7                     9                                           25   

                                                AMT_PAYMENT  
NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                 
1                     2                        90000.000000  
                      3                       135000.000000  
                      4                       1350

### Traitement des outliers

Cf. nos constatations, nous éliminons les outliers, et vérifions que notre relation de constance du règlement est alors bien vérifiée partout et constitue donc une propriété forte.

L'opération n'est pas triviale.

1. Récupérer l'index des outliers (cid, pid, n_inst).
2. S'en servir pour sélectionner les lignes de aggregated_by_version concernées par le drop
3. Construire l'index des lignes à supprimer
4. supprimer

In [173]:
display(aggregated_by_version)

DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
100001     1369693    1                     1                                  1709   
                      2                     1                                  1679   
                      3                     1                                  1649   
                      4                     2                                  1619   
           1851984    2                     1                                  2916   
...                                                                             ...   
456255     2631384    23                    3                                    96   
                      24                    4                                    66   
           2729207    1                     1                                   469   
                      2                     1                                   439   
                      3                     2                                   409   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
100001     1369693    1                     1                          3951.000000   
                      2                     1                          3951.000000   
                      3                     1                          3951.000000   
                      4                     2                         17397.900391   
           1851984    2                     1                          3982.050049   
...                                                                            ...   
456255     2631384    23                    3                         27489.689453   
                      24                    4                        308277.312500   
           2729207    1                     1                         11514.554688   
                      2                     1                         11514.554688   
                      3                     2                         42754.230469   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100001     1369693    1                     1                                 1   
                      2                     1                                 1   
                      3                     1                                 1   
                      4                     2                                 1   
           1851984    2                     1                                 1   
...                                                                         ...   
456255     2631384    23                    3                                 1   
                      24                    4                                 1   
           2729207    1                     1                                 1   
                      2                     1                                 1   
                      3                     2                                 1   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
100001     1369693    1                     1                                           1715   
                      2                     1                                           1715   
                      3                     1                                           1660   
                      4                     2                                           1628   
           1851984    2                     1                                           2916   
...                                                              

In [174]:
drop_index = aggregated_by_version.reset_index(level=3)
drop_index = drop_index[["NUM_INSTALMENT_VERSION"]]
display(drop_index)

NUM_INSTALMENT_VERSION
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                        
100001     1369693    1                                           1
                      2                                           1
                      3                                           1
                      4                                           2
           1851984    2                                           1
...                                                             ...
456255     2631384    23                                          3
                      24                                          4
           2729207    1                                           1
                      2                                           1
                      3                                           2

[12949013 rows x 1 columns]

In [176]:
drop_index = drop_index.loc[outliers.index]
display(drop_index)

NUM_INSTALMENT_VERSION
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                        
101080     2290883    1                                           2
                      1                                           3
101378     1133646    1                                           2
                      1                                           3
101826     2010823    1                                           2
...                                                             ...
450065     1932681    1                                           2
                      1                                           3
452983     1087181    1                                           2
                      1                                           4
                      1                                           5

[622 rows x 1 columns]

In [177]:
drop_index = drop_index.reset_index()
display(drop_index)

,SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION
0,101080,2290883,1,2
1,101080,2290883,1,3
2,101378,1133646,1,2
3,101378,1133646,1,3
4,101826,2010823,1,2
...,...,...,...,...
617,450065,1932681,1,2
618,450065,1932681,1,3
619,452983,1087181,1,2
620,452983,1087181,1,4


In [178]:
keep_index = drop_index.drop_duplicates(subset=drop_index.columns[:-1], keep="last")
display(keep_index)

,SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION
1,101080,2290883,1,3
3,101378,1133646,1,3
5,101826,2010823,1,3
7,102206,1096061,1,3
9,102731,1175528,1,3
...,...,...,...,...
612,443189,1277527,1,3
614,445335,2207258,1,3
616,449589,1265758,1,3
618,450065,1932681,1,3


In [181]:
drop_index = drop_index[~drop_index.index.isin(keep_index.index)]
display(drop_index)

,SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION
0,101080,2290883,1,2
2,101378,1133646,1,2
4,101826,2010823,1,2
6,102206,1096061,1,2
8,102731,1175528,1,2
...,...,...,...,...
613,445335,2207258,1,2
615,449589,1265758,1,2
617,450065,1932681,1,2
619,452983,1087181,1,2


In [182]:
import pandas as pd

drop_index = pd.MultiIndex.from_frame(drop_index)
display(drop_index)

MultiIndex([(101080, 2290883, 1, 2),
            (101378, 1133646, 1, 2),
            (101826, 2010823, 1, 2),
            (102206, 1096061, 1, 2),
            (102731, 1175528, 1, 2),
            (103092, 2663232, 1, 2),
            (103486, 1074494, 1, 2),
            (106176, 1854084, 1, 2),
            (107529, 1732595, 1, 2),
            (108813, 2673077, 1, 2),
            ...
            (441388, 1061547, 1, 2),
            (441931, 1413906, 1, 2),
            (442174, 2085596, 1, 2),
            (442388, 1110271, 1, 2),
            (443189, 1277527, 1, 2),
            (445335, 2207258, 1, 2),
            (449589, 1265758, 1, 2),
            (450065, 1932681, 1, 2),
            (452983, 1087181, 1, 2),
            (452983, 1087181, 1, 4)],
           names=['SK_ID_CURR', 'SK_ID_PREV', 'NUM_INSTALMENT_NUMBER', 'NUM_INSTALMENT_VERSION'], length=323)

In [183]:
display(aggregated_by_version.loc[drop_index])

,,,,DAYS_INSTALMENT,AMT_INSTALMENT,CNT_PAYMENT,DAYS_ENTRY_PAYMENT_START,DAYS_ENTRY_PAYMENT_END,AMT_PAYMENT
SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,,,,,,
101080,2290883,1,2,14,22500.0,1,14,14,22500.0
101378,1133646,1,2,132,31500.0,1,132,132,31500.0
101826,2010823,1,2,209,45000.0,1,209,209,45000.0
102206,1096061,1,2,168,12600.0,1,175,175,12600.0
102731,1175528,1,2,208,45000.0,1,208,208,45000.0
...,...,...,...,...,...,...,...,...,...
445335,2207258,1,2,91,45000.0,1,91,91,45000.0
449589,1265758,1,2,75,18000.0,1,75,75,18000.0
450065,1932681,1,2,135,32647.5,1,135,135,32647.5


In [194]:
# plus rapide que la solution suivante de difference d'index
# plus rapide également à l'utilisation, qu'avec un loc
mask = ~aggregated_by_version.index.isin(drop_index)

In [195]:
# à éviter en prod, trop lent
new_aggregated_by_version = aggregated_by_version[mask]
display(new_aggregated_by_version)

DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
100001     1369693    1                     1                                  1709   
                      2                     1                                  1679   
                      3                     1                                  1649   
                      4                     2                                  1619   
           1851984    2                     1                                  2916   
...                                                                             ...   
456255     2631384    23                    3                                    96   
                      24                    4                                    66   
           2729207    1                     1                                   469   
                      2                     1                                   439   
                      3                     2                                   409   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
100001     1369693    1                     1                          3951.000000   
                      2                     1                          3951.000000   
                      3                     1                          3951.000000   
                      4                     2                         17397.900391   
           1851984    2                     1                          3982.050049   
...                                                                            ...   
456255     2631384    23                    3                         27489.689453   
                      24                    4                        308277.312500   
           2729207    1                     1                         11514.554688   
                      2                     1                         11514.554688   
                      3                     2                         42754.230469   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100001     1369693    1                     1                                 1   
                      2                     1                                 1   
                      3                     1                                 1   
                      4                     2                                 1   
           1851984    2                     1                                 1   
...                                                                         ...   
456255     2631384    23                    3                                 1   
                      24                    4                                 1   
           2729207    1                     1                                 1   
                      2                     1                                 1   
                      3                     2                                 1   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
100001     1369693    1                     1                                           1715   
                      2                     1                                           1715   
                      3                     1                                           1660   
                      4                     2                                           1628   
           1851984    2                     1                                           2916   
...                                                              

Tout en un pour intégration :

In [196]:
import pandas as pd

drop_index = aggregated_by_version.reset_index(level=3)
drop_index = drop_index[["NUM_INSTALMENT_VERSION"]]
drop_index = drop_index.loc[outliers.index]
drop_index = drop_index.reset_index()
keep_index = drop_index.drop_duplicates(subset=drop_index.columns[:-1], keep="last")
drop_index = drop_index[~drop_index.index.isin(keep_index.index)]
drop_index = pd.MultiIndex.from_frame(drop_index)
mask = ~aggregated_by_version.index.isin(drop_index)
new_aggregated_by_version = aggregated_by_version[mask]

display(new_aggregated_by_version)

DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
100001     1369693    1                     1                                  1709   
                      2                     1                                  1679   
                      3                     1                                  1649   
                      4                     2                                  1619   
           1851984    2                     1                                  2916   
...                                                                             ...   
456255     2631384    23                    3                                    96   
                      24                    4                                    66   
           2729207    1                     1                                   469   
                      2                     1                                   439   
                      3                     2                                   409   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
100001     1369693    1                     1                          3951.000000   
                      2                     1                          3951.000000   
                      3                     1                          3951.000000   
                      4                     2                         17397.900391   
           1851984    2                     1                          3982.050049   
...                                                                            ...   
456255     2631384    23                    3                         27489.689453   
                      24                    4                        308277.312500   
           2729207    1                     1                         11514.554688   
                      2                     1                         11514.554688   
                      3                     2                         42754.230469   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100001     1369693    1                     1                                 1   
                      2                     1                                 1   
                      3                     1                                 1   
                      4                     2                                 1   
           1851984    2                     1                                 1   
...                                                                         ...   
456255     2631384    23                    3                                 1   
                      24                    4                                 1   
           2729207    1                     1                                 1   
                      2                     1                                 1   
                      3                     2                                 1   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
100001     1369693    1                     1                                           1715   
                      2                     1                                           1715   
                      3                     1                                           1660   
                      4                     2                                           1628   
           1851984    2                     1                                           2916   
...                                                              

In [197]:
display(aggregated_by_version[~mask])

,,,,DAYS_INSTALMENT,AMT_INSTALMENT,CNT_PAYMENT,DAYS_ENTRY_PAYMENT_START,DAYS_ENTRY_PAYMENT_END,AMT_PAYMENT
SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,,,,,,
101080,2290883,1,2,14,22500.0,1,14,14,22500.0
101378,1133646,1,2,132,31500.0,1,132,132,31500.0
101826,2010823,1,2,209,45000.0,1,209,209,45000.0
102206,1096061,1,2,168,12600.0,1,175,175,12600.0
102731,1175528,1,2,208,45000.0,1,208,208,45000.0
...,...,...,...,...,...,...,...,...,...
445335,2207258,1,2,91,45000.0,1,91,91,45000.0
449589,1265758,1,2,75,18000.0,1,75,75,18000.0
450065,1932681,1,2,135,32647.5,1,135,135,32647.5


Pour nous assurer que tout est solide, revérifions la relation de constance du règlement.

On commence par l'agrégation. Ici, on ne conserve pas la somme de `AMT_PAYMENT` que l'on sait à présent inutile.

Le nombre d'entrée ne change pas, ce qui est plutôt rassurant (on s'est contenté de supprimer quelques versions erronées d'échéances).

In [198]:
new_version = new_aggregated_by_version.reset_index()

new_aggregated_version = new_version.groupby(by=[
    "SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER"
])

new_aggregated_version = new_aggregated_version.agg({
    "NUM_INSTALMENT_VERSION" : ["min", "max", "count"],
    "AMT_INSTALMENT" : ["min", "max"],
    "AMT_PAYMENT": ["min", "max"]
})

display(new_aggregated_version)

NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100001     1369693    1                                          1   1     1   
                      2                                          1   1     1   
                      3                                          1   1     1   
                      4                                          2   2     1   
           1851984    2                                          1   1     1   
...                                                            ...  ..   ...   
456255     2631384    23                                         3   3     1   
                      24                                         4   4     1   
           2729207    1                                          1   1     1   
                      2                                          1   1     1   
                      3                                          2   2     1   

                                            AMT_INSTALMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100001     1369693    1                        3951.000000    3951.000000   
                      2                        3951.000000    3951.000000   
                      3                        3951.000000    3951.000000   
                      4                       17397.900391   17397.900391   
           1851984    2                        3982.050049    3982.050049   
...                                                    ...            ...   
456255     2631384    23                      27489.689453   27489.689453   
                      24                     308277.312500  308277.312500   
           2729207    1                       11514.554688   11514.554688   
                      2                       11514.554688   11514.554688   
                      3                       42754.230469   42754.230469   

                                               AMT_PAYMENT                 
                                                       min            max  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                
100001     1369693    1                        3951.000000    3951.000000  
                      2                        3951.000000    3951.000000  
                      3                        3951.000000    3951.000000  
                      4                       17397.900391   17397.900391  
           1851984    2                        3982.050049    3982.050049  
...                                                    ...            ...  
456255     2631384    23                      27489.689453   27489.689453  
                      24                     308277.312500  308277.312500  
           2729207    1                       11514.554688   11514.554688  
                      2                       11514.554688   11514.554688  
                      3                       42754.230469   42754.230469  

[12859104 rows x 7 columns]

In [202]:
pyt_max = new_aggregated_version[("AMT_PAYMENT", "max")]
pyt_min = new_aggregated_version[("AMT_PAYMENT", "min")]
new_aggregated_version[("AMT_PAYMENT", "diff")] = pyt_max - pyt_min
new_outliers = new_aggregated_version[new_aggregated_version[("AMT_PAYMENT", "diff")] > 0]
display(new_outliers)

Empty DataFrame
Columns: [(NUM_INSTALMENT_VERSION, min), (NUM_INSTALMENT_VERSION, max), (NUM_INSTALMENT_VERSION, count), (AMT_INSTALMENT, min), (AMT_INSTALMENT, max), (AMT_PAYMENT, min), (AMT_PAYMENT, max), (AMT_PAYMENT, diff)]
Index: []

## Abstraction des versions par échéance

Il s'agit de réaliser sur l'ensemble des données, des versions multiples d'une échéance en ne conservant que la dernière version.

Nous avons établi plus haut que 6.5 % des échéances sont concernées.

C'est donc une généralisation de la réduction opérée plus haut sur les outliers.

Récupérons d'abord la table de base :

In [1]:
from home_credit.tables import InstallmentsPayments

base_data = InstallmentsPayments.clean_by_installment_and_version()
display(base_data)

DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
100001     1369693    1                     1                                  1709   
                      2                     1                                  1679   
                      3                     1                                  1649   
                      4                     2                                  1619   
           1851984    2                     1                                  2916   
...                                                                             ...   
456255     2631384    23                    3                                    96   
                      24                    4                                    66   
           2729207    1                     1                                   469   
                      2                     1                                   439   
                      3                     2                                   409   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
100001     1369693    1                     1                          3951.000000   
                      2                     1                          3951.000000   
                      3                     1                          3951.000000   
                      4                     2                         17397.900391   
           1851984    2                     1                          3982.050049   
...                                                                            ...   
456255     2631384    23                    3                         27489.689453   
                      24                    4                        308277.312500   
           2729207    1                     1                         11514.554688   
                      2                     1                         11514.554688   
                      3                     2                         42754.230469   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100001     1369693    1                     1                                 1   
                      2                     1                                 1   
                      3                     1                                 1   
                      4                     2                                 1   
           1851984    2                     1                                 1   
...                                                                         ...   
456255     2631384    23                    3                                 1   
                      24                    4                                 1   
           2729207    1                     1                                 1   
                      2                     1                                 1   
                      3                     2                                 1   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
100001     1369693    1                     1                                           1715   
                      2                     1                                           1715   
                      3                     1                                           1660   
                      4                     2                                           1628   
           1851984    2                     1                                           2916   
...                                                              

Agrégeons par rapport à la version :

In [3]:
from home_credit.groupby import get_installments_payments_by_version

aggregated = get_installments_payments_by_version(base_data)
display(aggregated)

NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100001     1369693    1                                          1   1     1   
                      2                                          1   1     1   
                      3                                          1   1     1   
                      4                                          2   2     1   
           1851984    2                                          1   1     1   
...                                                            ...  ..   ...   
456255     2631384    23                                         3   3     1   
                      24                                         4   4     1   
           2729207    1                                          1   1     1   
                      2                                          1   1     1   
                      3                                          2   2     1   

                                            AMT_INSTALMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100001     1369693    1                        3951.000000    3951.000000   
                      2                        3951.000000    3951.000000   
                      3                        3951.000000    3951.000000   
                      4                       17397.900391   17397.900391   
           1851984    2                        3982.050049    3982.050049   
...                                                    ...            ...   
456255     2631384    23                      27489.689453   27489.689453   
                      24                     308277.312500  308277.312500   
           2729207    1                       11514.554688   11514.554688   
                      2                       11514.554688   11514.554688   
                      3                       42754.230469   42754.230469   

                                               AMT_PAYMENT                 
                                                       min            max  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                
100001     1369693    1                        3951.000000    3951.000000  
                      2                        3951.000000    3951.000000  
                      3                        3951.000000    3951.000000  
                      4                       17397.900391   17397.900391  
           1851984    2                        3982.050049    3982.050049  
...                                                    ...            ...  
456255     2631384    23                      27489.689453   27489.689453  
                      24                     308277.312500  308277.312500  
           2729207    1                       11514.554688   11514.554688  
                      2                       11514.554688   11514.554688  
                      3                       42754.230469   42754.230469  

[12859104 rows x 7 columns]

Isolons les cas à version multiple (89583 résiduels après élimination des 299 outliers qui violaient la constance des règlements d'échéances d'une version à l'autre).

Cela nous permet d'obtenir nos nouveaux outliers, et de réutiliser les fonctions précédentes de nettoyage de l'ensemble de base :

In [5]:
outliers = aggregated[aggregated[("NUM_INSTALMENT_VERSION", "count")] > 1]
display(outliers)

NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100012     2480304    1                                          1   2     2   
100019     1620327    6                                          1   2     2   
100034     1390369    1                                          1   2     2   
100039     1077565    1                                          1   2     2   
                      2                                          3   4     2   
...                                                            ...  ..   ...   
456202     1333058    1                                          1   2     2   
456207     2230233    1                                          1   2     2   
456227     1772808    7                                          1   2     2   
456240     1508947    7                                          1   2     2   
456255     2631384    1                                          1   2     2   

                                            AMT_INSTALMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100012     2480304    1                        9482.355469   49204.890625   
100019     1620327    6                       10518.615234   33850.664062   
100034     1390369    1                       22430.429688   36069.570312   
100039     1077565    1                       52513.515625  172486.484375   
                      2                       46326.734375  178673.265625   
...                                                    ...            ...   
456202     1333058    1                        7147.890137   10852.110352   
456207     2230233    1                        4403.250000   36096.750000   
456227     1772808    7                       11813.580078   38804.941406   
456240     1508947    7                       16721.910156   18542.429688   
456255     2631384    1                       54022.140625  615229.500000   

                                               AMT_PAYMENT                 
                                                       min            max  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                
100012     2480304    1                       58687.246094   58687.246094  
100019     1620327    6                       44369.281250   44369.281250  
100034     1390369    1                       58500.000000   58500.000000  
100039     1077565    1                      225000.000000  225000.000000  
                      2                      225000.000000  225000.000000  
...                                                    ...            ...  
456202     1333058    1                       18000.000000   18000.000000  
456207     2230233    1                       40500.000000   40500.000000  
456227     1772808    7                       50618.519531   50618.519531  
456240     1508947    7                       35264.339844   35264.339844  
456255     2631384    1                      669251.625000  669251.625000  

[89583 rows x 7 columns]

Obtenons la version des données de base avec juste la dernière version de chaque échéance :

In [6]:
from home_credit.groupby import _get_clean_installments_payments_by_installment_and_version

last_version_base_data = _get_clean_installments_payments_by_installment_and_version(
    base_data,
    outliers
)

display(last_version_base_data)

DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
100001     1369693    1                     1                                  1709   
                      2                     1                                  1679   
                      3                     1                                  1649   
                      4                     2                                  1619   
           1851984    2                     1                                  2916   
...                                                                             ...   
456255     2631384    23                    3                                    96   
                      24                    4                                    66   
           2729207    1                     1                                   469   
                      2                     1                                   439   
                      3                     2                                   409   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
100001     1369693    1                     1                          3951.000000   
                      2                     1                          3951.000000   
                      3                     1                          3951.000000   
                      4                     2                         17397.900391   
           1851984    2                     1                          3982.050049   
...                                                                            ...   
456255     2631384    23                    3                         27489.689453   
                      24                    4                        308277.312500   
           2729207    1                     1                         11514.554688   
                      2                     1                         11514.554688   
                      3                     2                         42754.230469   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100001     1369693    1                     1                                 1   
                      2                     1                                 1   
                      3                     1                                 1   
                      4                     2                                 1   
           1851984    2                     1                                 1   
...                                                                         ...   
456255     2631384    23                    3                                 1   
                      24                    4                                 1   
           2729207    1                     1                                 1   
                      2                     1                                 1   
                      3                     2                                 1   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
100001     1369693    1                     1                                           1715   
                      2                     1                                           1715   
                      3                     1                                           1660   
                      4                     2                                           1628   
           1851984    2                     1                                           2916   
...                                                              

Vérifions qu'il n'y a plus qu'une version pour chaque échéance :

In [7]:
from home_credit.groupby import get_installments_payments_by_version

last_version_aggregated = get_installments_payments_by_version(last_version_base_data)
display(last_version_aggregated)

NUM_INSTALMENT_VERSION            \
                                                               min max count   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                    
100001     1369693    1                                          1   1     1   
                      2                                          1   1     1   
                      3                                          1   1     1   
                      4                                          2   2     1   
           1851984    2                                          1   1     1   
...                                                            ...  ..   ...   
456255     2631384    23                                         3   3     1   
                      24                                         4   4     1   
           2729207    1                                          1   1     1   
                      2                                          1   1     1   
                      3                                          2   2     1   

                                            AMT_INSTALMENT                 \
                                                       min            max   
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100001     1369693    1                        3951.000000    3951.000000   
                      2                        3951.000000    3951.000000   
                      3                        3951.000000    3951.000000   
                      4                       17397.900391   17397.900391   
           1851984    2                        3982.050049    3982.050049   
...                                                    ...            ...   
456255     2631384    23                      27489.689453   27489.689453   
                      24                     308277.312500  308277.312500   
           2729207    1                       11514.554688   11514.554688   
                      2                       11514.554688   11514.554688   
                      3                       42754.230469   42754.230469   

                                               AMT_PAYMENT                 
                                                       min            max  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                
100001     1369693    1                        3951.000000    3951.000000  
                      2                        3951.000000    3951.000000  
                      3                        3951.000000    3951.000000  
                      4                       17397.900391   17397.900391  
           1851984    2                        3982.050049    3982.050049  
...                                                    ...            ...  
456255     2631384    23                      27489.689453   27489.689453  
                      24                     308277.312500  308277.312500  
           2729207    1                       11514.554688   11514.554688  
                      2                       11514.554688   11514.554688  
                      3                       42754.230469   42754.230469  

[12859104 rows x 7 columns]

In [8]:
display(last_version_aggregated[last_version_aggregated[("NUM_INSTALMENT_VERSION", "count")] > 1])

Empty DataFrame
Columns: [(NUM_INSTALMENT_VERSION, min), (NUM_INSTALMENT_VERSION, max), (NUM_INSTALMENT_VERSION, count), (AMT_INSTALMENT, min), (AMT_INSTALMENT, max), (AMT_PAYMENT, min), (AMT_PAYMENT, max)]
Index: []

Nous pouvons à présent vérifier l'équilibre comptable, compte par compte, et disposer d'un état d'oustanding fiable :

In [9]:
base_data = last_version_base_data

base_data["AMT_OUTSTANDING"] = round(base_data.AMT_INSTALMENT - base_data.AMT_PAYMENT, 2)
pos_outstanding = base_data[base_data.AMT_OUTSTANDING >= 0.01]
neg_outstanding = base_data[base_data.AMT_OUTSTANDING <= -0.01]
n_samples = base_data.shape[0]
n_pos_outstanding = pos_outstanding.shape[0]
n_neg_outstanding = neg_outstanding.shape[0]

print(f"% of oustanding installments > 0: {100*n_pos_outstanding/n_samples:.2f} %")
print(f"% of oustanding installments < 0: {100*n_neg_outstanding/n_samples:.2f} %")

display(pos_outstanding)
display(neg_outstanding)

% of oustanding installments > 0: 0.02 %
% of oustanding installments < 0: 0.70 %


,,,,DAYS_INSTALMENT,AMT_INSTALMENT,CNT_PAYMENT,DAYS_ENTRY_PAYMENT_START,DAYS_ENTRY_PAYMENT_END,AMT_PAYMENT,AMT_OUTSTANDING
SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,,,,,,,
100104,2697044,10,1,2290,2917.530029,1,2299,2299,2892.330078,25.200001
100149,2523334,12,1,29,7669.484863,2,67,19,7668.944824,0.540000
100430,1096316,5,1,4,6632.640137,1,15,15,510.075012,6122.560059
100784,1925191,23,2,15,38727.046875,3,6,3,24548.445312,14178.599609
100807,1548219,22,1,10,48913.199219,1,20,20,35221.230469,13691.969727
...,...,...,...,...,...,...,...,...,...,...
456112,2073486,8,1,4,13312.214844,1,29,29,379.079987,12933.139648
456141,2481967,8,1,9,5969.294922,1,15,15,5339.609863,629.690002
456183,2391408,23,1,4,8606.700195,1,36,36,552.599976,8054.100098


DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
100012     2480304    1                     2                                   477   
100019     1620327    6                     2                                   744   
100034     1390369    1                     2                                   569   
100039     1077565    1                     2                                   665   
                      2                     4                                   635   
...                                                                             ...   
456202     1333058    1                     2                                   271   
456207     2230233    1                     2                                   115   
456227     1772808    7                     2                                   462   
456240     1508947    7                     2                                   612   
456255     2631384    1                     2                                   756   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
100012     2480304    1                     2                         49204.890625   
100019     1620327    6                     2                         33850.664062   
100034     1390369    1                     2                         36069.570312   
100039     1077565    1                     2                        172486.484375   
                      2                     4                        178673.265625   
...                                                                            ...   
456202     1333058    1                     2                         10852.110352   
456207     2230233    1                     2                          4403.250000   
456227     1772808    7                     2                         38804.941406   
456240     1508947    7                     2                         16721.910156   
456255     2631384    1                     2                        615229.500000   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100012     2480304    1                     2                                 1   
100019     1620327    6                     2                                 1   
100034     1390369    1                     2                                 1   
100039     1077565    1                     2                                 1   
                      2                     4                                 1   
...                                                                         ...   
456202     1333058    1                     2                                 1   
456207     2230233    1                     2                                 1   
456227     1772808    7                     2                                 1   
456240     1508947    7                     2                                 1   
456255     2631384    1                     2                                 1   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
100012     2480304    1                     2                                            487   
100019     1620327    6                     2                                            750   
100034     1390369    1                     2                                            584   
100039     1077565    1                     2                                            695   
                      2                     4                                            639   
...                                                              

Là, j'ai l'impression d'avoir raté quelque chose. Mais ce n'est pas certain.

Observons quelques cas :

In [13]:
from home_credit.tables import InstallmentsPayments

data = InstallmentsPayments.clean()

Sur ce premier cas, on constate que les deux versions de la première échéance équilibrent le paiement si elles sont additionnées :

In [12]:
display(data.loc[(100012, 2480304)])

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,16,0,2,477,487,49204.890625,58687.246094
1,16,0,1,477,487,9482.355469,58687.246094
2,15,0,3,447,456,5242.859863,5242.859863
3,14,0,3,417,455,5242.859863,5242.859863
4,13,0,3,387,428,5242.859863,5242.859863
5,12,0,3,357,399,5242.859863,5242.859863
6,11,0,3,327,367,5242.859863,5242.859863
7,10,0,3,297,327,5242.859863,5242.859863
8,9,0,3,267,307,5242.859863,5242.859863


Ici également, même s'il ne s'agit pas d'une échéance initiale (échéance 6) :

In [14]:
display(data.loc[(100019, 1620327)])

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,30,0,1,894,910,10518.615234,10518.615234
2,29,0,1,864,910,10518.615234,10518.615234
3,28,0,1,834,870,10518.615234,10518.615234
4,27,0,1,804,870,10518.615234,10518.615234
5,26,0,1,774,840,10518.615234,10518.615234
6,25,0,2,744,750,33850.664062,44369.281250
6,25,0,1,744,750,10518.615234,44369.281250
7,24,0,4,714,716,23761.394531,23761.394531


Il semblerait que mes hypothèses amont soient fausses et qu'en réalité, en cas de versions multiples, les montants des versions s'ajoutent, tandis qu'on ne conserve que le dernier montant de règlement.

In [15]:
display(data.loc[(100784, 1925191)])

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,23,1,1,675,705,38857.546875,38857.546875
2,22,1,1,645,655,38857.546875,38857.546875
3,21,1,1,615,614,38857.546875,38857.546875
4,20,1,1,585,585,38857.546875,38857.546875
5,19,1,1,555,561,38857.546875,38857.546875
6,18,1,2,525,528,38727.046875,38727.046875
7,17,1,2,495,494,38727.046875,38727.046875
8,16,1,2,465,466,38727.046875,38727.046875
9,15,1,2,435,436,38727.046875,38727.046875


## Retour en arrière

Partant de la version de base, il n'y aurait donc, selon la nouvelle hypothèse, qu'à agréger en effectuant les sommes de montants d'échéance et en ne conservant que le dernier montant de règlement :

In [16]:
from home_credit.tables import InstallmentsPayments
from home_credit.groupby import get_installments_payments_by_installment_and_version

data = InstallmentsPayments.clean()
base_data = get_installments_payments_by_installment_and_version(data)
display(base_data)

DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
100001     1369693    1                     1                                  1709   
                      2                     1                                  1679   
                      3                     1                                  1649   
                      4                     2                                  1619   
           1851984    2                     1                                  2916   
...                                                                             ...   
456255     2631384    23                    3                                    96   
                      24                    4                                    66   
           2729207    1                     1                                   469   
                      2                     1                                   439   
                      3                     2                                   409   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
100001     1369693    1                     1                          3951.000000   
                      2                     1                          3951.000000   
                      3                     1                          3951.000000   
                      4                     2                         17397.900391   
           1851984    2                     1                          3982.050049   
...                                                                            ...   
456255     2631384    23                    3                         27489.689453   
                      24                    4                        308277.312500   
           2729207    1                     1                         11514.554688   
                      2                     1                         11514.554688   
                      3                     2                         42754.230469   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
100001     1369693    1                     1                                 1   
                      2                     1                                 1   
                      3                     1                                 1   
                      4                     2                                 1   
           1851984    2                     1                                 1   
...                                                                         ...   
456255     2631384    23                    3                                 1   
                      24                    4                                 1   
           2729207    1                     1                                 1   
                      2                     1                                 1   
                      3                     2                                 1   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
100001     1369693    1                     1                                           1715   
                      2                     1                                           1715   
                      3                     1                                           1660   
                      4                     2                                           1628   
           1851984    2                     1                                           2916   
...                                                              

De là, on réalise une agrégation par les versions qui somme au lieu de ne conserver que les cas d'extrémités :

In [23]:
import pandas as pd

def get_installments_payments_by_version_v2(
    data: pd.DataFrame
) -> pd.DataFrame:
    # Reset the index for grouping
    data = data.reset_index()

    # Group the data by relevant columns
    data = data.groupby(by=[
        "SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER"
    ])

    # Perform and return aggregation
    return data.agg({
        "NUM_INSTALMENT_VERSION" : ["min", "max", "count"],
        "AMT_INSTALMENT" : "sum",
        "AMT_PAYMENT": "last"
    })


aggregated_version = get_installments_payments_by_version_v2(base_data)

aggregated_version.columns = [
    "V_MIN", "V_MAX", "V_COUNT", "AMT_INSTALMENT", "AMT_PAYMENT"
]

display(aggregated_version)

V_MIN  V_MAX  V_COUNT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
100001     1369693    1                          1      1        1   
                      2                          1      1        1   
                      3                          1      1        1   
                      4                          2      2        1   
           1851984    2                          1      1        1   
...                                            ...    ...      ...   
456255     2631384    23                         3      3        1   
                      24                         4      4        1   
           2729207    1                          1      1        1   
                      2                          1      1        1   
                      3                          2      2        1   

                                             AMT_INSTALMENT    AMT_PAYMENT  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100001     1369693    1                         3951.000000    3951.000000  
                      2                         3951.000000    3951.000000  
                      3                         3951.000000    3951.000000  
                      4                        17397.900391   17397.900391  
           1851984    2                         3982.050049    3982.050049  
...                                                     ...            ...  
456255     2631384    23                       27489.689453   27489.689453  
                      24                      308277.312500  308277.312500  
           2729207    1                        11514.554688   11514.554688  
                      2                        11514.554688   11514.554688  
                      3                        42754.230469   42754.230469  

[12859104 rows x 5 columns]

Retour sur l'outstanding, cette fois-ci sur ce second niveau d'agrégation simple qui fait abstraction des versions.

On se rapproche d'un équilibre global intéressant.

En outre, on tombe sur un cas d'outlier surprenant du côté des outstandings négatifs (on règle un seuil de tolérance `tol` pour évacuer les écarts de quelques centimes) : sur 219 cas, 217 correspondent à une absence d'information sur les montants d'échéance (ils sont à 0).

In [32]:
# base_data = last_version_base_data

aggregated_version["AMT_OUTSTANDING"] = round(
    aggregated_version.AMT_INSTALMENT
    - aggregated_version.AMT_PAYMENT,
2)

tol_pos = 0.1
tol_neg = 0.5

pos_outstanding = aggregated_version[aggregated_version.AMT_OUTSTANDING >= tol_pos]
neg_outstanding = aggregated_version[aggregated_version.AMT_OUTSTANDING <= -tol_neg]

n_samples = base_data.shape[0]
n_pos_outstanding = pos_outstanding.shape[0]
n_neg_outstanding = neg_outstanding.shape[0]

print(f"% of oustanding installments > 0: {100*n_pos_outstanding/n_samples:.2f} %")
print(f"% of oustanding installments < 0: {100*n_neg_outstanding/n_samples:.2f} %")

display(pos_outstanding)
print(f"# neg.AMT_INSTALMENT nuls = {(neg_outstanding.AMT_INSTALMENT == 0).sum()}")
display(neg_outstanding)
display(neg_outstanding[neg_outstanding.AMT_INSTALMENT > 0])

% of oustanding installments > 0: 0.03 %
% of oustanding installments < 0: 0.00 %


,,,V_MIN,V_MAX,V_COUNT,AMT_INSTALMENT,AMT_PAYMENT,AMT_OUTSTANDING
SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,,,,,,
100104,2697044,10,1,1,1,2917.530029,2892.330078,25.200001
100149,2523334,12,1,1,1,7669.484863,7668.944824,0.540000
100430,1096316,5,1,1,1,6632.640137,510.075012,6122.560059
100784,1925191,23,2,2,1,38727.046875,24548.445312,14178.599609
100807,1548219,22,1,1,1,48913.199219,35221.230469,13691.969727
...,...,...,...,...,...,...,...,...
456112,2073486,8,1,1,1,13312.214844,379.079987,12933.139648
456141,2481967,8,1,1,1,5969.294922,5339.609863,629.690002
456183,2391408,23,1,1,1,8606.700195,552.599976,8054.100098


# neg.AMT_INSTALMENT nuls = 217


V_MIN  V_MAX  V_COUNT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
106652     1171606    8                          1      1        1   
119813     2184232    9                          1      1        1   
                      10                         1      1        1   
170117     1822101    1                          1      1        1   
                      2                          1      1        1   
...                                            ...    ...      ...   
453803     1071426    6                          1      1        1   
                      7                          1      1        1   
                      8                          1      1        1   
                      9                          1      1        1   
                      10                         1      1        1   

                                             AMT_INSTALMENT   AMT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
106652     1171606    8                                 0.0   5260.724609   
119813     2184232    9                                 0.0   2746.709961   
                      10                                0.0   2719.619873   
170117     1822101    1                                 0.0  11431.620117   
                      2                                 0.0  11431.620117   
...                                                     ...           ...   
453803     1071426    6                                 0.0  19179.585938   
                      7                                 0.0  19179.585938   
                      8                                 0.0  19179.585938   
                      9                                 0.0  19179.585938   
                      10                                0.0  18934.558594   

                                             AMT_OUTSTANDING  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                   
106652     1171606    8                         -5260.720215  
119813     2184232    9                         -2746.709961  
                      10                        -2719.620117  
170117     1822101    1                        -11431.620117  
                      2                        -11431.620117  
...                                                      ...  
453803     1071426    6                        -19179.589844  
                      7                        -19179.589844  
                      8                        -19179.589844  
                      9                        -19179.589844  
                      10                       -18934.560547  

[219 rows x 6 columns]

,,,V_MIN,V_MAX,V_COUNT,AMT_INSTALMENT,AMT_PAYMENT,AMT_OUTSTANDING
SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,,,,,,
255002,1783368,137,0,0,1,9856.980469,11647.980469,-1791.000000
262837,1877030,51,1,2,2,304193.781250,343790.687500,-39596.898438


In [33]:
display(data.loc[(255002, 1783368)])

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,86,0,0,2595,2609,6750.000000,6750.000000
2,86,0,0,2609,2609,2250.000000,2250.000000
3,86,0,0,2605,2605,9000.000000,9000.000000
4,85,0,0,2564,2573,6750.000000,6750.000000
5,85,0,0,2573,2573,15750.000000,15750.000000
...,...,...,...,...,...,...,...
187,2,0,0,55,55,20022.210938,20022.210938
188,1,0,0,18,34,5930.009766,5930.009766
189,2,0,0,34,34,3069.989990,3069.989990


In [36]:
display(data.loc[(262837, 1877030)])

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,75,0,1,2266,2273,0.000000,39596.894531
2,74,0,1,2236,2244,0.000000,39596.894531
3,73,0,1,2206,2213,0.000000,39596.894531
4,72,0,1,2176,2181,0.000000,39596.894531
5,71,0,1,2146,2153,0.000000,39596.894531
...,...,...,...,...,...,...,...
59,18,0,3,526,539,14997.150391,14997.150391
60,17,0,3,496,509,14997.150391,14997.150391
61,16,0,3,466,480,14997.150391,14997.150391


In [35]:
display(data.loc[(106652, 1171606)])

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
8,97,0,1,2922,467,0.0,6.300000
8,97,0,1,2922,2948,0.0,5254.424805


In [28]:
display(data.loc[(453803, 1071426)])

CLEAN_INSTALLMENTS_PAYMENTS,MONTHS_BALANCE,TARGET,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
NUM_INSTALMENT_NUMBER,,,,,,,
1,76,0,1,2293,2303,0.0,19179.585938
2,75,0,1,2263,2280,0.0,19179.585938
3,74,0,1,2233,2238,0.0,19179.585938
4,73,0,1,2203,2206,0.0,19179.585938
5,72,0,1,2173,2179,0.0,19179.585938
6,71,0,1,2143,2150,0.0,19179.585938
7,70,0,1,2113,2119,0.0,19179.585938
8,69,0,1,2083,2098,0.0,19179.585938
9,68,0,1,2053,2065,0.0,19179.585938


## Cas des échéances NA codées 0

Nous avons donc identifié un cas de données manquantes, mais codées à 0, ce qui est une très mauvaise pratique en l'espèce.

Identifions tous les cas ce sont exactement les 219 cas vus plus haut :

In [39]:
na_inst_case = aggregated_version[aggregated_version.AMT_INSTALMENT == 0]
display(na_inst_case)

V_MIN  V_MAX  V_COUNT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
106652     1171606    8                          1      1        1   
119813     2184232    9                          1      1        1   
                      10                         1      1        1   
170117     1822101    1                          1      1        1   
                      2                          1      1        1   
...                                            ...    ...      ...   
453803     1071426    6                          1      1        1   
                      7                          1      1        1   
                      8                          1      1        1   
                      9                          1      1        1   
                      10                         1      1        1   

                                             AMT_INSTALMENT   AMT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
106652     1171606    8                                 0.0   5260.724609   
119813     2184232    9                                 0.0   2746.709961   
                      10                                0.0   2719.619873   
170117     1822101    1                                 0.0  11431.620117   
                      2                                 0.0  11431.620117   
...                                                     ...           ...   
453803     1071426    6                                 0.0  19179.585938   
                      7                                 0.0  19179.585938   
                      8                                 0.0  19179.585938   
                      9                                 0.0  19179.585938   
                      10                                0.0  18934.558594   

                                             AMT_OUTSTANDING  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                   
106652     1171606    8                         -5260.720215  
119813     2184232    9                         -2746.709961  
                      10                        -2719.620117  
170117     1822101    1                        -11431.620117  
                      2                        -11431.620117  
...                                                      ...  
453803     1071426    6                        -19179.589844  
                      7                        -19179.589844  
                      8                        -19179.589844  
                      9                        -19179.589844  
                      10                       -18934.560547  

[219 rows x 6 columns]

Aucun d'entre eux n'est à version multiple. Cela signifie que l'on peut effectuer la correction qui consiste à reporter le montant du règlement dès le niveau des données de base.

In [40]:
display(na_inst_case[na_inst_case.V_COUNT > 1])

,,,V_MIN,V_MAX,V_COUNT,AMT_INSTALMENT,AMT_PAYMENT,AMT_OUTSTANDING
SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,,,,,,


Simulons cette modification de la chaîne amont, pour nous aider à mettre à jour le code de production :

In [41]:
display(base_data[base_data.AMT_INSTALMENT == 0])

DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
106652     1171606    8                     1                                  2922   
119813     2184232    9                     1                                  2921   
                      10                    1                                  2891   
170117     1822101    1                     1                                  2205   
                      2                     1                                  2175   
...                                                                             ...   
453803     1071426    6                     1                                  2143   
                      7                     1                                  2113   
                      8                     1                                  2083   
                      9                     1                                  2053   
                      10                    1                                  2023   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
106652     1171606    8                     1                                  0.0   
119813     2184232    9                     1                                  0.0   
                      10                    1                                  0.0   
170117     1822101    1                     1                                  0.0   
                      2                     1                                  0.0   
...                                                                            ...   
453803     1071426    6                     1                                  0.0   
                      7                     1                                  0.0   
                      8                     1                                  0.0   
                      9                     1                                  0.0   
                      10                    1                                  0.0   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
106652     1171606    8                     1                                 2   
119813     2184232    9                     1                                 1   
                      10                    1                                 2   
170117     1822101    1                     1                                 1   
                      2                     1                                 1   
...                                                                         ...   
453803     1071426    6                     1                                 1   
                      7                     1                                 1   
                      8                     1                                 1   
                      9                     1                                 1   
                      10                    1                                 2   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
106652     1171606    8                     1                                           2948   
119813     2184232    9                     1                                           2921   
                      10                    1                                           2890   
170117     1822101    1                     1                                           2211   
                      2                     1                                           2183   
...                                                              

Petite vérification rapide, car 221 != 219. Cela confirme l'unicité des versions.

**Note** pour un éventuel relecteur attentif, y compris moi, au cas où : la différence de 2 vient du fait que deux échéances sont nulles, et pour le montant d'échéance et pour le montant de règlement.

In [49]:
x = base_data[base_data.AMT_INSTALMENT == 0]
x = x.reset_index()
display(x)
x = x[x.columns[:4]]
display(x)
# Group the data by relevant columns
y = x.groupby(by=[
    "SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER"
])
# Perform and return aggregation
y = y.agg({
    "NUM_INSTALMENT_VERSION" : ["min", "max", "count"]
})
display(y[y[("NUM_INSTALMENT_VERSION", "count")] > 1])

,SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,AMT_INSTALMENT,CNT_PAYMENT,DAYS_ENTRY_PAYMENT_START,DAYS_ENTRY_PAYMENT_END,AMT_PAYMENT
0,106652,1171606,8,1,2922,0.0,2,2948,467,5260.724609
1,119813,2184232,9,1,2921,0.0,1,2921,2921,2746.709961
2,119813,2184232,10,1,2891,0.0,2,2890,684,2719.619873
3,170117,1822101,1,1,2205,0.0,1,2211,2211,11431.620117
4,170117,1822101,2,1,2175,0.0,1,2183,2183,11431.620117
...,...,...,...,...,...,...,...,...,...,...
216,453803,1071426,6,1,2143,0.0,1,2150,2150,19179.585938
217,453803,1071426,7,1,2113,0.0,1,2119,2119,19179.585938
218,453803,1071426,8,1,2083,0.0,1,2098,2098,19179.585938
219,453803,1071426,9,1,2053,0.0,1,2065,2065,19179.585938


,SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION
0,106652,1171606,8,1
1,119813,2184232,9,1
2,119813,2184232,10,1
3,170117,1822101,1,1
4,170117,1822101,2,1
...,...,...,...,...
216,453803,1071426,6,1
217,453803,1071426,7,1
218,453803,1071426,8,1
219,453803,1071426,9,1


Empty DataFrame
Columns: [(NUM_INSTALMENT_VERSION, min), (NUM_INSTALMENT_VERSION, max), (NUM_INSTALMENT_VERSION, count)]
Index: []

Correction des données :

In [51]:
na_inst_case = base_data.AMT_INSTALMENT == 0
base_data.loc[na_inst_case, "AMT_INSTALMENT"] = base_data[na_inst_case].AMT_PAYMENT

display(base_data[na_inst_case])

DAYS_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                    
106652     1171606    8                     1                                  2922   
119813     2184232    9                     1                                  2921   
                      10                    1                                  2891   
170117     1822101    1                     1                                  2205   
                      2                     1                                  2175   
...                                                                             ...   
453803     1071426    6                     1                                  2143   
                      7                     1                                  2113   
                      8                     1                                  2083   
                      9                     1                                  2053   
                      10                    1                                  2023   

                                                                    AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                   
106652     1171606    8                     1                          5260.724609   
119813     2184232    9                     1                          2746.709961   
                      10                    1                          2719.619873   
170117     1822101    1                     1                         11431.620117   
                      2                     1                         11431.620117   
...                                                                            ...   
453803     1071426    6                     1                         19179.585938   
                      7                     1                         19179.585938   
                      8                     1                         19179.585938   
                      9                     1                         19179.585938   
                      10                    1                         18934.558594   

                                                                    CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                
106652     1171606    8                     1                                 2   
119813     2184232    9                     1                                 1   
                      10                    1                                 2   
170117     1822101    1                     1                                 1   
                      2                     1                                 1   
...                                                                         ...   
453803     1071426    6                     1                                 1   
                      7                     1                                 1   
                      8                     1                                 1   
                      9                     1                                 1   
                      10                    1                                 2   

                                                                    DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER NUM_INSTALMENT_VERSION                             
106652     1171606    8                     1                                           2948   
119813     2184232    9                     1                                           2921   
                      10                    1                                           2890   
170117     1822101    1                     1                                           2211   
                      2                     1                                           2183   
...                                                              

Retour à l'agrégation des versions :

In [52]:
import pandas as pd

def get_installments_payments_by_version_v2(
    data: pd.DataFrame
) -> pd.DataFrame:
    # Reset the index for grouping
    data = data.reset_index()

    # Group the data by relevant columns
    data = data.groupby(by=[
        "SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER"
    ])

    # Perform and return aggregation
    return data.agg({
        "NUM_INSTALMENT_VERSION" : ["min", "max", "count"],
        "AMT_INSTALMENT" : "sum",
        "AMT_PAYMENT": "last"
    })


aggregated_version = get_installments_payments_by_version_v2(base_data)

aggregated_version.columns = [
    "V_MIN", "V_MAX", "V_COUNT", "AMT_INSTALMENT", "AMT_PAYMENT"
]

display(aggregated_version)

V_MIN  V_MAX  V_COUNT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
100001     1369693    1                          1      1        1   
                      2                          1      1        1   
                      3                          1      1        1   
                      4                          2      2        1   
           1851984    2                          1      1        1   
...                                            ...    ...      ...   
456255     2631384    23                         3      3        1   
                      24                         4      4        1   
           2729207    1                          1      1        1   
                      2                          1      1        1   
                      3                          2      2        1   

                                             AMT_INSTALMENT    AMT_PAYMENT  
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100001     1369693    1                         3951.000000    3951.000000  
                      2                         3951.000000    3951.000000  
                      3                         3951.000000    3951.000000  
                      4                        17397.900391   17397.900391  
           1851984    2                         3982.050049    3982.050049  
...                                                     ...            ...  
456255     2631384    23                       27489.689453   27489.689453  
                      24                      308277.312500  308277.312500  
           2729207    1                        11514.554688   11514.554688  
                      2                        11514.554688   11514.554688  
                      3                        42754.230469   42754.230469  

[12859104 rows x 5 columns]

Retour à l'outstanding : il ne reste plus qu'un cas unique négatif.

En tout, 3 250 cas d'outstanding ressortent, soit 3 cas pour 10 000.

In [53]:
aggregated_version["AMT_OUTSTANDING"] = round(
    aggregated_version.AMT_INSTALMENT
    - aggregated_version.AMT_PAYMENT,
2)

tol_pos = 0.1
tol_neg = 0.5

pos_outstanding = aggregated_version[aggregated_version.AMT_OUTSTANDING >= tol_pos]
neg_outstanding = aggregated_version[aggregated_version.AMT_OUTSTANDING <= -tol_neg]

n_samples = base_data.shape[0]
n_pos_outstanding = pos_outstanding.shape[0]
n_neg_outstanding = neg_outstanding.shape[0]

print(f"% of oustanding installments > 0: {100*n_pos_outstanding/n_samples:.2f} %")
print(f"% of oustanding installments < 0: {100*n_neg_outstanding/n_samples:.2f} %")

display(pos_outstanding)
print(f"# neg.AMT_INSTALMENT nuls = {(neg_outstanding.AMT_INSTALMENT == 0).sum()}")
display(neg_outstanding)
display(neg_outstanding[neg_outstanding.AMT_INSTALMENT > 0])

% of oustanding installments > 0: 0.03 %
% of oustanding installments < 0: 0.00 %


,,,V_MIN,V_MAX,V_COUNT,AMT_INSTALMENT,AMT_PAYMENT,AMT_OUTSTANDING
SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,,,,,,
100104,2697044,10,1,1,1,2917.530029,2892.330078,25.200001
100149,2523334,12,1,1,1,7669.484863,7668.944824,0.540000
100430,1096316,5,1,1,1,6632.640137,510.075012,6122.560059
100784,1925191,23,2,2,1,38727.046875,24548.445312,14178.599609
100807,1548219,22,1,1,1,48913.199219,35221.230469,13691.969727
...,...,...,...,...,...,...,...,...
456112,2073486,8,1,1,1,13312.214844,379.079987,12933.139648
456141,2481967,8,1,1,1,5969.294922,5339.609863,629.690002
456183,2391408,23,1,1,1,8606.700195,552.599976,8054.100098


# neg.AMT_INSTALMENT nuls = 0


,,,V_MIN,V_MAX,V_COUNT,AMT_INSTALMENT,AMT_PAYMENT,AMT_OUTSTANDING
SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,,,,,,
255002,1783368,137,0,0,1,9856.980469,11647.980469,-1791.0


,,,V_MIN,V_MAX,V_COUNT,AMT_INSTALMENT,AMT_PAYMENT,AMT_OUTSTANDING
SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,,,,,,
255002,1783368,137,0,0,1,9856.980469,11647.980469,-1791.0


## Agrégation mensuelle

### Table de départ

In [1]:
from home_credit.tables import InstallmentsPayments

data_by_inst = InstallmentsPayments.by_installment()
display(data_by_inst)

V_MIN  V_MAX  V_COUNT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
100001     1369693    1                          1      1        1   
                      2                          1      1        1   
                      3                          1      1        1   
                      4                          2      2        1   
           1851984    2                          1      1        1   
...                                            ...    ...      ...   
456255     2631384    23                         3      3        1   
                      24                         4      4        1   
           2729207    1                          1      1        1   
                      2                          1      1        1   
                      3                          2      2        1   

                                             AMT_INSTALMENT    AMT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                  
100001     1369693    1                         3951.000000    3951.000000   
                      2                         3951.000000    3951.000000   
                      3                         3951.000000    3951.000000   
                      4                        17397.900391   17397.900391   
           1851984    2                         3982.050049    3982.050049   
...                                                     ...            ...   
456255     2631384    23                       27489.689453   27489.689453   
                      24                      308277.312500  308277.312500   
           2729207    1                        11514.554688   11514.554688   
                      2                        11514.554688   11514.554688   
                      3                        42754.230469   42754.230469   

                                             DAYS_INSTALMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
100001     1369693    1                                       1709   
                      2                                       1679   
                      3                                       1649   
                      4                                       1619   
           1851984    2                                       2916   
...                                                            ...   
456255     2631384    23                                        96   
                      24                                        66   
           2729207    1                                        469   
                      2                                        439   
                      3                                        409   

                                             DAYS_INSTALMENT_END  CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                     
100001     1369693    1                                     1709            1   
                      2                                     1679            1   
                      3                                     1649            1   
                      4                                     1619            1   
           1851984    2                                     2916            1   
...                                                          ...          ...   
456255     2631384    23                                      96            1   
                      24                                      66            1   
           2729207    1                                      469            1   
                      2                                      439            1   
                      3                                      409            1   

                                             DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                             
100001  

### Lignes où `DAYS_INSTALMENT_START != DAYS_INSTALMENT_END` : 302 cas.

Le plus grand écart est de 29 jours, donc l'écart en mois est toujours nul, ce qui est une bonne nouvelle.

Cela signifie que pour l'agrégation mensuelle, nous n'avons qu'une seule information et non pas 3.

In [5]:
display(data_by_inst[
    data_by_inst.DAYS_INSTALMENT_START
    != data_by_inst.DAYS_INSTALMENT_END
])

diff = data_by_inst.DAYS_INSTALMENT_START - data_by_inst.DAYS_INSTALMENT_END
print(f"max diff: {diff.max()}")
display(diff.value_counts()[:5])

,,,V_MIN,V_MAX,V_COUNT,AMT_INSTALMENT,AMT_PAYMENT,DAYS_INSTALMENT_START,DAYS_INSTALMENT_END,CNT_PAYMENT,DAYS_ENTRY_PAYMENT_START,DAYS_ENTRY_PAYMENT_END
SK_ID_CURR,SK_ID_PREV,NUM_INSTALMENT_NUMBER,,,,,,,,,,
101080,2290883,1,2,3,2,33798.871094,11298.870117,14,9,2,14,8
101378,1133646,1,2,3,2,40643.867188,9143.865234,132,126,2,132,125
101826,2010823,1,2,3,2,51092.460938,6092.459961,209,196,2,209,209
102206,1096061,1,2,3,2,17968.769531,5368.770020,168,154,2,175,170
102731,1175528,1,2,3,2,61128.765625,16128.764648,208,179,2,208,193
...,...,...,...,...,...,...,...,...,...,...,...,...
443189,1277527,1,2,3,2,29120.714844,6620.715332,106,102,3,106,100
445335,2207258,1,2,3,2,59576.851562,14576.849609,91,76,2,91,79
449589,1265758,1,2,3,2,21932.730469,3932.729980,75,64,2,75,66


max diff: 29


0     12858802
10          21
11          20
13          16
9           16
dtype: int64

### Lignes où `DAYS_ENTRY_PAYMENT_START != DAYS_ENTRY_PAYMENT_END` : 640 382 cas.

Ce sont les DPD, qui s’égrènent sur différentes durées.

Attention, le DPD signifie qu'il existe un reliquat de dette. Il continuera a courir si par exemple la dette est réglée à 99 %.

C'est ce qui implique qu'il faudrait à nouveau faire un pas en arrière... pour pouvoir établir, pour une échéance donnée, la quote part mensuelle réglée jusqu'à extinction de la dette. Cela dégage un profil pertinent de promptitude à honorer sa dette.

In [9]:
display(data_by_inst[
    data_by_inst.DAYS_ENTRY_PAYMENT_START
    != data_by_inst.DAYS_ENTRY_PAYMENT_END
])

diff = data_by_inst.DAYS_ENTRY_PAYMENT_START - data_by_inst.DAYS_ENTRY_PAYMENT_END
print(f"max diff: {diff.max()}")
print(f"n DPD: {(diff > 0).sum()}")
print(f"n DPD > 30: {(diff > 30).sum()}")
print(f"n DPD > 60: {(diff > 60).sum()}")
print(f"n DPD > 90: {(diff > 90).sum()}")
print(f"n DPD > 120: {(diff > 120).sum()}")

display(diff.value_counts()[:5])

V_MIN  V_MAX  V_COUNT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
100007     1940724    1                          1      1        1   
           2462742    10                         1      1        1   
                      12                         1      1        1   
100008     1907290    10                         1      1        1   
100011     1430602    9                          1      1        1   
...                                            ...    ...      ...   
456255     2073384    2                          1      1        1   
           2631384    17                         3      3        1   
                      18                         3      3        1   
                      20                         3      3        1   
                      22                         3      3        1   

                                             AMT_INSTALMENT   AMT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                 
100007     1940724    1                        22678.785156  22678.785156   
           2462742    10                        3601.530029   3601.530029   
                      12                        3577.050049   3577.050049   
100008     1907290    10                       11986.155273  11986.155273   
100011     1430602    9                        14614.334961  14614.334961   
...                                                     ...           ...   
456255     2073384    2                        16400.609375  16400.609375   
           2631384    17                       27489.689453  27489.689453   
                      18                       27489.689453  27489.691406   
                      20                       27489.689453  27489.691406   
                      22                       27489.689453  27489.689453   

                                             DAYS_INSTALMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
100007     1940724    1                                       1076   
           2462742    10                                      1466   
                      12                                      1406   
100008     1907290    10                                      2029   
100011     1430602    9                                       2237   
...                                                            ...   
456255     2073384    2                                        588   
           2631384    17                                       276   
                      18                                       246   
                      20                                       186   
                      22                                       126   

                                             DAYS_INSTALMENT_END  CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                     
100007     1940724    1                                     1076            2   
           2462742    10                                    1466            2   
                      12                                    1406            2   
100008     1907290    10                                    2029            2   
100011     1430602    9                                     2237            2   
...                                                          ...          ...   
456255     2073384    2                                      588            2   
           2631384    17                                     276            2   
                      18                                     246            2   
                      20                                     186            2   
                      22                                     126            2   

                                             DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                             
100007     1940724   

max diff: 2902
n DPD: 640382
n DPD > 30: 346186
n DPD > 60: 21758
n DPD > 90: 9837
n DPD > 120: 7652


0     12218722
32       46544
31       42392
33       40315
30       35892
dtype: int64

Combien de paiements ?

In [10]:
display(data_by_inst.CNT_PAYMENT.value_counts())

1     12128485
2       718875
3        11001
4          586
6           69
5           68
8           10
7            7
10           1
9            1
12           1
Name: CNT_PAYMENT, dtype: int64

De quelle date démarre le DPD ? Quel diff entre les DAYS_INSTALMENT et DAYS_ENTRY_PAYMENT_START ?

La réalité, c'est qu'une date d'échéance correspond à une date d'exigibilité d'une dette.

Le DPD commence donc à courir dès la data d'échéance. Cela est confirmé par 9 720 208 cas, majoritaires, où la dette est soldée à l'échéance.

In [11]:
display(data_by_inst[
    data_by_inst.DAYS_ENTRY_PAYMENT_START
    != data_by_inst.DAYS_INSTALMENT_START
])

diff = data_by_inst.DAYS_ENTRY_PAYMENT_START - data_by_inst.DAYS_INSTALMENT_START
print(f"max diff: {diff.max()}")
print(f"n DPD: {(diff > 0).sum()}")
print(f"n DPD > 30: {(diff > 30).sum()}")
print(f"n DPD > 60: {(diff > 60).sum()}")
print(f"n DPD > 90: {(diff > 90).sum()}")
print(f"n DPD > 120: {(diff > 120).sum()}")

display(diff.value_counts()[:5])

V_MIN  V_MAX  V_COUNT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
100001     1369693    1                          1      1        1   
                      2                          1      1        1   
                      3                          1      1        1   
                      4                          2      2        1   
           1851984    3                          1      1        1   
...                                            ...    ...      ...   
456255     2631384    23                         3      3        1   
                      24                         4      4        1   
           2729207    1                          1      1        1   
                      2                          1      1        1   
                      3                          2      2        1   

                                             AMT_INSTALMENT    AMT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                  
100001     1369693    1                         3951.000000    3951.000000   
                      2                         3951.000000    3951.000000   
                      3                         3951.000000    3951.000000   
                      4                        17397.900391   17397.900391   
           1851984    3                         3982.050049    3982.050049   
...                                                     ...            ...   
456255     2631384    23                       27489.689453   27489.689453   
                      24                      308277.312500  308277.312500   
           2729207    1                        11514.554688   11514.554688   
                      2                        11514.554688   11514.554688   
                      3                        42754.230469   42754.230469   

                                             DAYS_INSTALMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                          
100001     1369693    1                                       1709   
                      2                                       1679   
                      3                                       1649   
                      4                                       1619   
           1851984    3                                       2886   
...                                                            ...   
456255     2631384    23                                        96   
                      24                                        66   
           2729207    1                                        469   
                      2                                        439   
                      3                                        409   

                                             DAYS_INSTALMENT_END  CNT_PAYMENT  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                                     
100001     1369693    1                                     1709            1   
                      2                                     1679            1   
                      3                                     1649            1   
                      4                                     1619            1   
           1851984    3                                     2886            1   
...                                                          ...          ...   
456255     2631384    23                                      96            1   
                      24                                      66            1   
           2729207    1                                      469            1   
                      2                                      439            1   
                      3                                      409            1   

                                             DAYS_ENTRY_PAYMENT_START  \
SK_ID_CURR SK_ID_PREV NUM_INSTALMENT_NUMBER                             
100001  

max diff: 65535
n DPD: 9720208
n DPD > 30: 1270840
n DPD > 60: 659909
n DPD > 90: 571123
n DPD > 120: 544826


0    3138896
1     517149
2     499831
3     452884
9     448364
dtype: int64

## Synthèse des DPD et indice de risque avancé

Une synthèse mensuelle par prêt, puis par client, devrait donc établir, mois par mois, la quantité de dette par tranches de DPD.

Comment construire une telle synthèse ?

    Pour chaque échéance.version
        Pour chaque règlement partiel effectué
            conserver le montant
            calculer le DPD
            pour info calculer le % de dette soldé
            pour info calculer le % de dette résiduelle

On dispose ainsi d'une table des niveaux de maturité des dettes honorées.

On peut réduire la liste des DPD aux valeurs uniques rencontrées, ou bien faire une synthèse par mois, ou tout grain de temps.

Partant de ces valeurs uniques, on repasse sur la table des niveaux de maturité, et on effectue la somme des montants réglés : cela nous donne une table d'association des parties de la dette principale réglés avec telle ou telle maturité.

A l'instant de cette synthèse, la différence entre la somme des échéances et celle des règlements donne la part de dette restant due. Son % peut être déterminé, mais pas son DPD, pour lequel on ne peut que fixer une borne inférieure (c'est le maximum des DPD connus).

En termes de traitements pandas, cela revient à :
1. Calcul des DPD (unitaires, mensuels, .. au choix, mais mensuels cf. notre objectif) sur la table clean (en amont de la table de base).
2. Grouper par (prêt, DPD)
3. agréger les montants réglés par somme

En sortie, on obtient, pour chaque prêt, la liste des montants réglés par DPD, abstraction faite des échéances. Si on détermine par ailleurs la somme des échéances sur chaque prêt, on connaît le montant de la partie devenue exigible du prêt et on peut donc établir pour chaque montant.maturité réglé, le % qu'il représente du prêt, ainsi que le % de la dette exigible qui n'a toujours pas été honorée..

Anticipons sur la pertinence de cette information pour le modèle.

Un client parfait aura une dette, qu'elle soit en cours ou soldée intégralement, dont le DPD sera de 0 pour 100% des montants réglés.

Un client qui a quelques retards, ponctuels ou récurrents, mais qu'il honore rapidement aura par exemple 80% des montants réglés en DPD 0, 20% en DPD 1.

En revanche, un mauvais client aura un % faible de DPD faible, et une distribution centrée sur des DPD élevés.

Cela fait penser à une espérance.

On peut imaginer synthétiser ainsi l'information pour la passer au modèle :

somme des % par DPD x DPD

par exemple, le client parfait aura une espérance de 0

plus le client est mauvais plus sont espérance de DPD sera importante.

Avec une telle formule, on peut se passer de la synthèse mensuelle qui introduira du bruit et obtenir un indice synthétique pertinent à propos du risque client.

Rien n'empêche de la faire à côté en vue des fusions de tables sur la base des mensualités.

Pour un client donné, tous prêts confondus, il suffit de procéder de la même manière, cette fois-ci en considérant les dettes réglées et leur DPD, tous prêts confondus.